In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchaudio
from os import path, walk
import torch.nn as nn
from IPython.display import Audio, display
from tqdm import tqdm
# import sys
# sys.path.insert(0, '../../src')
# import sofa
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/My Drive/all_samples_500ms.zip" -d "/content"

In [ ]:
def data_loader(random_sampling_file):
    data = list()
    f = open(random_sampling_file, "r")
    for line in f:
        file, az, el = line.split(" ")
        data.append((az, el.removesuffix("\n"), file))
    f.close()
    return data

In [ ]:
class RNN(nn.Module):
    def __init__(self, n_observations, n_angles):
        super(RNN, self).__init__()
        self.n_hidden = 1000
        self.gru1 = nn.GRU(n_observations, 256, 1, batch_first=True, bidirectional=False)
        self.gru2 = nn.GRU(256, 128, 1, batch_first=True, bidirectional=False)
        self.gru3 = nn.GRU(128, 64, 1, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(64*2, n_angles)
        self.dropout20 = nn.Dropout(p=0.2)
        self.dropout50 = nn.Dropout(p=0.5)

    def forward(self, x):
        x, _ = self.gru1(x)
        x = self.dropout20(x)
        x, _ = self.gru2(x)
        x = self.dropout20(x)
        x, _ = self.gru3(x)
        x = self.dropout50(x)
        x = torch.cat((x[:,0,:],x[:,1,:]), dim=1)
        x = self.fc(x)
        return x


In [ ]:
data = data_loader("/content/drive/MyDrive/random_sampling_file_hrtf.txt")
train_data = data[:320000]
val_data = data[320000:384000]
test_data = data[384000:]
del data

In [ ]:
fs = 8000
sample_length_secs = 0.5
n_observations = int(fs*sample_length_secs)
n_angles = 13*5
rnn = RNN(n_observations, n_angles).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(rnn.parameters(), lr=0.001)
az_angles = ["270", "285", "300", "315", "330", "345", "000", "015", "030", "045", "060", "075", "090"]
el_angles = ["-45", "-20", "000", "020", "045"]

In [ ]:
def open_hrtf(az, el):
  angle = (az, el)
  hrtfs = {
    (270,-45): np.array([[1.544712033382065e-07,-0.00020662157416427695,-0.0005693969453440447,-0.0006544460003052282,-0.0006421639090351612,-0.000638264897015858,-0.0006893761945177387,-0.0006785795613141678,-0.000641327888643242,-0.0006896953024948715,-0.0006954393211890636,-0.0006627881412917757,-0.0006473408781550587,-0.0007691343758955597,-0.0008084149567599306,-0.0004680926821621485,0.00830555550780986,0.012657160708515367,0.02300084540373481,-0.0004926239655171724,-0.005371354110794181,0.019921831174254997,-0.00188442635175281,-0.009198730630443247,0.001977037719196853,0.0025296005696822644,-0.001709062940690346,-0.00037941241829288517,0.0012053024267877734,-0.004960311376146315,-0.0018933264875195517,-0.0011350001921116599,-0.0016663001773594467,-0.000855457708652899,-0.0013833215061622671,0.0007673369626375136,-0.0003992153703838602,0.0007197147553004743,9.376337853595274e-05,-8.876721375144532e-05,2.556345917730613e-05,0.0004493255336913614,0.0008541124989685106,0.00024732701350509075,-0.0008732624021415002,-0.0019336001567282406,-0.0013087004041870045,-0.0019017084424758277,-0.0005901855244426747,-0.0001254876802308549,0.0006631895911966087,-0.0006874431022474245,-0.002049025213419387,-0.0009340574058910981,-0.0007531098708001845,-0.0012567970530198054,-0.0006373723953490507,-0.00021213383555540604,-0.0012031801082262362,-0.0019609371644182905,-0.0006941755987315182,-0.0006801715294011616,-0.0002600584972798629,-4.995999493725185e-05,0.00037551728612724735,0.0001954268898930013,0.00018479334901864736,-0.00014143965362898642,-3.726975931979939e-05,9.598220328272883e-05,-2.2823204899100994e-05,-7.58365001749964e-05,-0.00011076399818892272,-0.00010897032996469703,-8.832247933688312e-05,-4.9686666850278466e-05,-2.3735931675165502e-05,-1.4959278783166805e-06,1.2247391182134188e-06,1.3412109176731743e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [1.770391619759267e-05,-0.0004553735266960147,-0.0008800643393477935,-0.00019131596665362084,-0.0003209749625591036,-0.00022357379340494046,-0.001480593936066932,-0.007114042428816794,-0.007309460603387765,0.018863433799620834,0.2093713450162404,-0.02581106808723517,-0.04482442314764597,0.008627454449836824,0.08986280599864817,-0.05188367099174316,-0.040779171571721645,-0.003447554779068303,-0.014294708800912572,0.009931176173840905,-0.00510677974582403,-0.023544385096638625,-0.009548744000830963,-0.004116018557696858,0.003335843013218303,0.001794728056119655,-0.011922904475281399,-0.00184965909994077,0.006512306513671222,0.0004783127705260443,-0.00208697834462387,-0.003761678048847471,-0.0011627546540520248,0.00251252014366437,0.0010714732958326775,0.001212733040891513,2.6959074237615815e-05,-0.0003916146871891855,-0.0011837213653919472,0.0011291108851028686,0.001550865064804994,-0.0010420807233483841,-0.0025023201013011282,-0.008471053917716033,-0.004881777948852933,0.004180837713779149,0.004805440015843439,-0.0003460570252994285,-0.0006918780516124705,0.00018241290191724663,-0.0008076703392099672,-0.0006363328202012027,0.0005335881620768476,-0.00033788820479919394,-0.0009029463969383832,0.0006560980164374803,0.0004314148274385888,0.0005103067073081967,0.0002709396627036314,0.0002986154285792399,-7.037486827536377e-05,0.0005932950024722767,0.0014925595807354568,0.0001653942972644671,-0.0006109146754361107,-0.000912257427382328,-0.0004258355579920607,0.0001253590784206926,-0.00021377917117897859,-0.0001633310816245037,-6.43602792074117e-05,-0.0001951788146415111,-0.0001314208428045988,-0.00011558990644176909,-9.367335009056481e-05,-1.4243326003821927e-05,4.5134539096542985e-05,3.059569939033022e-05,2.6973437809034098e-06,-1.452664372059016e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (270,-20): np.array([[7.175659885555393e-07,-0.0001925588529831833,-0.0005707824700661691,-0.0006445866350150332,-0.0005478006452857219,-0.0005521200029746483,-0.0006213790897933634,-0.0006319626426210622,-0.0005620649778511803,-0.0005269724617967725,-0.000655005251713914,-0.0007152862545487966,-0.0006168902627521219,-0.0004893847993810314,-0.0005672702012231263,-0.0005637205974855589,-0.0005445377217552715,-0.0008884432905766361,0.020830791653688795,0.025554223258839725,0.011183774578104036,-0.0035516821098308463,0.00569065427405603,0.011572449249779484,0.0016072903777232938,-0.008912932779767447,0.0032707277049951505,0.0017164026968975986,-0.002160320702216857,-0.0003803990957527616,-0.003950952363489997,-0.001157389562711857,-0.00046829391454794795,-0.0006216362121748593,-0.00211460003608592,0.0014966807830002536,0.000717606750169316,-0.0012513085936831155,-0.0003675021858541506,-0.000922504622377091,-0.00026809419464188824,-0.0007380306164460852,0.0003141904056338337,-0.002222122839343733,-0.0006121740496887902,0.001215411744376283,0.0013168388816595163,-0.0016996088544774131,-0.0027669684040298264,-0.0003098440101078785,0.0004610689499476088,-0.0007036562276131273,-0.0006504091271636463,0.00024823588913881035,0.0013457661883854008,0.0006540407780369248,-0.00015204377526894367,0.0012563770750416469,0.001126525537224972,0.00019300113685902868,-7.692947081739082e-05,-0.00012001506214893687,-0.0003445970854362472,-0.0004055076235060436,-0.000574576052972318,-0.00034984801972560634,-0.0003571102509179928,-0.00032256644414106826,-0.0002976231998939104,-0.0003976379116341474,-0.0004243257155272829,-0.00030080599596517987,-0.00010047358097968466,-4.6977350217687086e-05,-4.180635160683334e-05,-1.8937062381515706e-05,-2.43279807405125e-06,-5.425610725807869e-06,-5.601978875673334e-06,-1.1051347067886282e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [1.3891110206109144e-05,-0.0009720860677612953,-0.00033977667617891014,0.004740644775057685,0.002919110441479925,-0.008565150681041142,-0.009782310423817567,0.003614771128858758,-0.0005617852528812589,-0.022765240279156256,-0.10588299108985137,-0.10557154616625601,0.03641680391669099,-0.06328429859978918,-0.024416581221746333,-0.011922164382755894,-0.017968258309080193,0.02008091851040646,-0.006710415186909447,-0.004710371765934141,-0.0035122209825357046,-0.003137434202988891,0.015908176766702184,0.004338578375395476,-0.008720575052217966,0.008346853345861058,0.005643992024237479,-0.004504276576056366,0.006732097621011984,0.0022991042482840254,-0.001253370696199294,0.002553023390663894,-0.002542872342887703,-0.0001292916055374346,-0.0014628724718757824,-0.0017756612577793055,-0.0020966428294511485,-0.003011247815319851,-0.00681142853092754,0.0005263062019047032,0.0004317898165976484,-0.004648734543220783,0.00203376744011342,0.0013121510564154874,-0.0023971739679478467,-0.0012285396645561621,-0.0008298517258255632,-0.000978585005164819,0.00034061769824663747,-0.0004052070726985359,0.0012074344684594091,0.001468651372528447,-0.001957519220290733,-0.0014504482995194355,-0.0009106790942644349,-0.000941091698032359,-0.0004683898979171322,-0.001644252241478296,0.0003064093657420339,-7.97853706939232e-05,-3.662550743381058e-05,-0.0007611086091891731,0.00018735606525360917,7.180256425057465e-05,-0.00016598453707866782,6.956029030470015e-06,-6.0560774518809917e-05,0.00024689725886984954,2.755039032592987e-05,-2.4510112158214977e-05,0.00021750031421071422,0.00010256695905401746,1.646020289997812e-05,-5.7156613962241805e-05,-5.860668200165562e-05,-6.924836426117856e-06,-1.6014348542189098e-05,-1.740479437100052e-05,-6.031246529596561e-07,5.939148185725846e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (270,0): np.array([[3.093300077711018e-07,-0.000203463402860475,-0.0005260165511453772,-0.0005626148821821338,-0.0006126371912976823,-0.000632713203707404,-0.0005806321497606866,-0.0005108519673730718,-0.0005340238196186943,-0.0005684228995839446,-0.0005525619153340658,-0.0005576365782400766,-0.0006270222393254842,-0.0007509664133887083,-0.0007119943274933989,-0.0005147349779840683,-0.000159263146107026,-0.00031214307518751596,-0.0012839589313728113,0.017445716763999388,0.028434427750571343,0.024433407928172647,0.006881939686093086,-0.013568096023821436,0.011607080589292435,-0.0015483961846069912,-0.0026667703621724888,-0.00556496325713612,-0.0007594878939081725,0.0006282361448086677,0.0004696623511498276,-0.002080895795898583,-0.0026038834430832745,-0.0010307303794673168,-0.0013758399953369869,-0.0007165639801715979,-0.001489404545725313,0.0005699845249799285,-0.0027311295812611786,-0.0017934115638045102,-0.0006672824888509268,0.0014659834122443542,0.0015824950216000045,0.0009296775481449673,-0.0006365410590572499,-0.002953097558729945,-0.0010906984744550809,0.0004945299933166235,0.0009257105554804029,-0.0009079884488068584,-0.0006572653917590307,0.0015782363123228618,2.1542435078610693e-05,-0.0006413874333825572,-0.0006841703835587968,-0.000893559069597478,-0.0013161974872069931,-0.0014820868032143731,-9.220298173661454e-05,5.971014073277353e-05,-0.0001256517367278053,0.0010864580688414733,0.0009510860859504302,0.0002802510577049349,7.509448625026122e-05,-0.000616594248982426,-0.0007041789597706663,-0.0005483887896082056,-0.0002458850972045961,6.261521094710955e-05,6.26794330504916e-06,-0.00017466706482009993,-0.0002179392427661831,-0.0001610056291301974,-0.00012187644351278835,-9.248266351244972e-05,-4.759134805313501e-05,-1.3584236309757761e-05,-4.058551548945611e-06,-3.829092982394194e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-3.832329274293397e-05,-0.00015369710319167594,0.00046494622430036576,0.00037923051232898635,-8.490663571401675e-05,-0.0035658857655378046,-0.007881042912095031,-0.00014269738831017162,0.027766016781199613,0.21764344023920296,-0.10320524531139752,-0.06728352423836007,0.14255852527431312,-0.0723681620106035,0.0375994356593359,-0.01672602444446645,-0.04394263675433167,-0.012854985212544343,0.0011964848067369434,-0.00916414131013763,-0.029027854672396013,-0.011077086114437418,-0.006047626613354207,0.00801680770657313,-0.0018987805298127684,-0.004486203005945962,-0.004513335113350405,0.0029780376404165685,-0.0016552701534313426,0.0010081904966754373,0.001614871260279323,-0.002423497513877839,-0.0004987497726445725,0.0003070474279652021,-0.0003610008543252499,0.0024172955036999948,0.0009594000835871357,-0.003663419263632284,-0.0002606066215743875,0.002113041650350802,-0.002802769431074332,-0.0012045878942274063,-0.0029103226382602362,0.00020664243797486868,-0.0022659850067169507,-0.000792845054845854,-6.388331567571235e-05,-0.0032493345206101943,-0.0016571878975479748,-0.00041390162142915075,-0.0012670782717625455,0.0018399421493478512,5.5695207633235825e-06,0.0010691297022951878,-0.0004817191466261164,-0.0008407689638843145,0.0008220810673436251,3.5034490190854864e-05,-0.0009295347890777605,-0.0009872753356273537,-0.0015673547123906063,0.00026420053589549944,0.0004966084744296186,0.0007589341027510584,-0.0004824644611812705,-0.00050628039579792,0.00019996619281057422,0.0002620845568442421,-0.000529777154854587,4.288973783484989e-05,8.63323353105128e-05,1.9451192231737673e-05,3.808522245403638e-05,1.5311206716647044e-05,6.970934234161618e-06,-1.223992886068356e-05,-1.8373482293053654e-05,-3.4558155420790462e-06,3.1413572338318074e-06,4.0592383266929246e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (270,20): np.array([[3.7144670914780734e-07,-0.0002002017867554244,-0.0005567002882767447,-0.0006466419563719225,-0.0006595591298852764,-0.0006205753489088849,-0.0006196635453880983,-0.0006473481859166558,-0.0006343081944243613,-0.0006669405692975173,-0.0007003044330028301,-0.0006722568659604136,-0.0006984532408926687,-0.0007345633500519289,-0.000707776113442654,-0.0005445623854329972,-0.00015713082058889914,-0.0006087893802740193,-0.0016371933208033562,0.03817011462877525,0.06880258394702826,-0.04875955163424726,0.02053120614214879,0.00514796351690066,-6.225916074310402e-05,-0.001727037917944992,-0.004891628200065018,0.005833738395914331,-0.0015916528894682317,-0.0028203369616993023,-0.0040531530361499705,-0.0015916625627193615,-0.001377061281462627,-0.005415429239867855,-0.0031199034131547186,-0.0022250237799568873,-0.002088873496069454,-0.0028283768137156175,-0.001122281845805982,0.0015585466391283714,0.00017356458423563502,4.797622993282503e-05,0.00017834837117001974,-0.0015084095545861143,-0.0011960433839826262,0.0015390800580543495,0.001614764028154297,4.300698581916728e-05,-0.0016172965909089024,-0.00034872208968682837,0.0001601597084459654,-0.00043665568393570705,-0.0011808492896275633,-0.0013699829523913369,0.0014800175003903424,0.0004620666116283717,0.0007065675011548286,-0.0016965681438763079,-0.0011740695186449877,0.001094377424712097,0.0005020736433537542,0.0003792859029471039,0.0008916849437383686,-7.070751773022674e-05,-0.0007414203559815321,-0.0005705663937202438,-0.0004868213376186164,-0.0010230194799159488,-0.00048129022728883147,-0.00041726909889974273,-0.00021552757494811714,-9.013900882218964e-05,-9.796953868422301e-05,-3.902045248414747e-05,-6.24504136701239e-05,-2.4631053799421347e-05,-7.16215319037184e-06,-4.322101783126034e-06,-2.480485223810943e-06,-6.142129180005791e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-3.052011077012907e-05,-0.0002608935278439831,7.264087769766369e-05,-0.00017382899686712287,-0.002372917096447304,-0.0029578679822859534,-0.000743113131534588,0.0013750150317450606,0.009032099912083051,0.09308873477600305,0.06842872723042227,-0.010866708003470402,0.025584923523721326,0.022269011546954067,0.04275672669215472,-0.01812150037246635,0.034110145405476716,-0.020605799749717685,-0.014563581016157751,0.009448553765759801,-0.016124476838457583,-0.022750550649139258,0.00957308938361818,0.01007562644243844,-0.0017590858610249996,0.0024387382150600307,-0.008127465508912413,-3.17250872745687e-05,0.010421547849399755,-0.004737395561315931,-0.004156764345951461,0.001414387160067673,0.0016848201894174464,-0.00019842863510567961,0.0013285145370679175,-0.0002669900734618588,0.0019655547000790663,0.000636568899240193,-0.0016855614740486635,0.00166689209280243,3.7021682139267634e-06,-0.0033518908060717343,0.00043237725326587074,0.0017050413778284014,-0.0014887578803925421,-0.0017755767558986233,0.0017866091668857426,-0.00328170066548134,0.004602130451306713,0.0017433918396704273,-0.005095993960299912,-3.4961173271153737e-06,-0.0018934356658879556,-0.0025696042558712414,-0.0024206983499008277,0.0012316978139548557,0.0003633342706185586,0.0015478227207530303,-0.0020287127469910803,-0.002461193281337549,0.0005137423956996504,0.0013133101324423254,-0.0010206664871687937,0.0003498940206963179,0.00012373090262989537,8.078037315452048e-07,-0.0006784047463109804,0.0001730340663876437,0.000319857489439243,-8.053307911037001e-05,0.00019374641229639455,8.410527620278133e-05,-9.992446225411625e-05,-3.733635622508431e-05,-2.0822178189758834e-06,-2.9634639891601272e-05,-1.4903391626833276e-05,-9.55209702706246e-07,5.858768840240341e-07,1.478273000358277e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (270,45): np.array([[-2.1483394637110477e-07,-0.00023106915313136393,-0.0005162003665727081,-0.000468706357816847,-0.0004979641369314109,-0.00051159147686383,-0.0004927988484712106,-0.0005626832544217161,-0.000664492744631293,-0.0006249030293523669,-0.0004888774421006366,-0.0004644217838391622,-0.0005854429553288134,-0.0010902483614372012,-0.0011025367577194756,-0.0004695810128993122,0.003472993534628078,0.05143441410778266,-0.01657062859590761,0.005071648264676465,0.010770471105667902,-0.0019251094309709847,-0.0037787748107505816,0.004789768723711326,0.004809624074104383,-0.006045628444786807,0.007243416094807304,-0.0006811037833212954,-0.0008245567629110469,-0.0005390234004996617,-8.758229203739975e-05,0.0035987899004894354,-0.0009417426513435685,9.234340848911858e-05,0.0004232052574611605,-0.003653524948402505,-0.0004317267551148511,-0.0033374250861404023,-0.0028393378809126037,0.0007167981702649241,0.0015746164778255804,-0.0016811750774027182,-0.00144804461012951,0.0018044905688218088,0.0011664178167255924,-0.0021389816432757485,-0.0005335321188721339,-0.0002395232341109278,-0.00029763223313391895,-0.0015107102252141659,0.0012224437004140051,9.537551364233295e-05,-0.001625973977296347,-0.001516789011035939,-0.0011621846008815781,-0.0013847139368478058,-0.0004752908099946754,0.0006373045893938166,-0.0002735135281683288,-8.270516591325564e-05,0.0008702961328682593,0.0027074947339097424,-9.101648279725692e-05,-0.000993446412035227,0.00046545238458595774,0.00024100952038168217,-0.0007184285682193892,-0.0004342208962423038,-5.336477552571316e-05,7.952140612248063e-05,-7.267470429826022e-06,8.11786576299567e-05,-8.242021405872517e-05,-4.0808256206472304e-05,-2.7867885270410783e-06,-3.89110001822316e-05,-4.306217776530515e-05,-1.8379898086537356e-05,-8.936350368804357e-06,-1.8703962119499138e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-4.008848901649274e-05,0.0002864050464261656,-0.0010834485906126908,-0.004689996033278606,-0.002000077398000223,0.003902007740494973,0.002109238741708936,-0.0011197627866916008,0.005903988473220387,0.004507145172268321,0.14605972056612124,-0.13106719105230918,-0.020601063364885906,0.028543995102893516,-0.042844992453448305,-0.012593686470656432,-0.02520002460700918,-0.0007057056773924754,0.012704491238116573,-0.022352155380760585,-0.009794193347993571,0.006611891552862065,-0.011745605800152324,-0.01580275580727386,0.010920320699971925,-0.007383259805568552,-0.006003060414309482,0.004257022188398644,0.00018134381284370601,-0.0008580032189693762,0.0055541338208611405,-0.0012794683036084105,0.0021946654318808476,0.0017159514777307491,-0.002183432941951256,0.004607184360437849,-0.0022178987423757015,-0.002545741569644248,-0.00013198058027391797,0.0005409494744617638,0.000987651605898179,0.00016276994033484848,-0.0019794470315787783,-0.000783312263259054,-0.000829187277025154,-0.0015039682161019368,-0.0006621243169067897,0.0010580256179501006,-0.001388863141361616,-0.0026185511223145897,0.0016025463500336987,-0.0008607063106188489,-0.0020440442063441888,-0.002725962135639175,-0.002191772759804768,-0.0014233692612635458,3.321222118433384e-05,-0.0017802758688608012,0.0013235374912828597,0.002221053499366526,-0.0007272762965420267,-0.0014263331880600256,-0.00035725898809866994,-0.0020395299296519158,0.0019254698389734087,0.00025709726296924466,-0.0002112404668229553,0.0009130903403008137,-7.421371780506544e-05,-0.000669245067084365,0.0003996616481296683,-6.592980906843501e-05,-0.00015978677862792194,0.00015878253288412558,-1.9312980350578248e-05,-5.5217255199489027e-05,-5.6713126323818335e-06,-1.8114532044944237e-05,-8.642371892746837e-07,3.2960828201141526e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (285,-45): np.array([[1.7175101684120575e-07,-0.0002119454833578601,-0.0005628232846522306,-0.0006216850718759781,-0.0006686729213454302,-0.0006609528729816368,-0.0006240872226965238,-0.0006409831493794961,-0.0007173529790854539,-0.0007076318560989663,-0.0006739443531533542,-0.0006841864260636139,-0.0005637269526654533,-0.0005579638383999606,-0.0011087720555366793,-0.002265049541091448,0.014554128047435468,0.011997616339029883,-0.008607379207647661,0.013508319566099899,0.019577563675011667,-0.013623640370347192,0.002108154623946619,-0.002045391833294539,-0.004434589237380114,0.001495857871693594,0.0023985084485199227,0.0018724308004743901,-0.003245135368849718,-0.0028589935139228116,-0.0019059715763815096,0.00041532874012653863,-0.0002725327749644278,-0.0018790311902908198,0.0008704148759571273,0.00023249891359872988,0.0008171228753358052,0.00012274628064002608,0.0007166235571738116,0.0003762473604526134,-0.0008597502409100347,7.431824470980409e-05,0.0005474698421750989,0.001077449641431914,-0.0008663068331623699,-0.0026364294369358643,-0.0016969682129064335,-0.000970718696626811,0.00032988587975658646,9.443004712691138e-05,-0.0011129385302288378,-3.568908480930963e-05,-0.0004886792525142371,-0.0014471687210820016,-0.0013923711633089984,4.3028207626513044e-05,-0.001132288471175888,-0.0012501785594002755,-0.0007735729790465776,-0.001418698095332469,-0.0005531768912891409,0.0002690084907453431,4.269579717334432e-06,-0.0003554901473156482,0.000561871472707203,0.00043608733976157905,-0.00010614674420495464,1.4740695798447409e-05,-4.8504483804296045e-05,1.123662965859123e-05,-0.00010717107373864083,-0.00014969033776763766,-9.656415407784482e-05,-9.851586806272942e-05,-7.96814803479906e-05,-3.810170100350949e-05,-1.963426092139304e-05,-5.376523422570225e-06,1.4538715167666806e-06,2.936142166541849e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-1.8374344296505406e-06,-0.0002602825334152597,-0.00034097814785465867,-0.0007528685438174771,-0.0009530444852996172,0.0009158979820492205,0.00023284506208097702,-0.006937769477615314,-0.010647972112062552,0.004490970598439478,0.11173079285767634,-0.06962585229698356,-0.03576538110428497,0.02120130695933888,0.12836444035322292,-0.08775391961023922,0.0077734170431840955,0.01694277454281808,-0.000984196191529575,0.017115702849942047,-0.023292493046431446,-0.018632840409956188,0.004619411406644828,-0.0033157349967560903,-0.0054227636316357755,-0.0037441166122733883,-0.010343539727061766,-0.0023401435383213973,0.0057194297745973306,-0.004906681752443122,-0.0018802691031840348,-0.0026097997286082942,-0.002786123553598194,0.001911431422036418,0.001131310101688588,-0.0028785893756227607,-0.00021414879510360584,0.0008260713909868107,-0.0006206675272661213,-9.969026188038693e-05,0.002980684265404619,0.0011675799401785615,-0.003506196850814377,-0.007946785817102136,-0.005917593344709258,0.0031227959575524404,0.005686390736379377,-0.0003787759159190113,-0.0008913376570429787,0.0009047861967329896,-0.0014108566947225584,-0.001992944299023575,0.0018450250792913347,0.0004324422613518203,-0.0017079421567786452,-0.0011618026352391154,0.00020320357854252468,-0.00047574130941932237,0.00015591887435934364,3.57844873488396e-05,-0.0001160152441834307,0.0012054322557050099,0.0011564508985910575,0.0003584429763730525,-0.0001273204596987394,-0.0012683387952963684,-0.0005048230164053893,0.0003729341117332879,-0.0003297358513266293,-0.0002802925425999047,-5.7018716791445245e-05,-0.0002771088737461675,-0.00015451281509668471,-0.00010817521041840744,-0.00010411327513755812,2.093617734751251e-05,6.355642415262265e-05,2.5180848986082285e-05,3.1008711792278256e-06,-9.783611854634684e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (285,-20): np.array([[1.976338225122099e-06,-0.00020639882727124117,-0.0005878144287417915,-0.0005833552473421442,-0.0005015065076559107,-0.0005657933525176275,-0.000636728165124144,-0.0006109585009521094,-0.0005292939424098628,-0.000601160330467596,-0.000748356002918259,-0.0006220757420564676,-0.00044302845242935314,-0.0006119332329370193,-0.0007772214533513158,-0.0005222769408691576,-0.0002611906423038374,0.006442952934991434,0.016834928159077965,0.023060436438217025,-0.004749655615348342,-0.008931952327770625,0.009228312603074301,0.0017254360194688784,0.002673916256170344,-0.007706714442498128,0.004822617876654474,0.0012502608429890187,-0.006221663027986528,0.0012347614249536203,-0.0012737792305519149,-0.0016345278890276572,-0.0028750749771453603,-0.0008343333148747305,0.0030164610202004253,-0.0008215547195705514,1.595896687255157e-05,0.0005747646030250642,-0.0005561136906282428,0.00021293434993891884,-0.0005173658444839534,-0.0016440661075684023,-0.003754867259683185,-0.0007519721045363715,0.0002812490272452613,0.0007083736409756494,-0.0004318618173859912,-0.0007803598777808628,-0.0006182568255564217,-0.0005994749800227738,-0.0005614359845627063,-0.0006573786816104783,-0.0006226827337094565,-8.236227748929824e-05,-0.0005311654054785727,0.0009864498379241252,0.0011055707642912882,-0.00015403793506019062,0.00010180783627351617,0.0007312838290194089,-0.00028372754921800506,-0.0001915705911394186,-0.0007664073626478093,-0.0003795697848459466,-0.0004209949821702206,-0.00014986672344603256,-0.00022234105937537997,-0.0004559874673662706,-0.00037237731833177854,-0.0003788656941407169,-0.00037187862342509656,-0.0002844731450407211,-0.00011457890795179258,-4.63804835271978e-05,-1.8586341829936084e-06,-1.0733151528343413e-06,-3.412644244064827e-06,-7.555979384905445e-06,-4.675886805969409e-06,-9.413963688673568e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [7.85941527967484e-05,-0.001184219130277031,-0.0038475312875983552,9.583269030012011e-05,0.0023626592274972294,-0.002215396422465257,-0.0027176484903197054,0.0030064918554348003,-0.006063091315901626,-0.0666360407718824,0.08345848294126117,-0.08716414477928205,-0.07213576108738229,0.02985651129329979,-0.06838199653363745,-0.03485209407937552,0.0026244369746282648,0.006730833444334916,-0.024562642531822467,-0.015752157318488766,0.0022514738262514014,-0.006095370120831643,-0.004275819689988918,0.00039174738481681845,-0.009928704164129292,-0.004249524460563226,0.006741955521196805,0.0003631159409057644,-0.00174268787240363,0.000808544248932379,-0.003967984783455409,0.0018216425364402137,0.0028119861830184206,-0.0023815598603402328,-0.0015090547283418852,-0.00042366039561738354,0.0004418212912956705,0.002302881963094237,-0.008882928244223248,-0.0016925181593623539,0.0034868723587076216,-0.0030947009020816213,-0.000370614844863041,0.0023642724374211923,-0.0020299619318194673,-0.002281401568960544,-0.0022257124103860046,-0.0015354052577050423,0.0017786215105490613,0.0004462662061805597,-0.0016502868579672304,0.00017336643000005555,0.0007098581381653015,-0.000523921006549363,-0.0011189887388946078,-0.0014762651258150065,-0.00011710639098241543,-0.002083535846276573,-0.0004651370168592971,-0.00039674997861612147,-0.0003045471401702166,-0.0005947198728100936,-0.00015169473274398112,-0.00024489172306567885,-0.0002501801461384997,-0.00025779712802799444,-0.0002659102755725384,4.538914599397046e-05,0.00023346241697306402,3.7663627920529126e-05,-3.4926826869608805e-05,-2.21444551949523e-05,2.4132360213441464e-05,-1.3739087597379305e-05,-5.4944249396351526e-05,-1.2630089436414081e-05,-4.110810967955267e-06,-1.1139933433275948e-05,-2.570391861140553e-06,2.712549939329757e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (285,0): np.array([[6.551725417117469e-07,-0.00020462374991156654,-0.0005512087010968316,-0.0005718437021326807,-0.0005555798931163076,-0.0006344455284460167,-0.0006803814517820487,-0.0006129996289048398,-0.0005104348719491717,-0.0004707819707723554,-0.0005653108152437701,-0.0005983940694314507,-0.0005206960949192936,-0.0005012328560301345,-0.0007745219980187206,-0.0008677523588562215,-0.0007297043326409705,-0.00048288891779419573,0.007100381993876706,0.024286850748169136,0.008386833122209184,-0.00828322062108536,0.010433111999016958,0.011871089231543383,-0.011704359370781743,0.0023181476701068762,-0.003217384746446866,-0.007049664477831263,-0.005396831848235655,-0.0018409384645559734,-0.0033989954461112193,-0.004592305895847198,-0.004826496189702758,-0.001584360355769802,-0.0015956941410201107,-0.0005451378049027601,-0.0008755233238619475,-0.0005082196763393325,-0.00025198706154756247,0.0004747736714777827,-0.0015552857773149984,0.0027209406272824576,0.001590502059817222,-0.0013936035827341194,-0.00037742748414001855,0.00012692162251741064,0.000722954474762451,-0.0020631987908811713,-0.0019506092835765159,-0.0006607159212654275,0.0008822655458976318,0.0004393484671013944,-0.001391575131305487,-0.0005004130432664672,-0.0012474218352421845,-0.0010891915618193714,-0.0014779311441327466,-0.0009144028106198986,-0.0008379067572499966,0.0007379505427397406,0.0006874858563996357,3.9159843694903846e-05,0.0005488510316443317,0.000112102759031125,-0.00026536104762041893,-0.0005448815381966588,-0.0005410496782118962,-0.000343412048254346,-0.00015296975048720247,-0.00020090755700871577,-0.00025412993934137933,-0.0001704387786462218,-0.00015491846090753396,-0.00018405096652129166,-0.0001243565990119841,-6.532614977592988e-05,-3.372006083491399e-05,-1.3361986935046093e-05,-2.775130364535702e-06,-1.481744808304897e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [5.8615444895817215e-05,-0.0007708155782759729,-0.0019131732640157465,0.0003567491086594876,0.000614025548645625,-0.004047605605398465,-0.005724312833136643,-0.0024849792062080866,-0.004594694768984568,0.01902533065161384,0.028101219941745404,-0.11829240108599028,0.1996785393870465,-0.03791511523576977,0.061168609872220385,0.014264870330756237,-0.006259183137440692,0.0470239203428071,0.010780490513294976,-0.045559963500091075,-0.0012973113102565338,0.01245050537751995,0.007423572189904458,0.006274997870847672,-0.01264065382180865,-0.006129325192303627,0.007819809512743435,0.0029843357027266515,-0.008558386526846408,-0.003998272092731143,0.003143398118037524,0.004314254925318264,-0.000654023997016525,0.00106944637391477,-0.0008115680337573892,0.0018014325120364087,0.001554908827778483,0.0013704765170800331,0.0021481893900871848,-0.0017253488799600352,-0.002798087318693339,0.0019173185956150747,-0.002492533139911529,-0.00017704762236315429,-0.0002744432757008266,-0.0027548273816136604,0.0003576809344622976,-0.0021936703486445095,0.0010468836364258961,0.00021727435321638384,-0.003910006710886919,-0.00022842473636748253,0.0019640128735879474,-0.0023291755446287546,0.0017673008202444612,0.0002060640276081004,-0.0022030175748099794,-0.0011174160654451983,-8.693024882717703e-05,-0.000910763351208464,-0.0007857747356983818,-0.00018447599084441582,0.00012159307272054389,0.0004806086116563773,-0.00036257259454222873,-0.00038530904101250445,0.0004516167000010329,-6.003402019905737e-05,-0.0006022674084495086,8.67320078414254e-05,0.00023567082764099716,-2.627772706364535e-05,5.4110314860035236e-05,-4.5689530485067274e-05,2.4085902626086224e-05,3.9871539811304196e-05,-5.278283404896181e-06,-4.109303969208951e-06,9.08300682874803e-07,-2.1724966423421812e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (285,20): np.array([[-3.552079974041767e-07,-0.00020001823581858595,-0.000540802181682078,-0.0006383645757976006,-0.0006416534823400618,-0.0006276584947708892,-0.0005955950815393701,-0.0006313722823226004,-0.0006643059544176793,-0.0006560627707336958,-0.0006107162460018413,-0.0006794109416626423,-0.0007468563091963627,-0.0006965783402575949,-0.0005042647751412068,-0.0004304292504184376,-0.0005438254703642414,-0.0006655418283994158,0.02372475264517787,0.04321017291325026,-0.003990810171272452,0.019777556192857034,-0.008033997545460712,0.01283503792771789,-0.003256450958246662,-0.002712286336705917,0.0011849370517808537,-0.0024753485757776344,0.0023423425764216646,-0.005504522395047294,0.0010998269248639555,-0.004137141477996262,-0.0030337305378872552,0.0011741726143085483,-0.0015110858268055907,-0.0012544911546184144,-0.000987585138886033,-0.0008250778202748546,-0.0008280682889932724,0.0004649430956467757,-0.001291608511560374,-0.002318249623035143,0.0011482391343319977,-0.0005779141247135644,-0.0014015680926701035,0.001329222558786826,5.897959386799091e-05,-0.00023695152764511857,0.0005513034571123618,-6.505705333602674e-05,-0.00042733276924158584,-0.0016050320473788317,-0.0016049466993111367,0.0005634371373703024,0.0026202466755880353,0.0005134761035866656,-0.002189627586927298,-0.0010573159408198537,-0.0014055932889282504,0.0005030255886711732,0.0020977132052533787,0.0010147958031220428,-0.0001872561057598499,-0.00036063225597919303,-4.020199261585188e-05,-0.0005955750351409461,-0.0006971019346997181,-0.0009265460971293763,-0.0006273249028843083,-0.00012417312641303487,-0.00020251637424184714,-0.00013330774685836144,-0.00010548532015898031,-0.00012230108408459614,-4.655722924635824e-05,-8.558568064585885e-06,-5.305198715046823e-06,-6.134623632355796e-06,-2.376806357004264e-06,5.242143743850033e-09,0.0,0.0,0.0,0.0,0.0,0.0,],
    [2.314013869947022e-05,-0.00056122714130197,-0.000421566511324472,0.000518670507633298,-0.0005623523531250284,-0.001968807501974431,-0.0016602686063251655,-0.0034373290575239555,-0.0073344152171065154,-0.003696873266799202,0.22644447102196957,-0.22308129605318236,0.08442244354546417,0.016876068150123606,-0.00875905328416069,0.0007154277014492868,0.02425679479629929,0.04263166618803504,-0.0026654876062562347,-0.005148880318675493,0.009745415606936921,0.027894822545797775,0.0017301340621867995,-0.0108937473832183,0.0035355489679131423,-0.0011535823489099005,-0.003343664074022563,0.004331028448625366,-0.00040802916181897366,-0.0034212473374337906,0.004785813076082701,-0.0006506325844999851,-0.003959312782898626,0.005193512404037427,-0.0038313970175084928,-0.0036196311961782684,0.002466910054803658,-7.579169099086887e-05,-0.0023031833711759375,0.0012980823171113503,-0.001541773269881023,-0.0006969300968402187,-5.730672949975811e-05,-0.001143204594193285,-0.0020585107567310406,-0.0030477269223747098,0.0007211523905799489,9.069528316799692e-05,0.0008780955869202509,-2.0294988260074133e-05,-0.0032245302109707085,-0.0006359891615886765,-0.00023314988265068598,-0.0036127082628895422,-0.006035782320510459,-0.00016998828057643718,0.002698233076519706,-0.0023471363779306907,0.0005868372619584478,-0.0015251474695964602,-0.0009178723075825832,0.0008662906591773549,0.00030042591055290537,0.000191125653150275,-8.75013134904846e-05,2.1857392209175325e-05,-0.00036757648412992505,0.00012584719018848332,0.00017597122645875295,-9.30292016602957e-05,0.00011412548014722867,-8.315339320250255e-05,-5.051984436775162e-05,6.301281806573596e-05,-2.9404835203932776e-05,-2.9604806916080408e-05,2.9246973355018872e-06,-4.591476874838792e-06,1.0821404333503627e-06,2.9677715857037134e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (285,45): np.array([[1.6381109816965664e-06,-0.00017737566209487569,-0.0005808335469506463,-0.0006539821029887367,-0.0005551298195547376,-0.0006334404908278024,-0.0007317049039133992,-0.0006955724060037048,-0.0005344057166983175,-0.0006096270566696734,-0.0009055748424799757,-0.0009748994051559489,-0.0003935992279069712,1.7925249806710103e-05,-0.00012781170985543648,-0.00037435184525398896,0.017244880781660803,0.06525968087566558,-0.021850155974596542,0.007519716837575611,0.02488032273577433,-0.0023202538134394143,0.003563194814715723,0.00409710112783056,0.0034848660201155,0.004243375877847851,-0.002369459882276881,-0.0026603411110402452,-0.003266468790741944,-0.00037107339231549045,-0.004502818195555554,-0.003331875583054821,-0.0025864314628413335,0.0012428742687119398,-0.005346433914444775,-0.007634973350412717,-0.0025611978324137225,-0.0028389233213358673,-0.0019718091994030173,-0.0014430728681653962,-0.0005166243741654059,-0.00196504644863119,-0.0004143172802008149,0.0005875147511461799,0.0012837926715693862,0.0009483850405672867,-0.0007030476224891869,0.00029808164181098566,-0.001232758664869007,0.0013453284154210886,0.0012672739924716543,6.3914925152694265e-06,-0.0009066895955553179,-0.0006553802477113548,-0.0018435126847045683,-0.001212619485823013,-0.0010966961838110797,4.971660177192447e-05,0.0002278872334452392,-0.00045229312476825333,0.0017706785553616236,0.00031891409232897227,-0.0001930862966409511,-0.000630157343546425,0.00015846007696727814,-0.0006661823114222061,-0.0005462547813176232,-0.00014079956630075736,1.229953663810372e-05,5.359801979378104e-06,-0.0001232403737714974,-2.1829559815163285e-05,-8.900928151182529e-05,-0.00010777092481644759,-1.899308919649299e-05,-5.391205412611634e-05,-4.000570850427228e-05,-1.7657408478793932e-05,-9.376642626631479e-06,-1.5604011129609486e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-8.630266280132496e-05,0.0004201759771746576,0.0009761197022493701,-0.0029668107086819983,-0.00438473608911579,-0.0007144694087257616,0.002486781963701922,0.004258196974825586,0.00851943014226622,0.016142300478368794,0.13646151733488252,0.08888643099343248,-0.007276708656923575,0.05006761586652089,-0.0009861483730753751,0.013977779859411829,-0.001962366650265224,0.019811021410965756,-0.010131137012492674,-0.018664049375209338,0.01622694589101025,0.0020047280596871453,-0.015782425216640954,-0.011470781267136501,0.0035108720714012716,-0.01576550489566193,-0.006142816365935671,-0.0029131640485046567,-0.0033656883652578374,-0.0038197884025037987,-0.0020872125960744226,0.00039431512943849606,0.0009781099123848547,-0.0029145335781104995,0.0020073683903717968,0.0029996959467860442,-0.0012246941161820595,-0.0013711786408609687,0.0014344590256087884,0.0015895602565919819,0.0006586804652424326,-0.0007546161359463599,-0.00036875451589564933,-0.0019314910784691975,-0.0021596499791735383,-0.00027218390680643973,-0.0008267107904956023,0.00134083740050892,0.001390698201146365,-0.0021122268731014965,-0.001108990616409204,-0.0004665183912320459,-0.0007613354895878234,-0.0032155408521545082,0.0005775935000674306,6.750176282758618e-05,-0.0015174750780861672,0.004354549507459704,-0.0010196563363399727,-0.00024204456321568903,0.002054003545040122,-0.0013935060758416917,-0.0017715147442957753,-0.00040310217031015423,0.0010869201540730832,-0.000613556102374426,-0.0003281134102429798,0.0004580971069079294,-1.0594709622670374e-05,-0.0002945550017551098,-0.00019869997283683473,5.553678841483431e-05,-0.0001263590902253245,-3.1789236191240784e-05,4.93844693381654e-06,-4.0287831576154856e-06,-3.179401891941528e-05,-1.3681485036780583e-05,4.691904940546895e-06,2.992514130215911e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (300,-45): np.array([[2.978264214029928e-07,-0.00021422705293007742,-0.000583169112966263,-0.0006464798619107622,-0.0005983308728132657,-0.00064962077379994,-0.0006975764882326207,-0.0006444312239537635,-0.0006922854097992958,-0.0007907921771118378,-0.0006782835508031139,-0.0005359764744910236,-0.0006589693890181777,-0.0007932799956126672,-0.000445251728584013,0.03812793721462707,0.02035975205547975,0.0020276467103803415,0.007032840146739198,0.0015237172669611322,0.009844947027399052,-0.005962867973146937,0.00039077288903052845,0.005600750554900801,0.00214916282560019,0.0032004957201761243,0.0009673688444068746,-0.002105914881454935,0.0005969815676195466,0.000807054604100445,-0.0014987579776577647,-0.002052813884806129,-0.00046892360779378735,-0.00011344849685086774,0.0006694424686880168,0.0008665309617651426,-0.0011124661175622543,0.0008472816453820616,0.000721864152002024,-0.0014809754783069611,0.0008267808474447312,0.0008786058057951604,0.0014653578213020555,9.754819777167594e-05,-0.000758942514623429,-0.0019959169588203074,-0.002300223195994692,-0.00044878718965841855,-0.0006362545367853128,-0.00021294210822848562,-0.0019463133795469766,-0.0012985158169455716,-0.0003294240124810942,-0.00018254770778182596,-0.00039769358197416316,-0.0017491700337888205,-0.0021078179058472432,-0.00048628110052377363,-0.0007643289787958137,-0.0008358682200639916,0.00019196354368753536,-0.0001922799844403857,-0.00032789660668135586,0.0004954197211523508,0.00040151248343515543,0.00013864644874241898,-0.0003539681011071786,-1.58520362516774e-05,5.850388007409117e-05,-5.490440690693915e-05,-0.00015723405449165597,-0.00010002123166073238,-0.0001240618550006972,-8.982492006454953e-05,-4.935674629080285e-05,-3.1957064277150875e-05,-2.268390850579594e-05,-1.1043860430690309e-05,-3.2422551510133657e-06,-2.0324528690708006e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-4.546862545311017e-06,-0.00017574397218666333,0.00018523874995690038,-0.0009743613408785547,-0.0013286092759927474,-0.0001929961566300656,-0.0003131815557436684,0.0015109230224382671,0.005742593208325815,-0.004227731350254009,-0.054761653689562685,0.01811698701908934,-0.01262801867154468,-0.10177595425150096,-0.06453661073346598,-0.037954150773349994,0.04923501008673748,-0.012700412249146866,0.008803368501141079,-0.0025163416435936667,-0.016783520623236004,0.026583846868017947,0.013840925215268767,-0.00836392871416094,-0.0019906297110322954,0.0038100186028897556,0.0033733343576384505,0.0003266826028374145,-0.003657348711269066,-0.0027789302258469507,0.0035729068413990215,-0.000497694426081139,0.0027498264745429105,-0.00218336673233226,-0.0008118464815125961,-0.0005115274920473721,0.00306753667607162,-0.0023408298948041015,-0.0014338605339680463,-0.00014090135846870415,0.002360333944208587,0.0035543867960675193,-0.001910647022561438,-0.008153223652979831,-0.007711410877934187,0.0031742238023822035,0.0065758462714492006,0.00015537240365609415,-0.0023337610171769254,-0.00020589056297004148,-0.0003790388802090157,0.0005843826778727146,0.0007387723550370104,-0.0005386821761506991,-0.0014471890102239102,-0.0013238103337157847,-0.0004969691521921675,-0.0008055384836328059,-0.001153320801353247,-3.537759816342687e-05,0.001539011671290736,0.00047529066386079523,0.00021716085973917047,0.0019437036972722276,0.0008174696412594151,-0.0013753584462831367,-0.00030950038341707775,1.608032894821967e-05,-0.00017107410601516698,-0.00016907467021038102,-0.0003047636646843371,-0.00025126900811347244,-0.0001127190644589405,-0.00014472751078239014,-5.446314540372872e-05,2.1130311546801675e-05,4.1733824192370326e-05,3.4764069390322516e-05,6.776480327371145e-06,-1.2866540754710467e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (300,-20): np.array([[-6.0981368669825055e-06,-0.00017151006195191136,-0.00038369697254005223,-0.0006328364422979404,-0.000725556541557465,-0.0005487856389454054,-0.000494384375785298,-0.0007019031334946764,-0.0008969067279695396,-0.0008159399871680689,-0.0003192378504971248,-0.0003628372333670341,-0.0013001159865928728,-0.0015111627509674333,-0.00042409830411684047,-0.0007975851455805474,-0.003481229613481422,0.008552895281773958,0.0038359154581769955,-0.022196423733003443,-0.009820047304894863,0.018126814107595892,-0.005897708612361769,0.001089652844923249,-0.003138554358816267,0.005100329622287216,-0.003164275951347139,-0.004479471800514013,0.0006706716100012225,0.001004324177347395,-0.001481655854610037,-0.003180025922976329,0.0019238995092812627,0.0024482675082694513,-0.00020441110700932617,0.0006553032831717532,0.001783207963994628,9.790859402827042e-05,-0.0002742669304777312,-0.0010833813046106028,-0.0012609460813493962,-0.0014205730227268938,-0.0010750628662660057,-0.0020533826152843754,-0.002154763568308172,-0.00020739259193820587,-0.00043420256679359997,-0.00017127565726346596,-0.00030425565293725566,-0.0006586344458944984,-0.0005452908759711871,-0.0008891587354669904,-0.00020982662053453994,-0.0003844593175974339,0.0005977323962531822,0.0004255400521156808,0.0004633588079655548,0.00012367933136818104,0.0006789096147405899,0.00018710124967773336,-0.00044092494340150154,-0.000606328630383437,-0.00027939382890677326,-5.004430278735622e-05,-0.0004017829732843251,-0.0004916713771569611,-0.0005037609139827941,-0.00039619774667084737,-0.0002841415276860097,-0.00037464764073966974,-0.0003665201101382436,-0.00015708002472726674,-0.00010617269292214731,-4.10117988455221e-05,-8.589306642434826e-06,-9.934496111551795e-06,-9.633056957366833e-06,-5.493188536367135e-06,-3.724148554764032e-06,-8.89262143188696e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-5.9207208810163656e-05,0.00019752888481686798,0.0017946010905306622,-0.0006543219095965809,-0.004378531747466606,-0.00019270335132951367,0.005670381029760628,0.0005309620130901122,-0.003436634986224811,0.018112123797859366,0.31225241671661935,0.04433844438073708,-0.06101414642881072,0.19741906960719285,0.006023367616475197,-0.04372097465895244,0.025540775597292217,-0.005433992964452187,-0.004134446244599812,0.004172121961984124,0.009842100021124319,0.0016248194366719417,-0.003087717023124225,-0.003884240727384446,0.0035342806396684396,0.0018434755457757068,-0.0058137670042976075,7.654160546665473e-05,0.0029877824987598585,-0.001802316543942763,-0.0008682398406448756,-0.0034333989567365076,-0.002502243201699299,0.0017267344925506747,-0.0005332195342948291,-0.0018911263908249017,0.001673787031167607,8.024349415153806e-05,-0.0027055668563300845,-0.0001948034628145272,0.00034458522588004715,0.0009654038628815583,0.0009460868603264963,0.0001237275069824435,0.0007256207734249197,0.0004392671436241356,8.183558209366689e-05,0.0011325921321484118,-0.0004836195863374384,-9.279721620364403e-05,-0.0017036684502799158,-0.000744328009350493,-0.00018493552503495727,0.00011687002415144554,-0.0006122856517861997,-0.00013956678680632282,-0.0005810494993819595,-0.002404228635791149,-0.0018807290112629212,-0.00030215214638024617,-0.0007018966456363335,0.0009870520372117224,0.0010704739775903417,-0.00010957994649831953,-0.00014409406632498705,0.00023147212944370056,0.0002358887371418768,-0.00015151812557614587,-9.867480166982549e-05,0.0002606067680707623,0.000224184322406323,3.922360841565683e-05,-8.246785743967431e-05,-3.790866090882016e-05,1.550369658379989e-05,1.4401003726663968e-05,-8.813733785297251e-06,-8.325047379264741e-06,-1.8137863261357953e-06,-1.0140246332482404e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (300,0): np.array([[-1.218038740628277e-06,-0.00021458340228336237,-0.0004901638457321414,-0.0005036735712422179,-0.0005733583748486619,-0.0006061774134439171,-0.0006108782401490175,-0.0007160711125441719,-0.0008142345081255706,-0.0007129283995221542,-0.0005116122153262208,-0.0005089943136789612,-0.000572451236130609,-0.0006641263879334375,-0.0009107446772520267,-0.0008266057036204826,0.001379571732879234,0.03828378412278401,0.043859138997522126,-0.008253632184394795,-0.008128818467180382,0.013054898436149499,0.008897687091430805,-0.001588358659371393,0.0015633119213519838,-0.005129657923429944,-0.00667316298931974,-0.00264060446612701,-0.004480263086358522,-0.002167134111453532,-0.006231129109276233,0.0018480687215966503,-0.0026762398050821376,-0.002451199639555334,0.002041087871036485,-0.002818810465319027,0.000845938560752184,0.001179844504810986,8.61056114945737e-05,0.0011036535258366806,0.0018133257690361008,-0.0005401121070539615,-0.0018238326488957052,0.0009096910311039135,0.0013506790734255172,0.00023909694459544746,-0.0006399922386209561,-0.0007076093064788783,-0.0020089769776178027,-0.001254060091491714,0.00022878356280750653,0.0010103033750857896,-0.0005723806368351372,-0.0019479120231761671,-0.0003248848323001801,-0.0005151068601471528,-0.0005846790679151327,0.00021618051297329343,0.0003795541542124939,-0.00011665053064681337,4.4515144731944084e-05,0.00032986848614194764,-2.3750289248535315e-05,0.00015871327612490595,-0.0005816819579582146,-0.0004562605299169627,-0.00017954193773203836,-0.00014457650118343958,-0.0005618961856016984,-0.00040482129050595483,-0.000324653869070352,-0.0002415820925710578,-0.00018783960849647492,-9.471815428688578e-05,-5.842482307182821e-05,-7.251817144001656e-05,-3.25290945127589e-05,-4.584348733577523e-06,8.46113024106278e-07,-3.561712736662821e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-5.147348996161022e-05,0.0001536172386761608,0.0005002007942582142,-0.0007081007529684085,-0.0022268951680930065,-2.5862831468964095e-05,0.003823273314443005,0.003172620262698333,0.0005160884179541478,-0.00983765105733953,0.2880345585840039,-0.10228827684014749,-0.13239613822977137,0.05027032836937329,-0.028663420654312658,-0.0035029916114188557,-0.01345033591493229,-0.027325725272637262,-0.02470329009065806,0.04999153461873163,-0.019603276586409146,-0.022854759666889067,0.01473617329825725,-0.005574976022919001,0.0012778484842030987,0.002148143189999188,-0.014486890459216157,0.0012742335793761192,0.00619324852380171,-0.0056477832126157,-0.0007408250459713951,0.0037791665136965935,-0.0008261062961672826,0.0005229156619377902,0.0008235118326650209,-0.002187563384914227,0.0028711976969487004,0.0035654181165008557,-0.0024122855574049543,0.0014711241530119316,-0.0018302362541288604,-0.0025827767096108095,0.0031932455984371344,-0.00029894076491372587,-0.0008387740561681646,-0.0022843455916740194,-0.0021571968262210288,0.0005598879654251914,-0.0004956580925670266,0.0002979235593891559,-0.0021174254989825433,-0.00404104212868469,0.002296717596315017,-7.17755321826544e-05,0.0006878888101154118,0.0009095495049749559,0.00017155468883512856,7.661526927366452e-05,-0.0006558255003514302,0.00030020497750717366,-0.0007520301171706988,-0.0005894647274278854,0.0007568541485368204,0.0005671102153130458,0.00014867810104683273,0.00033717851989148825,-0.00013675671655161378,-0.0006159684193567496,-9.685418316084012e-05,-0.0003354525513273881,0.00010275781219362658,0.00010277631087699834,-6.738747107371553e-05,3.003159815215454e-05,3.560968732503068e-05,-8.042093850405412e-06,2.016795945180587e-05,6.1033805304594165e-06,-4.28239477517152e-06,-1.0443053257724153e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (300,20): np.array([[-1.0275267137280804e-06,-0.0001925067860616751,-0.0005556545606144108,-0.0006550832085583584,-0.0006377599078964134,-0.0006050921468845793,-0.0006618622433801477,-0.000727140883405954,-0.0006694922667371599,-0.0005540862600455356,-0.0006600248273457561,-0.0008220168047565958,-0.0007006273804354129,-0.0003451826012441033,-0.0002458549389169309,-0.0006335024170193773,-0.0016887705590700175,0.03909538556515638,0.0361685567641012,-0.023451877562454197,0.01490291791393913,0.0038444344751481702,0.010858219846930852,-0.004942775352444569,0.0002187801048378141,-0.0013261475599535067,-0.0052786089863118835,0.001723679986876258,-0.009158129903046774,-0.006799051961552787,0.0023351883455876216,-0.001327085066304704,-0.0038185537881818233,0.0007652458336937431,0.0030074475949762374,0.0003967269095370405,-0.0009729940963111734,0.0024423550613676347,0.00022026485000931133,0.0011196097150876114,-0.001447944471901159,0.0010115136762351685,0.0010664209116554577,0.0001717497777154236,0.0022040960658932544,0.0002208831011313755,-0.0013785373812500927,0.00014592416439390191,0.00027088568342763844,-0.0006502602743730043,-0.0010251653439754857,-0.0007052051313120616,-0.00029112085619541797,0.002130104187302281,0.001195845679614029,-0.003603816266748998,-0.0034543603551993427,-5.243141026629071e-05,-0.00048792898261087753,0.0014007570290345767,0.0001878586808989108,-0.00032218575804290135,-0.00016734579352385554,0.00012398017141404912,1.4969269286992941e-05,-0.0005571639329893305,-0.0010524137789243218,-0.0006939575995880424,-0.00011019442569039226,-0.00023638518676120745,-0.00022548391900380812,-0.00021655685128800216,-0.00022764178640946554,-0.00010683359244765026,-3.5662151257060566e-05,-1.1342014015565656e-05,-4.330664761321304e-06,-2.9813820417380807e-06,8.906891355186254e-07,7.315317583768016e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-3.081103406238552e-05,-1.0207523632835506e-05,-0.00028802345267705235,-0.0017183093727368263,-0.0013075720464138364,0.0003763834630484105,0.0006715845716434757,0.0005745618811410657,0.0020428864508653375,-0.005065808295210483,0.21334017008013434,-0.14159272434192777,-0.033232968596530424,0.027681495731425202,-0.05792329120913612,-0.013370533417633725,-0.015284709024944958,0.016734818625011298,0.023670870545559785,-0.016813509117766908,-0.020913743752366376,-0.007228455209592085,0.02190674851828988,-0.00837604330935498,-0.013983423131534365,-0.009909919201479675,-0.0032383518360985467,0.0029934712690103094,-0.0019514373581949102,-0.01412330028315046,-0.008374914839374891,-0.0001324708374599655,0.0023501890065529287,0.0009024467889405745,-0.005546363870524087,0.00033602047078687853,0.0007648460961535708,0.0003089451081463795,-0.0018860799521989422,0.00033468500983110797,-0.002017785818378021,-0.000668949058715977,0.0030395171173558046,0.00044613077940904955,-0.002364738203723473,-0.0016771903440396393,6.143203246109472e-05,0.002703416594422719,-0.0026304869655029398,0.0005214118339175589,0.0024787401791097445,-0.001159179985028173,0.0010425247175208409,-0.0023206231852524618,-0.004117919270707671,-0.0016980657530870085,0.0030584719188562807,-0.0012784043153375062,0.0014114566729841829,0.0017297109867103952,-0.0023558828043777993,-0.000582895324846671,1.5118394594430903e-05,0.0003417060287509355,0.00041785043687852104,-0.0006895587117757842,-0.0001932147076136873,0.0001436426414742831,0.0004476365800834255,8.82660876853975e-05,-8.52257908353212e-05,-0.0001251097778660789,-7.406510650399675e-05,6.709177198339453e-05,2.8800621752507232e-05,-4.346948923731689e-05,-2.168536849599067e-05,3.2954577631717845e-06,2.0246063738927602e-06,-8.717044231465549e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (300,45): np.array([[-2.61854974843681e-06,-0.00020644587654974864,-0.00043021662431642384,-0.0004943083829584297,-0.000564203709576527,-0.0005096017716339656,-0.0005067185522337997,-0.0006248200366788666,-0.0006567168050492596,-0.0004911194590868701,-0.00034224363168689994,-0.0006553246966038984,-0.0012329655693303428,-0.001190425384794257,-0.0006678703260706951,0.0008640423401857736,0.044192010208368326,-0.004158838372684806,-0.012476379319101847,0.012647469583641869,0.005870061000993606,-0.003172706970624325,0.00020926671679353054,0.0038855301762668443,0.0026296179845872554,0.0018562753596799754,0.003651685103339111,-0.0011830297755145453,-0.0009955257135211102,0.0038930971981219844,0.00010060683650274047,-0.004004321562385566,0.006084486528476881,-0.008236464406028834,-0.008976465560020016,-0.0013358550347231369,-0.003379652309487274,-0.0003624927846927474,-0.0011888057700654132,0.0011731705622173316,0.0007375991186868328,-0.0008123718864927544,0.000974467861189248,0.0014087421014189385,-0.0010236423685957053,-0.000623490105722886,0.0005397072632054486,-0.000843410857822868,-0.0012045280323205512,0.0015820308259704178,0.0010706637315839509,-0.0004516088542815196,-0.0007819668536323091,-0.0002998102310918388,-0.000814785293420208,-0.0014254686399717605,-0.0008732841229445464,0.0003097162548635137,-0.0005740510265900806,0.0004836749593060702,0.0004298348857820184,0.0010844997494694796,0.0008616644502487343,-0.0014571805762023527,-0.001015003634727207,-0.0008094662933176188,0.0006494027175213566,0.0005669320335097393,6.923101430255633e-05,4.0414758009968134e-06,-0.0001765645900163488,-0.00020326601776564382,1.1986221538554947e-05,-2.683375611349103e-05,-9.432950478147111e-05,-2.604312020960184e-05,-4.0938087538048025e-05,-2.5948583447507866e-05,-6.3250129893706786e-06,-1.6577687073261432e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [7.560275081434621e-05,-0.0005733572475218798,-0.0014888585989818886,0.0022337017913189863,0.0029037803075475594,-0.003206262845373965,-0.005083284069858352,-0.0013464263334659228,-0.005106667183336187,-0.008963664905909021,0.030203400861070193,0.28171756452071856,-0.25394949192471933,0.04678115304603836,0.07188571122158802,-0.018387979613282507,0.007013501609703039,0.00831063625292214,-0.011387929521251209,0.019426288776481764,0.019200992101791034,0.002599117074667305,0.0035821005566736602,-0.011693755590372069,-0.013786602327472319,0.005613824234707593,0.0021823038825227255,-0.01689883776018935,0.004366161267003553,-0.0005247034548005238,0.00012028485687578161,0.003862264364333515,-0.006815450139659054,0.0025905013001782094,-0.0005868221371665122,-0.0019142544313363477,0.004511192387218349,0.00019326577180273807,-0.00352827187578624,0.003033131214389884,-0.0002943253224562237,-0.0014165170912746705,0.0023553564467959564,-0.0012589086599382384,-0.005310243810137785,0.0004415146540442312,0.0003810056372499297,-0.001138687417043063,0.0006401269419704625,0.0011732874386893902,-0.00015688578284170292,0.0037095881413873517,-0.0004166799737995404,-0.002881916147073503,0.006132817416655838,0.0018996950584278531,-0.004138688429088024,0.0053810014183192315,-0.0022536512447540406,-0.0030555868697556654,0.0011660112373761248,-0.002334508825271108,-0.0014898839750257394,-0.00031478793105133127,-0.0003879781987353919,-0.0006823488003606495,-0.0007565676702181163,-0.001094134536113593,0.0007898172666771912,0.0002794478532583013,-0.0008375991661184397,0.00021590936106459133,2.0903306208693675e-05,-0.00012034718541474814,7.071271788670929e-06,-1.799448060424955e-05,-6.9854778482584225e-06,9.722047438040615e-06,-3.206624086854934e-06,3.7946831334493894e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (315,-45): np.array([[-2.7695941236060966e-07,-0.00020778682197687687,-0.0005798923662801441,-0.0007194128162571583,-0.000730623006690812,-0.0005696566720682462,-0.000661656233185752,-0.0009159823789715692,-0.0007323987317477372,-0.00047297906106421054,-0.0006920915064811306,-0.0007732035939528941,-0.0006645536979038101,-0.0016421752783466277,-0.0031831642537945844,0.01288170870288553,0.01215573216114604,-0.030208848157251438,-0.026873949617627586,0.025707357121482595,-0.026993888317082326,-0.009518429504027835,-0.007691251164300184,5.958083435710681e-05,-8.793067148886864e-05,-0.007315236846254111,-0.00428691788177087,0.0005867862825403446,0.0039292159803024015,-0.0023984846707452542,-0.0008782643616711371,0.002126788341668317,-0.001626669161030418,0.0022570173359288187,-0.001119095163496652,-0.001231459716356739,0.0017669723215371841,-0.002183541914459838,0.0011987185517676523,-0.00014287231425088966,-0.001007962958526075,0.0010794974622795473,0.0007472397600853673,0.0006549697218299935,-0.0011591160742916455,-0.002156093886308323,-0.0004236291339566497,-0.0006496959347462343,4.716070240750897e-05,-0.001841417195246761,-0.0026012400366136564,-0.002213480886845276,0.00010156575639399518,-0.00015111136078778872,-0.0013334313061042146,-0.00041736737060616027,-0.0009796366640687248,-0.0008416745721670331,-0.0007546422756852857,-0.0010475412180993655,7.7172418601802e-05,5.31957297568988e-05,-0.00021138996251317263,0.00034517848929603085,0.0002926068185384077,-0.0008737843964009491,-0.00033358646082807305,0.00027057888082445374,-3.148306916755186e-05,3.404673023470605e-05,-0.00016155502014357766,-0.00018449915853025427,-5.935375419812797e-05,-5.728524332881793e-05,-6.1558503088669e-05,-2.9343884792742254e-05,-3.6514440245607943e-05,-2.0157036731727066e-05,-4.741559307502185e-06,-8.144153229902746e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-3.2797787296016735e-06,-3.531337905887719e-05,-0.0011760664366132108,-0.0008977907488987286,0.0002493655718545625,-0.000938913860907512,-0.0013758666432455034,-0.0001787645023879935,-0.003602505138066953,-0.005343866980769901,0.014474166794356614,0.2518261455684954,0.060444899687649455,-0.09724564173632051,0.028821836853336454,0.1820760545445115,-0.11719011875797859,-0.035805405467217916,0.024901676990595735,-0.010256632841682767,0.023708673830215115,-0.013851717876559193,-0.02520016622866305,0.014271715790154287,0.008698563138738128,-0.004173646757024024,-0.005526484516177481,-0.009772870400000905,-0.0013233350598686974,0.006917786708656529,-0.0010178145877010644,-0.004511824060500515,-0.0010091501344681594,-0.0002649616522682556,0.0040340653986407505,0.002837510708927148,-0.0035067631749846045,-0.0004870816281074304,0.0021672207450181146,-0.00031581641359635916,-0.0008044285829766335,0.001304656270111502,0.0006459153873024767,-0.0057272400258299395,-0.008288000519565446,-0.0016318588534885339,0.002810404853213268,0.00119184794614039,-0.001184789852782935,-0.0003980684103326673,0.0006115914040352935,0.00031167424320435814,-0.0011183464728869875,0.0007000036554394895,0.00020231142320035504,-0.003206939089649779,-0.000946473759197383,0.0008949078503306509,-0.0017110493501654362,3.980023859694746e-05,0.0015486640844882143,-0.00037467216740965894,0.0007853061438601681,0.0018939480043506235,0.0001574657987392922,-9.606502541166741e-05,-0.001379672957362106,-0.0008426370218784738,0.0002963914482526119,-0.00027979340966085436,-0.0004023514878385972,-5.573862494762747e-05,-0.00020456226736696275,-0.00010745597289090237,-1.1471402266936049e-05,-3.943470453250966e-05,2.1100560141110393e-05,3.445371228168482e-05,7.187668481947401e-06,-1.4490396309495254e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (315,-20): np.array([[-1.0457239666890324e-05,-8.613070641478052e-05,-0.00042099257583658645,-0.0008277026225994733,-0.0007747420097320038,-0.00044541642364215057,-0.0004912466827005251,-0.0010066176394939765,-0.001060642824050587,-0.0002196224246019282,3.9006110427321434e-05,-0.0011810413110566534,-0.0017879100341550824,-0.00046891087055433436,-0.0006448722245389873,-0.006170704882925001,0.021129261555385418,0.006731621748484462,-0.02662949984883449,0.008293117902577429,0.021369209708902465,-0.014209059052349815,-0.006651901475184657,-0.0032745019644758034,-0.004739048311721712,-0.0034707071826357155,-0.0003620065478472035,-0.002983090171675753,-0.0033000824961358106,0.0025080169155293577,-0.0037011699995674443,0.0019571097020302554,0.0012683199648131065,-0.00016488272731533872,0.0009047037471344702,0.00031358588463585097,-0.000359426448393761,-5.085070893311842e-05,-0.0014091193318731565,-0.00017709635551503797,-0.0003658533934010568,-0.001203809859297722,-0.0013803859741114545,-0.0012719746354470155,-0.0016027688016025182,-0.0011960395608709175,0.0006547361381066373,-0.0009068525673034017,-0.0005622167344847545,-0.00046925595503007886,-0.0006388844225937401,-0.0005210523943136592,0.0010350960550032221,0.0002022710451217357,-1.5146240113442982e-05,-6.627304725820089e-05,-0.00018595058921547656,0.00029960139341045675,0.0002498749463543604,-6.698978820574296e-05,-0.00040011433009942064,6.427822595359135e-05,0.0001583822012560553,-0.0009118457807961596,-0.0007801763161039759,-0.0005127240953385795,-0.00025805127625555454,-0.0002802901657132055,-0.0002782962097989293,-0.00026534302415486796,-7.368048697587821e-05,-0.00011484216400939472,-8.855358057215265e-05,-2.8415879141352173e-05,-2.6334710265254372e-05,-1.7150373754942357e-05,-1.4534979158072638e-05,-9.26733509720787e-06,-3.9596005778508675e-06,-5.694708518888079e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-6.842984332814068e-06,0.0001523345702821645,-0.001962862365839401,-0.0033669976309712276,-0.0001272273991633461,0.00147480830078581,-0.0014950615646941614,-0.0020714734568811305,0.0010691245935804568,-0.004407763122484699,-0.04575373063307523,0.12731673823644124,-0.06591778878151211,-0.11198954288983852,0.16442414101383604,-0.0020415173849436864,-0.049673617422699705,0.02930003254747987,0.03229991310292902,-0.02179706811606949,0.0008948741187626287,0.006138723664909823,-0.017398957280159347,0.0034990415892943902,0.006869908316311177,-0.012476070601668448,0.00034684168948992015,0.005113916625642725,0.00023179783466296876,0.0008668360651284727,-0.0020610506624158293,0.0004445155671210961,0.0021465409955920113,-0.002220079534781796,0.0011454665050958566,0.0001384373357775073,-0.001543630177260077,0.002919990063790259,0.0009122433720195796,-0.008391115477846591,0.0009713540152372681,0.004529260484570515,-0.005629258282393235,-0.003002926623507268,0.0020573325587679757,0.0008906336300299724,0.0003146432654024655,-0.0013971126476801307,-0.00144541322857634,0.0015955482351090594,0.0006327103591763138,-0.0020348245749040937,0.0009727061225782263,0.001578116896206777,0.0006780360349307414,-0.00040035583352655526,-0.0020331433351211815,-0.0007127643868526007,-0.0021452710429729497,-0.001015430107213591,0.00011409881665663456,0.00038915620195423804,-0.000542101487082359,-0.000569107275629474,-0.0003281278181523795,0.00021230233541166477,-0.00025434192558330927,-0.0005167343782779973,0.00010490265018929793,0.00026327821300674964,0.00012844344815254497,5.845342904200654e-06,-4.812665788981233e-05,6.428593867112707e-06,-1.4384878954912034e-05,-2.818236466775335e-05,1.168524277023885e-05,-2.9080408943184855e-06,-9.224931047840453e-06,-6.494924118618345e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (315,0): np.array([[3.082197174067199e-06,-0.00021730449456594525,-0.00045816525465822464,-0.0003972335405611796,-0.0004956863554761588,-0.000698558490630991,-0.0009623349578350073,-0.0009164419577041203,-0.0006417656742785976,-0.0006261055880006044,-0.000883519855220437,-0.0006519459254331113,1.9259672922114122e-05,0.000158127565433887,-0.00024796260068477195,-0.0007117097679687204,0.06802619527145506,0.05552014508835747,-0.015502837393203173,0.007408795099464029,0.00013686794243633778,0.0035022605770933268,0.0033462991043699407,-0.0040474273566788975,0.02113276133159257,-0.00332131779037001,-0.00992770186128067,-0.002498678021710329,0.0033276053224203894,-0.007553985424632778,-0.0029842572710736663,-0.001609291971951278,-0.0020084548952244365,-0.0034960355947534286,-0.003994768596534798,0.0005471783713795753,0.0014786467427744111,-0.002000164975498655,-0.0007962696473381919,0.0033851560200104883,-0.0005027761397965497,-0.0015089513358668171,0.0002904338146687891,0.0015639634870232985,-0.0005769152319174105,-0.0003784995050271622,-0.0015377985094925215,-0.0013658691818502586,-0.0012651478841769564,0.00025681508334945855,-0.00020197630087699295,-0.0017386630732968843,-0.001000233246494471,0.0007729460050688254,0.0012823289595648923,-0.0011839927630206948,0.0001929261728173141,0.0009154743090211655,0.00047119945559656476,0.0007785827808068455,-0.0005102975355296663,-3.5258134001274175e-05,-9.3703778821164e-05,9.383430913306348e-05,-0.00022822296807730868,1.908901467756355e-06,-3.8966201582589054e-05,-0.0006759675445854704,-0.0005189430161952977,-0.0005324553966570343,-0.00030053962867715233,-0.00016725895329655058,-0.0001030319934558932,-4.6154779691151065e-05,-5.615058371338843e-05,-3.793647299387214e-05,-6.711209965959518e-06,-8.823985632168506e-06,-5.324278484071423e-06,-6.967369238555033e-08,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-5.0283466162199816e-06,0.0002811850474968831,-0.00027776052386214957,-0.0015733450524854792,-0.0002919828622756053,0.0006184149522214398,-0.0011320654376941,-0.0011464772820437333,-0.0010080181591247122,-0.010042949498420323,-0.0225899514461876,0.10629876298708746,-0.1805037262486415,-0.01764983407734713,-0.019420462200735275,0.015016258630475496,-0.02072177622623931,-0.049730820754367966,-0.004199539724397665,-0.0279100913961096,-0.007230403776397602,-0.01984796152610324,0.0017020822507423335,0.005833759905113501,-0.004105265051294654,0.002630794201588016,0.002968398503435638,-0.0012323043088080612,-0.0036273588280752852,0.00477700750139406,0.000329997461732155,-0.0006377056255207523,0.0003989212534813798,-7.496271268870894e-06,0.001840882169694723,0.0006295064568707757,-0.002128657059494044,0.0023610796572788783,0.003558255653456866,-0.0028422732508261027,-0.0009584171263356578,-0.00289984225326734,0.0003267262373272075,-0.00035596175325001056,-0.0033809866799471345,0.0010725323670189408,-0.0036358551655917746,-0.00021705580563605753,0.0021267778980867064,-0.0009455365003648256,-0.0004926743268017388,-0.0015313214452116386,-0.003955469262089616,-0.0008096063787024326,-0.00024926915954024103,0.0024723763699181743,0.0010096613042254307,-0.0006382846175966201,-0.00043205461113722255,0.0011130004370701841,-0.0009516471119040324,-0.001564454422468075,0.00016568335011575162,0.0002795402476304915,0.0005566220352606114,7.999249744864196e-05,-2.7526657666405703e-05,-3.316580593294633e-05,-0.00034071128535285126,-0.00020277181667940441,-8.106167531466903e-05,8.659306747015237e-05,-2.9527157428446114e-05,-2.6986928728201974e-05,-2.7584644116685335e-05,6.76893799084697e-07,1.5056062299141755e-05,1.3599252719690016e-05,2.351670180689055e-06,-4.2543677262855273e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (315,20): np.array([[2.3010439561942974e-06,-0.00023551397496548573,-0.0006358394767380382,-0.0006519392562529741,-0.0006190239194296189,-0.000713310350292367,-0.0008006965666025522,-0.0007303805159919272,-0.000559234288060178,-0.0007421625123410343,-0.0009997492732752979,-0.0007354495714092627,-0.0002189438361764283,-6.939650177034699e-05,-0.0006638259050088441,-0.0033421963568711294,0.05914802199326099,0.030294492354165037,-0.029964738190728413,0.03220177343950196,-0.0030283788221359847,-0.007350993798454999,0.016648444956146757,0.009518315840551945,0.0006463478026286309,0.004213892044146674,0.01575585076539796,-0.015376143952038285,-0.00594327516515543,0.003984709229535592,0.0014227026388245942,-0.0031911009124767995,-0.0015640533998727525,0.003698560050531961,0.001382862392983396,-0.0028419873750376618,0.0008429364743650955,-0.0010417713680652074,-0.000457181644988468,-0.0028261954361179747,-0.0018776440779463839,-7.690998415025732e-05,-0.0002391831874855693,0.0016692307779356097,-0.0007301737875238941,-0.0003396229785890488,0.0008695215836642674,-0.0010029004481981565,-0.001018938636174925,8.23515637971833e-05,0.00015929398042413616,0.0017279341433376538,0.002186901228498164,0.0010711296127628633,-0.003811573847678976,-0.0029209688491359967,5.447949489475853e-05,-0.00022297076571546593,0.000713298549208817,0.00017131170407770703,-0.0009260367258960746,-5.909136522719519e-05,0.0015330494244785275,9.838019679979065e-05,-0.0006414668982442814,-0.0007152416459297728,-0.0007470768934081691,-2.5434939477010825e-05,-2.090256999863546e-05,-0.0001770957745992898,-0.00040892788692710864,-0.0002468883166126597,-0.00012830873625989714,-7.81560260149659e-05,-1.9556318768092774e-05,-1.9764110775851527e-05,7.714876589825657e-07,8.448056447394577e-06,5.298315972899468e-06,3.0009275886362964e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [5.178429007917695e-06,-3.0483808902527672e-05,-0.0007893396907059695,-0.0008431777750229554,-0.00014983141939426593,0.00018933936853505375,-0.000931496605450943,-0.002201902620018799,-0.003822951706700624,-0.006912598158086622,-0.013727016533933852,0.2116220472808397,-0.28210060269533166,0.0241729117451458,0.03486351652374504,-0.0169767451143489,-0.038707447490078276,-0.014661796777516735,0.02089577329037593,0.029044417333618566,-0.014462541410362835,-0.022180024950905812,-0.006330268253003916,0.02940839083039361,-0.01475608968769422,-0.00465766776562979,0.006672376800497985,0.007163337517363332,-0.0017557593276384992,-0.0037507026046711993,0.00894076464062133,0.004701560517312389,-0.0076411000581048305,0.004864513833225434,0.0005125738518417729,0.0013481118738874847,-0.002512061857246428,-0.0008437378599406988,0.0014507464960935143,-0.001329996117146497,-0.00036537774438758663,0.000985876199367227,0.0005038023957153289,0.0017254158401660644,-0.0008075767023627728,-0.0010518237491683384,-0.000529047316502144,0.0011861759387677808,-0.002761720390067214,-0.003015195465771363,0.004735264062537436,0.0017988930085333227,-0.003540111421825743,-0.0025723779085945146,-0.0034466214966078507,-0.0034412344187231135,-0.00030446238027246486,-0.0010011125601263654,-0.0019276359121291392,0.0021196069978345433,-0.0008224758463424366,-0.0013474904837372003,-0.00018377975814658587,-0.0007615765975932714,-0.0005897487345868122,-0.000613877617345702,0.00031443131314583505,-5.104912323856319e-06,-0.00012460307174457522,0.0002545061405281905,-1.4326854328252902e-05,-0.00012655073273011097,-3.3428187887617044e-05,4.182801439434895e-05,-1.0767470064271343e-05,-3.240385070883843e-05,2.5402702077302378e-06,3.6461323600209016e-06,-2.2090525756875254e-06,-1.30569659171542e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (315,45): np.array([[4.867515454004082e-07,-0.00032775953945341207,-0.0007754128605634245,-0.0008152230086200601,-0.0009240546375701364,-0.0007881089226699595,-0.0005954654087134365,-0.0007783541590953297,-0.0010969493786994735,-0.0008552374717419424,-0.0002561150058567604,-0.00015590982999932993,-0.0005816019281856634,-0.001145000453830432,-0.0041177993710854385,0.030250422623197555,0.038960462358754035,-0.05492289123032477,0.006313907173507067,0.002883187436917512,-0.007596096098914646,0.002312716830674975,0.000847146566228531,0.003646567575932804,-0.0011712344168267456,0.0009487054384965916,4.161702685854661e-05,0.004512261065709627,0.007989284241699618,-0.006624313310588654,0.0046159335693781,0.0038347479572552715,-0.010698751637223312,-0.0032258118992222425,-0.0024007533865644643,0.002140894559724996,-0.003707111300755508,-0.001961700318358402,0.0030308227737534184,-0.0002705632230140967,0.000540492221493945,-0.0015756754373952052,0.002085607557616327,-0.0002674826635450775,-0.0010361581232505525,0.001091576157868988,-0.0011472069594375286,-0.0011747522474225057,0.0003077863383912872,0.0005746219499522548,-5.400757576496797e-05,-0.0012532353496351423,0.0005140368264362118,0.00022461128686993934,-0.0010398983667351481,-0.0013347414479788504,0.0003581079473011423,0.000126414801558354,-0.0011727313944609706,0.0011432625335161713,0.0014892794199908544,-0.0002098682921560588,9.047260860990009e-05,-0.0024518959207414487,-0.002172421473332492,0.0003843825161360021,0.0008170678475839145,0.00022790062433526275,0.00014554608932993438,-0.0004082433351751554,-0.00027659694861558294,-5.008817505469006e-05,-0.0001628194618080247,-2.3033117584400564e-05,-4.4935278260041835e-05,-8.448330471365548e-05,-2.675758175671737e-05,-2.0094741476611123e-05,-9.78146827006026e-06,-5.95484173025153e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-6.151045376383155e-05,-0.0003451394925316785,0.0004238805169563601,-0.0016570612682511624,-0.004635224662608441,-0.0010927608795616487,0.003647465382340039,0.0011847193321626592,0.00048320295523609813,0.006956558855931901,-0.005774640452509382,0.12898961868063527,-0.12026694648175867,-0.02952418324105861,0.061028080426279316,-0.07232599250749708,-0.020474005391054574,-0.007417056681814148,0.011804596769633033,-0.006906876147448496,-0.02312754861862276,-0.008270923199484446,0.00220077799271247,0.015008354363643102,-0.010080236056279702,-0.0043971724049849245,-0.007739846340570052,0.0073786421222315855,0.011410489053898114,-0.0031601655542764476,-0.00013244062972603342,0.0009429245431609007,0.005820314468255341,0.0005973294897293248,-0.002562872926422487,0.003995199838501289,0.0038980618130737745,-0.005347864707964545,0.000689804755096497,-0.0015435466944414954,-0.0013670731095497535,-0.0010694498274071477,-0.0015950994313612007,0.0003113184702445783,0.000549170946278655,-0.0030978466082101645,-0.0005170590382829113,-0.0006778925629867405,0.0005162116623517834,0.001827385878528416,-0.0008444424509443919,-0.000880361159835378,-0.0020520803181581313,0.00024225589494090668,-0.0024304648736167703,-0.0014767073335560288,-0.00020642861259144183,0.0015837663057347811,-0.0013437827062794513,-0.003402970084668894,0.002844806670982494,-0.00046259338495168687,-0.0018485967562434757,0.00015610091726953789,-0.0007659233450348255,-0.0003085598472066745,-0.0005802830805197844,7.391733526587786e-05,0.00036823663800705596,0.0003227580533151989,-0.0001384597055407866,6.650723648404681e-05,-0.00014296144848630353,-0.0002095087525937439,2.939509740160289e-05,1.0095469325430862e-05,-6.431213921047674e-06,-7.917844713267342e-06,3.955393124839883e-06,1.6850238131511796e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (330,-45): np.array([[-4.529042668265766e-06,-0.00020494000650408438,-0.0005362902687423703,-0.0006758575435161156,-0.0007205516503286811,-0.000704517315054627,-0.0005756536764991396,-0.0005951999863239351,-0.001072061582195137,-0.001236254887439787,-0.00010190875403094283,0.0013615930586125342,0.0008261198790344041,-0.006479810048936984,0.06710723198607411,0.06298913818753639,-0.02547880886924705,-0.007248036602285082,0.02465791032191893,-0.0008691671975111657,-0.004051944415000379,-0.00033774434218468555,0.005042338532230406,-0.002458378554766299,-0.00423196999091577,-0.0014663582508819392,0.0011580658385039384,-0.0016688564322000692,-0.001954514461697006,0.0024153233294886165,-0.0036680198050728628,-0.0014619643715014436,0.00168011401182938,-0.0011388896195446564,-0.0002694806688122483,-0.0016577710185851898,-2.483144241699853e-05,0.0018680890438555065,0.00013991152138934361,-0.0009925191908348213,0.0003814645774664387,0.0021625303932155866,0.001669092301640691,-0.0007590279071117132,-0.0013736621351111088,-0.00043564436671211787,0.0003002663424316966,0.0004536937692863269,-0.002761183700124735,-0.0038270127046087917,-0.0008930848988357676,0.0002771973996066299,-0.0016818159315275573,-0.0009347388212745186,0.0004036664741507078,-0.0007022887795796986,-0.0005156249844568312,-0.0003687375461496594,-0.0007547863155140096,-0.00024869484695377685,-0.00018996552665492537,-5.697928513588474e-06,0.0004993282185663565,-0.00024862882098811097,-0.0007929894989095875,-0.0005527149185883061,5.710977511482635e-05,0.00036900704808515936,0.0002644553930657195,-0.0002623554974019489,-0.00017341399171976246,-5.980679939755328e-05,-7.307106670947226e-05,-4.773951564327839e-05,-3.480817144451039e-05,-7.568741854987447e-05,-4.881363144095328e-05,-1.6112471692938422e-05,-5.1722529137428405e-06,-9.855924855770961e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-3.5617909818118634e-06,-0.0003466405635118161,-9.923676634847846e-05,-0.00018232467373243354,-0.001565618061647785,-0.0008287419036960154,0.0004527549529738993,-0.0010935271780658918,0.00013375637766120264,0.005019593380548263,-0.002252202305038616,-0.04032578656840083,0.03823620302014948,0.02300714047837649,-0.13593604829620604,-0.10325916701551656,0.07276557457229096,0.008719450024407,-0.07221323746468458,0.02325510776718336,-0.010904174708423973,-0.014161139748633336,0.027488045596409175,-0.011395503214987474,-0.01758943320493275,0.011518393529513216,-0.00890540631952534,-0.00152752277645537,0.002112655255338447,-0.008354418628603134,-0.004647974276533897,0.007082750243089615,-0.0033852376040550652,0.0014680865750885982,-0.0002574263776102023,0.00044461533567365173,-0.0003553932177064475,0.0010760662498210095,-0.0030088184944587044,0.0004257751448915768,-4.8366438605423964e-05,0.00014180809550113385,0.000606187318940439,-0.0005916286798421568,-0.002657006847031424,-0.005327150627920224,-0.00019011289000122846,0.0038322413382368753,0.0011950514285224464,-0.000391172298783925,-0.00015245795551246667,-0.0008718826251233215,0.0004417755796311799,0.00014312329517295654,-0.0016807917216758793,-0.000896647271416312,-0.0009757607321123072,-0.001050259522291384,0.0002558383290232328,-0.0016525352404491505,-0.0007965449149858601,0.0019526260552271764,0.00023279521904992568,-0.0001960957106755916,0.0018008779476907993,0.00023931929907842966,-0.0006626103218635449,-0.00034508713319955067,-0.00064114903638173,-5.9032416077287824e-05,0.0001775337151231112,-0.0003196096612359743,-0.0001482995093696871,-6.294357824617567e-05,-0.00011826923955899023,4.371047954757923e-06,7.298824962532276e-06,-3.3147112735088126e-06,9.516608979789188e-06,1.5496205577797492e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (330,-20): np.array([[2.252672508759135e-05,-0.00022813480416929247,-0.0010414761052371865,-0.0006786201119648898,-0.00011058089133058073,-0.0005790121874543505,-0.0014413718422232446,-0.0011459893514799448,0.000188043046717585,0.0002137400123350455,-0.0020651005955321937,-0.0030311872279818347,-0.000281366476335237,0.0003203643545070385,-0.007988371386410675,0.00918224910973219,-0.02201385142267327,-0.03404767839607681,0.02797935344806817,0.011994165209933594,-0.016643225179900516,0.0015905895566085274,-0.003922149485122437,-0.006534227548454567,-0.003440635712916746,-0.004205710128076446,-0.0010355864434247574,-0.0066407555809668345,-0.003978267546387117,-0.00040261218440085884,0.002393701804497377,-0.0003774627031903216,0.0011333709149423802,0.0025481829307740485,-0.000895937662957143,0.0007578255224684789,0.00030281310619367265,-0.00043660181202634645,-0.00024496765905353626,-0.001303612169827175,-0.0010765550189136708,-0.0009639713246049328,-0.0018225911451143603,-0.0016463616182631742,2.6045369841657058e-05,0.0007696746339032062,-0.0010554462167292728,-0.002952612374571026,-0.00031973867723976006,-0.0006392341580125045,-0.0005367854782748665,-0.0007167880460076807,0.0013150478984339513,0.00024127562505663587,3.83607503408826e-05,-9.557607054963067e-05,0.0006970882058629441,-0.0007229869111690774,-0.0002506706512115685,-0.0001190951619079757,0.0008972009316732161,0.0003416816445989685,-0.0007079255063652897,-0.0006603448694322248,-0.000348050515756332,-0.0006648733060002615,-0.000521113859315444,-9.490410812509111e-05,-9.764408581909077e-05,-0.00010099087291934626,-0.0001592200506839272,-6.896692408299657e-05,6.6264650727137205e-06,-4.636653620323722e-05,-4.759026117633835e-05,-2.5232609520596286e-05,-1.7340276791562513e-05,-1.2405390441955865e-05,-4.956834305281599e-06,-6.875108140065868e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [8.851648298255298e-06,-3.619677514540575e-05,-0.0008225493156414495,-0.0009623661315887359,0.00012248605937735318,0.0016202798422084242,0.000244496471108929,-0.0055185802617677745,-0.007000425536186844,0.0008655261711160412,0.006915867244453951,0.04494688115420092,-0.04147676992980996,-0.13637930821937425,-0.020989638296355265,0.04835449348207511,-0.05893240069675227,-0.05106990780609648,-0.01423432591470272,0.0027329711193739103,-0.013496758610727994,-0.007946984247532126,-0.00524011726476185,-0.003838082049780381,4.3778786219210314e-05,-0.0006257006770989811,-0.0007081959532709331,0.006476552251516367,0.001985874078560421,-0.00243905799208112,0.0008908931594141914,0.0031310900308140106,-0.0023843595814657934,0.0015471306239684922,0.001769137071525387,-0.003104968172014899,-0.0016563787179055868,0.0004180518747242361,-0.0014714576172977544,-0.00392838757386798,-0.0047790525097379825,0.000554621812646359,0.0031216998534668973,-0.0032218454856035236,-0.0027540880920674943,0.001431261924645619,0.0008133481527721737,-0.001190403533267844,-0.0009354325159368382,0.0005828731223347413,0.000529473606549696,-0.0007089643098388522,-0.001067264401947657,0.0006132744089263993,0.0011308412187512632,-0.0002729061190577386,-0.0012485479216765459,-0.0008693519960889589,-0.0011815758729247763,-0.0018409327199812408,0.00016052933194925942,0.000827224787641205,-0.00019851661366376952,-0.00017830356903145029,0.0005403991336540193,0.00025237623157336553,-0.0001579588077673296,-0.0005767878969137242,-0.00023541437057802812,0.00026304830565174913,0.0001545500264030908,-3.419536387787555e-05,2.9653807866686564e-05,1.1661992558509322e-05,-3.669864605091147e-05,-3.0119232483707433e-05,-2.381777072511103e-06,-2.4019168176202286e-06,-6.03144528380465e-06,-8.989347239167084e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (330,0): np.array([[-3.950106674885155e-06,-0.0002016967513079765,-0.0002351551724922306,-0.00036144797155406505,-0.0008152899509905438,-0.0009150310353548821,-0.0007055359084548752,-0.0005854949501251043,-0.0008727652006546611,-0.0012970721049108444,-0.0008866903367895823,0.0006448167537827497,0.0018527735321480438,0.0007053393552795565,-0.008262189432719322,0.08243203304209355,0.048726244549068085,-0.02769863720685738,0.03670378941110804,0.0017304639127508085,-0.009419200317028538,0.007230895743096068,-0.0016013637766138316,0.008618897423070516,-0.005794751400718681,-0.009897264680190808,-0.011260636736381332,-0.010120548592025333,0.00010930221136799916,-0.0033461282502101693,-0.0010051258737591435,-0.004143932325944309,0.001476398251671769,-7.955474246138476e-05,-0.002327851803156871,0.00109057025650027,-0.0016222457736723625,0.0005709359358293362,0.0011131364926053168,0.00014119847776438826,0.0004050240471611274,-0.00045071123844525114,0.0013160783078278865,-0.001364240015066666,-0.0007783678928546695,4.2931257649896684e-05,-0.0008732283974233244,-0.001377810184381454,-0.0023101614583215862,-0.0013468032488928266,-0.0006723544339298797,-0.00029013770595128944,0.0010905922822310773,0.00022172978515828002,0.0007155972834253003,0.0013520329458381517,0.0011670295717857442,-0.0011211363659833415,-0.0003866678685314868,-0.0009759418877962778,0.0001466897975814099,0.0005432201339862618,-0.0001560208861382421,-0.0005006573738768357,-0.0002709208534036545,0.00016553234507421524,-0.0006487951506109046,-0.0007915234862548296,-0.0006311009954660817,-0.00033537454341230245,-0.00018362419046166542,-0.00021911181060835696,2.362298402242392e-05,2.601957842883476e-05,-5.85413396879682e-05,-2.8402430611642254e-05,-1.574006946322418e-05,-8.24656967062713e-06,-2.039218871123826e-06,-3.5219082254661127e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [2.8904353035325277e-06,-0.00014440350864167666,-0.001055884943043139,-0.0012486158625755542,-0.0006289945621196014,0.0009020993968845011,0.0012998327858447123,-0.0027987356251420723,-0.007509585169512977,-0.005650020171155477,0.006286363948338556,0.1132106115744672,0.01854406736480022,-0.20553584423372984,0.06596301567235896,-0.06910310860736439,0.021603396771131322,0.004457760517805803,-0.016300264550942405,0.003884255428075787,-0.038064386409707746,-0.045259442730526164,0.02152542522431662,-0.017163424249403973,-0.029745364410259545,0.008943186945241991,0.0015444455240879974,-0.010960092625229718,0.001801490417813569,-0.0011452712612470212,-0.0064827461011529395,0.00170732271587859,-0.0012174091338856571,-0.00422090506953188,0.0004543209267614746,-0.0015062539284334763,0.0007717824253250878,0.002275822628127159,0.0010174549799445093,0.0013541815334498274,0.001579561203824156,-0.0014686786441178653,-0.0016464169966476192,0.000770839608949354,-0.003183421041566477,-0.0007319388643735617,0.0018090527041722236,-0.002482081256359945,0.0017493345745290064,-8.099520985972476e-05,-0.0033664951231702532,0.0007240168272176028,-0.0011748530120628427,-0.0038441627136099926,-0.0004299603234541932,-0.001012886104059279,0.0013453127045317714,0.0001341632818833612,-0.0002731578647291294,-0.0014165650012008023,0.000497483648715151,-0.0007371220418612229,-0.0015302711379055205,-0.0002821041397052385,-0.00037582506220218957,0.0004030770349504734,0.0002570056507127458,-0.00034774593948362924,0.000245535241016485,-5.518914995952025e-05,-0.00038070135779633867,0.00012052477189621978,-2.213454829336047e-05,-0.00016227680470469257,4.18556097410815e-05,-1.9848622277816495e-05,-1.202015750307911e-05,9.421955319488835e-06,4.050155561644403e-06,6.473534195818522e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (330,20): np.array([[8.638015114236767e-07,-0.0002484973011229936,-0.0006036486999459439,-0.000675410852080541,-0.0007198946466404428,-0.0007844719580791745,-0.0006929176121863844,-0.0005349613101285645,-0.0007774249866382579,-0.001057436537940816,-0.0008053851333963575,-0.0002107131122565762,5.0219702270684566e-05,-0.0007880754152403897,-0.006083530488096672,0.07874191967271847,0.014831962450183976,-0.044056320093721464,0.05174931244064569,-0.004583980308396834,-0.01567588405470802,0.018831831590173688,0.006693743288013744,0.012871973027450736,0.014036126401751848,0.02063708939872969,-0.004992582898047971,0.004655134789494885,0.008785158389011323,0.0018772844348602625,-0.006142042639130624,0.0027764166037673095,0.0035459761765006315,0.0008095458737048027,-0.0009123889930332104,0.0006815069948382854,-0.0010066590200082622,0.0007178638283942356,1.2073788122170657e-05,-0.002925845700192915,-0.00036615239255553583,0.0011188192885341053,-0.000206202492840843,0.0002480158220261519,-0.0003441258140927777,-0.00021218223503059703,-0.0005885242274701155,-0.0010147327512073797,0.0007514676565607075,0.0019852149573303065,0.0018614335259219549,0.0014754984291843307,-0.0010281224831404205,-0.0027501947540991083,-0.0012038082251057656,-0.0004887835249696126,-0.0030172255981912935,-0.0003580108625001887,0.0008127334911834223,-0.0007100175475410402,-0.0011579012015174265,0.000598658231428628,-0.0003159832393891156,-0.0005713842306685769,6.701706998181598e-05,-0.0007400245384312567,0.00011161219666951992,0.00031684208012992896,-0.00037759876662923615,-0.000281492882630258,-0.0002495755149601609,-0.000202957982232727,-0.00013652291854938389,-2.592749751323486e-05,5.652808193412118e-06,2.5850660139386865e-06,1.339124178208727e-05,1.0679861623257909e-05,-2.700989715982443e-07,-1.0465229674745846e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [1.3545293096160682e-05,-0.0001809592270625211,-0.001073656158560593,-0.0010687044705888187,-0.0002788175571129164,4.38293073709668e-05,-0.0006462638000474349,-0.0023447391648984232,-0.0030002229924471577,-0.0016952330484806687,0.006966504127973661,0.08813285408310777,0.14761483445337165,-0.23642334919990513,0.14117179550783138,-0.02572122588093545,0.02952201079300144,0.017085776876185926,-0.0032082675124990787,0.010861019470692415,0.015724500500588285,-0.01941016129668053,-0.013322708765873461,-0.011484637060504725,-0.006694949970293988,-0.003963098840704063,-0.004195749929973894,0.0023321466177125254,0.010244132558489502,-0.008657968373496068,-0.005211384119291337,0.014199242245885552,-0.003251214968715807,-0.0019400147842288233,0.0029321447856592913,-0.003196170258808839,0.0017019530753510828,-5.895636133036992e-05,-0.003244932898367832,-0.00043538718035858616,-0.0010654136021861205,2.824280076947868e-05,0.0009466189380029319,0.0007546189627751533,-0.00010119886922823011,0.00035418324868334203,0.0009206628473797166,-0.0025399305127106523,0.000696682456920907,-0.002983610741772607,-5.001304120961578e-05,0.004546419326234285,-0.0007471237448864695,-0.005918607005674193,-0.0007614642757468927,-0.001413715192813177,-0.001395042430249567,-0.0016671329951058972,-0.0007716716550354783,-0.0026466249146579962,0.0014813535635569048,0.0013258761887493206,-1.3481791175598267e-05,-0.0014549659659027388,-0.0016546035629672869,-0.00011135788439092261,-0.0006383358227906009,-0.00013593239483901013,0.00019261098125881963,-0.0001637877028130447,8.361447199013208e-05,0.0002028661003285098,-7.459125204355116e-05,-2.7272791148948304e-05,-2.4076972906236048e-05,-7.818960061870407e-05,3.6919268719435424e-07,1.827036117562728e-05,-3.6462022684746558e-06,-1.3493800316237265e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (330,45): np.array([[4.243141839647146e-06,-0.00025193754204261477,-0.0005811264992392665,-0.0006917379740844291,-0.0008353318612268831,-0.0007625708491864553,-0.0005076333253945188,-0.00056637208924042,-0.0011622955658053477,-0.0012613448798361646,-0.00031281240501922897,0.0005517250344682958,0.000440273712664985,0.0010436153978777907,0.03694844946457343,0.09679835688777551,-0.05672458454502926,0.019758632320401558,0.02240497554309146,0.0037275966399872595,0.0043924264239370075,0.004085806725529592,0.015647289773288935,9.207271011610826e-06,-0.0013787009620590757,0.005468486750696351,0.006432991258724218,-0.002748743623110696,-0.007343735190582371,0.004171985177522517,-0.006834018871354868,-0.014454723278349424,-0.006729774570926743,-0.0031731311748127186,0.0004040471055609994,-0.004855265753464369,-0.003996270654734606,-0.0007815706386875931,-0.000818869101435055,-0.0003844394159858849,0.0008059645988937435,0.0009190489695366779,0.0001077394704797348,0.0010438028924828274,0.0006741119120604292,-0.0009204287847762096,2.5376138626123895e-05,1.662221334887761e-05,0.0001065221285532913,-0.00010867285292434512,-0.0007655574409325075,-0.0008922009548212218,-0.0005394923082031371,8.238754991674981e-05,0.0003979379150778425,0.0006667013547871259,-0.0008014844948947236,-0.0013482311183399087,0.0010532766910121697,0.0039301158580522305,0.0004884464932480997,-0.0010352074865832832,-0.001188057260856763,-0.002366534075067541,-0.0012650323930886176,0.0001762382631933637,0.0011451507493455758,-7.40919426367156e-05,-0.00038902098766369043,-0.00027967209118417107,-0.000534257547619184,-0.00035920456229717487,-1.1171447733084049e-05,-0.00011642537325108419,-0.0001321944432092464,-5.6093183628763724e-05,-4.421801373867044e-05,-2.141525011403912e-05,-4.2227641425260746e-06,-8.005958601383561e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-4.728435667703609e-06,0.00023704591999657403,-0.0010224158733164988,-0.0018951188279713767,0.001166102247285257,0.0022570405767439955,-0.0015674267560762882,-0.0032616069881514697,-0.0025560429950197004,-0.006487786211868509,-0.007967006906517941,0.020959553188937878,0.2232436713136584,-0.3227278652104086,0.06485726359561021,0.06827113743294129,-0.03213937338953215,-0.025847903953745595,-0.008323510334431185,0.004983478713781654,0.017470306638084382,0.02027018747442849,0.0017274504095025202,-0.004881561154322053,0.012423359601253499,0.00705596452009608,0.013188309589231187,-0.011939393253132482,0.003419664240803868,0.002440616027853806,0.005490288409776003,0.008520833440091167,-0.010118115975253526,-0.0029774110660859736,0.008969041456532784,0.0021936954538429353,-0.003576813292257113,0.0033523594468045036,-0.0025904081791730763,-0.0006738319802624441,-0.00018393368767213603,-0.002808610961887862,-0.00034582222243243117,0.001679666842260451,-0.0013335466335041997,-0.0024630479243685885,-0.000983750786725421,-0.0004932505674978455,4.112834721752436e-05,0.00026469064832071893,0.000384458941409414,0.0013132073159267126,0.004547975234727218,-0.001653130472102396,-0.0021460941258897633,0.005308771558549632,0.002478997256576301,-0.0010286694425133897,0.0010160323058423374,-0.0015836568793664923,0.0010125244285775728,0.0012719317003089437,-0.003800431361103219,0.0006274193020836518,0.0007742305679712087,-0.002604459955353214,0.00020749826530069284,-0.0002369112937836299,-0.0007826456786098618,0.000391852563773887,-0.00016119349865536053,-0.0006313805591271622,8.624574022122669e-05,-0.00015610462069112508,-6.64604874264638e-05,1.2456646920379339e-05,-3.0170417284474424e-05,1.1226064888277982e-05,5.244797267281566e-06,-1.2446025135469705e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (345,-45): np.array([[1.1465338482223634e-06,-0.000246242881189277,-0.0005093745668194904,-0.0006294004690409778,-0.0007872235953696393,-0.0007184279465817368,-0.0007613950557969288,-0.0006052105315248902,-1.9980804176215533e-05,-0.0005688633169669777,-0.002825909109161029,-0.0034541335586333,0.003072547852324793,0.06029753784751411,0.02993274452540555,-0.03589735696306057,-0.03459341914255784,-0.03958147967962951,0.0239833038187336,-0.007446202033094936,-0.009810106737004921,0.000300803761683433,0.0017924231656346531,-0.002287146101240815,9.174900917266447e-06,0.0019332707846477005,-0.00019618071642745285,0.0056269743854462815,0.002386514260779888,-0.003941416128093904,0.0035449755870081696,-0.00016114345285318752,-0.0027315524314429976,0.0001184231531278908,-0.002989348483989216,0.0020405262502522707,-0.001031835261575989,0.00013650458083774392,-0.000624724535173442,-0.0015423700456140461,0.001199945199003276,0.0018434899153687166,-1.6451190486921588e-05,-0.000738207949329848,-0.001517357870648345,-0.0004413967020711486,-0.00011319971156949763,-0.002129133655278704,-0.0028377996532388174,-0.0014974266104038595,-0.0007260111415122342,-0.0005149325299622236,-0.0004401350516462286,-0.0020121616550676184,-0.00032768490061403746,3.6447108382884236e-05,-7.13504689143207e-05,-0.00041007997925914804,0.00035874643457041384,-2.311818158036936e-05,0.000481453014306835,0.00031934734248926383,-0.0006226965573173564,-0.0004538281427549327,-0.0008981968668040599,-0.0003763377936649654,0.00045430016396449344,9.405939308050414e-05,-0.000370203070715165,-0.00015903594067615738,-0.0001143058671979435,-7.473942267478107e-05,3.12924740677542e-05,-6.417378701638678e-05,-0.00011688464884164162,-7.932074326437517e-05,-4.380262710325149e-05,-1.1156942657209362e-05,-3.8654491864379065e-06,-1.2020932135466729e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-2.1231635723822366e-06,9.132853461605175e-06,-0.000589848065589582,-0.001359117586659509,-0.0001702568346234606,0.00022151919060334858,-0.0013094863785515912,-0.0005541256642687263,0.0002491932492753579,-0.004464438810022848,-0.0058352294494248,0.010550351365273998,0.15182391845553528,-0.0080317955765669,-0.09070785623418502,0.050569790755920156,0.02869101169458325,-0.1160635853394646,0.05080582793854993,0.008355024931900368,-0.005022502594765248,0.018080669076384715,-0.01931529579752748,0.010396370324364846,0.01646807462860599,-0.006482585603776577,-0.0012419684110623298,0.000829998264135459,-0.004668586328001361,0.004089765317509759,0.0044811963219823165,-0.005317385229955753,-0.002031097194265939,-0.0006033665535736432,0.0007325606716717871,0.0016699471194014286,-0.00046230059050814094,-0.0034543802986689532,0.003029314178559348,0.00046768530227147507,-0.0004848735064161348,0.0007191107882608882,0.0002639405691736536,-0.00011013606675990703,-0.0015126324290041778,-0.0022294196931122593,-0.002470600821241017,-0.00017767880187796454,0.0005055646023917612,0.0007296210186173715,-0.0009969214928739372,-0.0002658558760186771,0.00014479895732372923,-0.00038232303135857937,0.00035634254999310455,0.0003145775615866766,-0.0018559651848541903,-0.0002882606292098928,-2.4167779776887776e-05,-0.001303025655713633,0.0010121133766550006,-0.0003160350121869611,-0.0005570004369974822,0.0012350130410110692,-0.00016849277750282762,0.00030506880640067684,0.0006872283851162747,-0.001311364140912682,-0.00038409790676581843,0.00015215067682276435,-0.00035151351439390927,9.96411992651464e-06,1.7632973039268315e-06,-0.00019221794077906314,-5.0972672768148204e-05,-4.791895054112867e-05,-3.876134949707419e-05,3.6552686541479648e-06,3.3925358030396078e-06,6.152640780249027e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (345,-20): np.array([[3.570354080312687e-05,-0.00048509956852672603,-0.0009581289312198943,-7.585684166013822e-05,-0.000248264534256013,-0.0018522472409631195,-0.0022063493677445998,0.00011450097701669687,0.0013864490176793156,-0.0017598983686759539,-0.004177873338364132,-0.0007618944486556408,-0.0005378879651088725,-0.021959725546374778,-0.008133391115158036,-0.03443016614488119,-0.07065741870185574,0.028139632305125284,0.045213215629440424,-0.039285912300668864,-0.00806231096841298,-0.007748994969110202,-0.006405055775107392,0.00018633686267560432,-2.1618777720964417e-05,-0.007605797274845391,-0.007628105705381891,0.0011449075207825097,-0.002872704746920738,-0.0008506646641225077,0.0023002876151018573,-0.0006072495872721837,0.0010003480525629864,-0.00021000429292987222,0.002979184233605834,0.0015755910027784592,-0.003153067754040587,-0.0007159388603174984,0.000617860556900325,-0.0017693547063889088,-0.0013859612950507772,-0.0020465779245083774,-0.003470194131320458,0.0002834506690840689,0.002235276352277891,-0.0016410011667031404,-0.002111760749923223,2.943677281203294e-06,-0.001632923770367802,-0.001898701678729381,-0.0001371155224000706,-0.0001388102245819649,0.0004297018183498218,0.0004208502002083861,0.0012090207835643474,-0.0004394953230759697,-0.00027287307829972497,-0.0010529159562604974,-0.00015812834683383498,0.0004857647646358816,0.0003678919706663806,-0.0005287194608855186,-0.0009560997934765327,-0.00039695405194698755,-0.000623295191018764,-0.0007145752326768093,-0.0004414128164284121,2.6011583346591352e-06,0.00022515801136614288,-0.0001237448605667994,-0.00012338958452671586,-4.032081380708239e-05,-7.050139294175372e-06,-1.974249128328653e-05,-1.7201976902068994e-05,-2.6233130859464857e-05,-1.900426633908222e-05,-1.3054618736522833e-05,-6.533653420170214e-06,-6.136466017826158e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [5.974878866653707e-06,2.6005113976208174e-05,-0.001036033478233565,-0.0008889678551561337,0.0018055097347498793,0.001802585840585126,-0.0019910600344769202,-0.0022659826476382236,0.000938831124823034,-0.0010126037604373352,-0.0033052414993922086,0.015419578849595366,0.21319334777217855,0.04073060455752174,-0.10796462452592569,0.03699681623460585,0.026949087259079685,-0.04237124425153252,-0.0018164847238611942,-0.028250119734480907,0.004002554749909697,0.009004734170328254,-0.0077646492173623904,0.00799021514786766,0.005661497283347201,-0.008170562603910358,-0.0013126910896333569,0.006033530625991701,-0.002125803706657604,-0.0009236422974350936,-0.0008258673873883416,-0.0006892337324103226,0.0047634007117679875,-0.0009556625594250893,-0.0026188476074218985,0.002950765463401804,-0.0017607653463919074,-0.004427814975006872,-0.0007761722754607214,-0.0008593398267854784,-0.0037390892364305814,-0.0030145648567487435,-0.0003039731413292958,0.0026661125109249924,-0.000984093970660003,-0.0018384437562833542,0.0015758067998341626,-0.0011143904096182868,-0.0022554810979478306,0.00028011208942036565,0.00043944477135057905,-0.001111896415694616,-0.0007844419680302103,-0.00025599334055970347,0.000267671007412512,0.00018615861766246977,-0.0009908549274075544,-0.0009902560542552165,-0.0009142165006242361,-0.0008042055945438756,-0.00040381210721196194,-0.00014837099679574788,-0.0002098112759855321,0.00016762107050438728,0.0002644181489040457,0.0005840577221450289,0.0003246205399066102,-0.00035267034809617463,-0.0002667473288112622,-3.315561576352164e-05,2.7304695561251437e-05,1.6071516916862754e-05,-2.293341513329958e-05,3.195066829166961e-05,-5.3537615968733455e-06,-4.277107714758272e-05,-2.0005667779900396e-05,-6.344093480294373e-06,-6.635658105994718e-06,-1.1152918597661355e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (345,0): np.array([[-3.8020500261160793e-06,-4.827019149017971e-05,-0.0002609522230875664,-0.0008277229904606824,-0.0008050521139791138,-0.0003408012107343173,-0.00042422950917314487,-0.0011722827232119534,-0.0019334273472099742,-0.0012162611767417256,0.0014765924793652397,0.0038933326046747805,0.001113468564991234,-0.0189615763929915,0.08516437872407179,0.02918752379996754,-0.043138683132188174,0.0743816852407214,0.008829439282393334,-0.020191154201920386,0.015205535349140415,-0.00845615901482714,-0.003737863083379173,-0.00808070222713289,-0.012409565047164547,-0.023200977387198687,-0.00895393457007966,-0.0016943172696233478,-0.004535412289443096,-0.0031860470750907023,-0.0037401690086820544,0.0005761276188910919,0.0003854644134777906,-0.0024190023844539673,0.0006989262097584853,0.0025240079486375955,0.0011055924674307717,0.001185995014067556,-0.0002855859984886686,0.0026434321100902635,0.001408123971536138,0.0006372390284139605,-0.0015365751737594947,-0.0017444298974352195,0.0017448508492778592,0.00012517936014111886,-0.004404952761698532,-0.0037995681144077486,-0.0004016677169262239,-0.002636034053024137,-0.000742846120147038,0.000759114155046258,0.00045586131161287017,0.0005087565281581231,0.002096455531661288,0.0007561362584185042,-0.0017915225835547833,-0.0008363519973159574,-0.0012230051660230127,-0.000268559609707047,7.302753142108421e-05,-0.00012248663921570686,0.000162295593660165,-3.6247279608538088e-06,-0.00036725806327369214,-0.0011049802299248073,-0.0006969276849344925,-0.0006512138223693276,-0.00021344023781709888,-0.00010058859946677461,-0.00017176779573165504,-2.6517740030792976e-05,5.957276411463762e-05,-7.051957705027161e-05,-5.801832262080262e-05,-2.2141209290096362e-05,-1.0652438651204971e-05,-1.5641697833863527e-06,-3.455130057147195e-06,-6.32371756684762e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-1.4973760953503912e-05,-0.00011828797267699986,-0.0019293953275598453,-0.003355253242421827,-0.0017777806340759643,-9.54285938347738e-05,-0.0002608145024286351,-0.00014351912002814993,-0.0013391457320489004,-0.005853400686115042,-0.003924308599627175,0.021358149679332895,0.2434430380320934,0.11448508509573643,-0.11591774719455318,0.120088778009996,-0.08303666359830053,-0.004163668844292576,0.08005322727405292,-0.015635050271000907,-0.030253959454637717,-0.003136068121131961,-0.00724852715686036,0.025102695429762924,-0.006163385441344541,-0.018861273224250054,0.004268619802242627,-0.001898035215134453,0.003287713649749502,0.0050598007793232445,-0.0068077607960027875,-0.006630069514687169,0.00427846268052665,0.0014370743200714069,-0.0026552614763263676,-0.0009789773706548382,-0.0018997876617086395,0.0001062972221693811,0.005153397277426109,4.306894703417668e-05,-0.0007349831722621266,0.0015386805208015795,-0.0014463346689709208,-0.002145329953835506,0.00015822048476136588,-0.0014812644102541082,-0.00050396455164051,0.0019021317794576352,-0.0003555316167664572,0.000690714864947095,-0.0028607805615230883,-0.0010375126169420161,0.001049594460533651,-0.002644362384390237,-0.0025490046892363877,6.033044791571076e-06,-0.0004518344768240458,0.0007028823037500274,-0.00023595359727240537,-0.00013984399435830832,-0.0001581854871523805,0.00032637989330269127,-0.0001769866226437052,-0.0018301441218887815,-0.0007935401903386804,-0.00022590136915700266,0.0001770531485955393,0.0004954442467087865,-0.00016837597035382102,0.00020743477767749607,0.00016301518307587791,-0.0003403575727723929,-5.932817296144197e-05,-2.189590506363967e-05,-0.0001083706008079169,5.8062527237440764e-06,-1.1690308849470943e-05,-6.998641814048013e-07,6.445184418604958e-06,4.4999932087907096e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (345,20): np.array([[-4.811493406235048e-06,-0.00020164985971369168,-0.0006803250001424085,-0.0008457274836729512,-0.0007033569993151632,-0.0005152646206121475,-0.0005836937963647653,-0.00099731278764088,-0.0010903020969061897,-0.0006905638283075674,-0.0003258106058065675,-0.00012474298369324677,-0.0009502611458074377,-0.010098048090603277,0.0853427252841233,-0.03069852654066077,-0.0683251326935749,0.06517801518597222,0.004194829231944327,-0.02527462846118181,0.021117262616584243,-0.0023985067071148297,0.0017663706244590836,0.001144274326871828,0.006386563770128699,-0.005026392244321663,0.003482760350299195,-0.005742273179895311,-0.0022839267804266434,0.00038387255831312425,-0.0067605923879188404,0.002822708668673754,0.0008406529225547962,-0.0024367251230025615,0.0005328511392702768,-0.0013581212239498314,-0.002873882962534309,-0.0010884109742726185,-8.320103828227207e-05,-0.003352690363407752,0.00035974971519558426,0.0018571514649421735,-0.0007252090454277441,-0.0006121603287493504,-0.0007193780858628971,-0.0008804055510106864,-0.0012859284474079225,-5.439295529362538e-05,0.0022732509647363204,0.00303168986193742,0.0009322550240840713,-0.0009812389814791995,-0.0026056179378283343,0.002234542359302182,0.0013058984581838808,-0.003679558426740502,-0.0005897323426052484,0.0009620664351136078,-0.0004413420757105835,-0.0007978961254316527,0.0004225037883524584,-0.0017951870007569548,-0.0005545435217405082,0.0004520619223900655,-0.001048090577220237,2.2220658946936517e-05,0.0004901647926926318,-0.0005024325361633419,-0.00039772579214527947,-0.0002707236735377466,-0.0002249076881216913,-0.00019864759576019558,-2.7341942559085677e-05,-3.642108374284528e-05,1.3922924612647253e-05,2.8422756402461394e-05,-1.8849805244095868e-06,-3.596920082954709e-06,-4.53745720430988e-06,-1.1385175727856076e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [1.8636292190797723e-05,-0.00025104800209910846,-0.0013795563852134981,-0.001276435730948472,7.791898153143939e-05,-0.0002685141613272829,-0.0017676184445310664,-0.0010674163143507681,0.0011782437158537624,0.001540640089862609,0.002422590935103127,0.011992587917373338,0.1859275735117275,0.10243445840161268,-0.15442138251231485,0.14520780849064663,-0.07815498757570138,-0.015110597609717747,0.05294559930527559,-0.007509460731092707,-0.020170909805780922,0.025052027199327486,-0.011267939176342782,-0.0069949234664659875,0.0007734063788405488,-0.0017338614754273869,-0.005293999389231414,0.005452399870086041,-0.0013395780870209223,-0.002172754912378725,0.0013933714537702447,-0.00015212741371621445,0.0031990854983155715,0.0014738998714342508,-0.002409662050780984,0.0008406997203921511,-0.0015699176261284695,-0.0012939092783171334,0.0019130554547806412,-0.002157440624852549,-0.0027206911291436466,0.0009525055496653986,-0.0001723655272919984,-0.0002530100241923734,0.002150455774065061,0.0003482351668135412,0.0004024863409277743,0.00044831048484277486,-0.0009412278187371133,0.0013325928175943133,-0.002666274191620404,0.0016298473791109143,0.001066820449789823,-0.004697709253033155,-0.0005240425965871258,0.0012325210626036973,-0.0008806018189142366,-0.0025896739108834067,0.000201031140616166,-0.0032152754232170028,-0.0007407362713205859,0.0037341120487685247,0.0004914519880022709,-0.0009025933845534798,-0.0010127173608292656,-0.00042503005849685814,8.806061592974897e-05,-2.6002674874045065e-05,-2.951229485281176e-05,-4.861165107299242e-05,0.00017597814555476085,1.3472511696174443e-05,9.316329709116794e-05,1.2876331985958147e-06,-0.00011229398116179756,-5.849584437299415e-05,-7.854231982055622e-06,8.186345436884373e-06,4.322511018056162e-06,-9.720604823857426e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (345,45): np.array([[-3.818992096943698e-06,-1.646910843790866e-05,-7.665752850595862e-05,-0.00041065855077055785,-0.0007835701803758199,-0.0005052289513058847,0.00026861743603722496,-5.592663213653771e-05,-0.0013187492858253362,-0.0013399568315085385,-0.0003382116320029307,-0.0002965519704492023,0.0017271603276372083,0.029229815052242484,0.1260051052821754,-0.07684539153164008,0.012435760361592796,0.0394199409745798,0.0017709971794707792,-0.010994823594666508,0.013919611945266506,0.01855697368095954,0.00016646231689976232,0.004222011010316827,0.012081639251212374,0.010010387694648471,0.0028616706850292705,-0.0014731082456215894,0.008231562854645161,0.009072545688888604,-0.002619436359498622,-0.011087288990487186,0.00046563168949496865,0.000513450862160606,-0.002179743692240052,-0.004304302306308196,-0.0011936401401724007,0.0003472101857628058,-0.0006404063113837194,-0.0014319817743459016,-0.0022579520928030163,8.476883689881923e-05,-0.0012176991177851022,-0.0007612509331814593,-0.0009857253317565415,-0.0011778814568111705,0.00019045431821342973,-0.00018378869785339592,-0.00093271757786962,4.209042996769424e-05,-0.0006735014584463137,-0.0008228542145731996,-0.0008715269087103292,5.4311169117755284e-05,0.0016871418502678525,0.0010523846945204715,-0.0009301683710742725,8.831893703570652e-05,0.0021227722817653712,0.0028576602629538793,-0.0012860650645289177,-0.000376061433426864,-0.0029895984356241747,-0.001692706782863363,-0.0008456056745738687,0.0013490885946571907,0.0003256571910618604,-0.00017291316274265054,-0.0003075813541307928,-0.00048335494066014696,-0.0005065719687404783,-0.00018562084345144836,-2.554991690448554e-05,-0.00013583592061537303,-9.145097978932553e-05,-4.654288568972868e-05,-5.860823612799764e-05,-2.135802655270713e-05,-3.953053118701703e-06,-7.854480361893128e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [4.988459326516042e-05,-0.0004857089105400662,-0.0008872898712839204,0.0008610217504051702,0.00018218178312868043,-0.002393267343906835,-0.002294965871342043,0.00011958770190994779,0.0016816706549359285,0.0029463353685218216,0.005145797997042376,0.005479213956756465,0.11702157709998051,0.096136450957019,-0.0765763251351403,0.08043599370421024,-0.027669430580173154,0.010108327096707292,0.053776914164077955,0.00587437356118355,-0.025604663245615666,0.0007798161614870558,0.008712652957324946,0.0035663187091561424,0.004665003088380988,-0.02333926182680286,-0.021165589991223966,-0.00923792184990717,-0.0030604933879916836,0.00011342488014460149,-0.006839467758862956,-0.017071211020439063,-0.0006697255718867693,0.0028377096993311484,-0.0033494221208681773,-0.004056993100400782,0.001243506912668469,-2.2924931543541543e-05,0.0030448532701612798,-0.0033795816696609478,-0.0016387181136015244,0.003527733294513772,-0.0005306303878597953,-0.0021668312532054346,0.0014325166118266009,0.00021122400620127038,-0.0011909454014178652,-0.0014003535000826181,-0.000832772545360033,0.0008749919370093524,0.0011723578322499159,-0.0014277767153671006,-0.0005633537580237669,0.0013572196977632849,-0.0011384035043232276,-0.0027108795377585562,0.0021877183967477913,0.0032572295006697556,-0.004083721192354368,-0.0010357584564583636,0.0020497004947009957,-0.0003338930199072009,-0.004137724763519965,0.0003430546585126621,0.0018152196754063078,-0.0024907894534693044,0.0007839813157897305,0.0002579070291404094,-0.0004743697517331299,-0.0003559529449368656,0.00018314593462972253,-0.00024884268922705367,-0.00017332019046952907,0.00010409634829712362,-0.00010971181882625994,-9.23189970427833e-06,3.0035520150491063e-06,-2.009404719305316e-05,3.353292482854466e-06,7.958073550958586e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (0,-45): np.array([[-1.1841441868160822e-06,-0.00030153245003087354,-0.0006090028423879066,-0.00035619948805181795,-0.0006443956638349212,-0.0013947652725870593,-0.0008303460488694463,0.0006043445014006801,-3.5896253360442045e-05,-0.0024449323062490915,-0.0037497320833601994,-0.003466830256099854,0.005166391442358187,-0.002735276553556855,-0.025154150286223514,-0.06438123256856507,-0.07553033868700605,0.030840141320811837,-0.004463392005410292,-0.02913513620790688,-0.008636012208592142,-0.008774865405746174,0.005864476060590998,0.0002656634469756933,-0.002118511099575996,0.0026298166320054286,0.003910736880616535,0.0013383424005800164,-0.0011247772419268533,0.007961159802477233,0.0018336321522219184,-0.0017580706138139825,-0.0008286392532907221,-0.003876312394301753,0.004480277215153057,-0.0014695867327126288,-0.0006965456012327883,0.0005794529232121295,-0.0007842293539866673,-0.0002038108977559169,-0.00018578967066884233,0.0015091205235089745,0.0009908140109015165,-0.001494089903722212,-0.0019760001788336717,-0.0010975360288201568,-0.0007164200500602535,-0.000706891300849802,-0.00013420106192691225,4.434108644464258e-05,0.0002019320637049077,-0.0011391330009668924,-0.0007876118735543836,-0.0009284649309140394,-0.00015698599222132727,-0.001228540859262367,-0.00013184262254101246,-0.00022468511091307052,0.00015849941064047997,0.0014725957837785776,0.00016989193200466724,-0.0011853821476172192,-0.0005788726118854429,-0.00040157645875553145,-0.0007434420543444187,0.0006409469937953457,0.00012528406861351003,-0.00046406638324631057,-0.0004251876082012357,-0.00029694824423404185,-3.6795928738407946e-05,4.300636789649722e-05,-8.926144865040224e-05,-0.0001226205460226842,-0.00010475099656561663,-7.630755524230984e-05,-2.0181368583105845e-05,-5.645175224043057e-06,-3.9855360263845795e-06,1.7669712571829978e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [6.8875267190803e-06,-0.00031184318598951417,-0.0008653908005494159,-0.00015025293187828057,-0.00026248915099620125,-0.0013440092506555442,-0.0008315668458767904,-0.00011434293727762693,-0.0012018173099025537,0.0002479217514295681,0.003997259842161026,-0.0005442345657088813,-0.026457849133169748,0.0656707236905863,0.07380615527941489,-0.06773029632252388,-0.0639439382193083,0.07620209956451922,-0.014480769750281975,-0.04387467517056519,0.01693503983205556,-0.020857302811704323,-0.0041001538190113375,0.015910164090485682,-0.019503190004070612,-0.009022947268988795,0.007169641701297726,-0.0015427375196820708,0.0024496258189312474,0.00022894775688274546,-0.006584401091710301,-0.001273496619097146,0.003688620591578732,-0.0019438904960725476,-0.0010225741659699403,-0.00010913620385032649,0.0011568950985442338,0.0014401942311569108,-0.0005164796345124986,-0.0017078914407533746,0.0012761207475822905,0.0015927394884921414,1.6164907814351334e-05,-0.0007146762373797832,-0.0009044659760817633,-0.0007018082520995206,-0.001445297600779271,-0.001206641522230752,-0.0014361418693228103,-0.000598797136523143,-0.0011512909352429965,-0.0008002654975390899,-0.0011129303386992627,-0.00033882234649508237,-0.0005156487831485249,-0.0003761276650675291,0.000615620670215853,-0.000399325804611072,-0.0016980689398519344,-1.7021658307120454e-05,-0.0004283385419181204,-0.0008753798309459886,0.00018206360303324947,-0.0006082092235232395,0.00011747279523468208,0.000945783488433954,-9.476394532752118e-05,-9.939764717845336e-05,-0.00012926599881771353,-0.0005417163354308533,3.160363697998667e-05,0.00012454692701169552,-0.0001271513648214718,-2.2242916784533256e-05,-8.901896482096593e-05,-0.00011109672734545637,-1.9638088675922305e-05,-3.0957276247848477e-06,-2.8970717819291386e-06,2.734010602792529e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (0,-20): np.array([[7.30445651652789e-06,-0.0006706764884942137,-6.809431853248793e-05,0.0005691707334410955,-0.002119301903129751,-0.0038297990529270617,-0.0005635208728948523,0.002791883621491225,-0.00036808245091990297,-0.0047518499589212105,-0.0014246008520691304,-0.0007653796326211357,-0.039941220543564006,-0.004495139479264608,-0.05458681322962209,-0.09380507777834254,0.05953491494800623,0.06323977396554215,-0.056984124884415246,-0.01752806569699717,-0.024443850874007213,0.003468209706851001,-0.0035516534904161193,-0.004694387296907422,0.004044494997404085,-0.009508383095439908,-0.003911664835544289,-0.003889902097893594,0.007620974999841937,0.0008494754405124748,-0.0014471161947801929,0.006521045932888708,-0.001702702801447395,0.0037052414366062665,0.0031981487877442006,-0.0009390972415412828,-0.0016430542693779645,-0.002785154067028954,0.0010377486264735003,-0.00103893552631235,-0.0032552208849796517,-0.00438390168592818,-0.0013881134385028344,0.0031011905197847276,-0.0010109342714345634,-0.002166178455310175,-6.857141802339656e-05,-0.0024921127839654285,-0.0004814501690949695,-0.002204459670451769,0.00017413183208568264,0.0017497594846142203,0.0007021882495188756,0.0003668350514842088,-0.00016504581728097887,4.337141064496036e-06,-0.0010747006147914585,-0.001118076272405857,7.866105848343909e-05,0.00100403497780131,-0.0007417828613189627,-0.0009225586440814753,-2.993043950829447e-05,-0.000705669047056983,-0.0014372932335077078,-0.0008000617740143754,0.00016697825253468186,0.0005627966462938161,-0.000256642888934301,-0.00030834488251868747,-4.35034865661262e-05,6.994294099756096e-05,1.814505164885313e-05,-1.0087251475090314e-05,-1.5906886998753743e-05,-1.3788704487733241e-05,-2.3608783403349212e-05,-1.621154313611638e-05,-3.634292434763848e-06,-5.930554889423087e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [3.63902370788194e-05,-0.00020050940769783014,-0.0010031335749618908,-0.00044202544227821684,-0.0002952706302887973,-0.001171669506174175,-0.0021289880120786836,-0.002695132092370027,0.00013629450137046762,0.005539060082181122,0.004387118739167116,-0.0036147254625102138,-0.014457750687566188,0.1849508922668638,0.11961733238456176,-0.05183991839732043,0.05022939251881758,0.05510920837970745,-0.05526067336752191,0.012343017779779763,0.00014851947277002176,-0.021112480554314566,-0.0064496816540867315,0.0017795030399446311,0.003054954669242444,0.0034331362400064126,-0.005453839884769914,-0.002167426548010694,0.00015601418272103972,-0.002501941339901586,-0.0005460309645063009,0.00047571548922530033,-0.0041157101859256575,0.0003105080931850892,0.0032074196127621787,-0.0006824278244683823,0.0009186695617493564,0.0005430366302600486,-0.003072383539465317,-7.834524625498236e-05,0.0016880290002164924,-0.002420603980862539,-0.00041658682684301136,0.0029677651502963373,0.0018883162238738714,0.0005833441580949378,-9.13196291026145e-06,-0.0021119805230840395,-0.0019306749513012052,-0.0017679904365991528,-0.0006935239872523775,0.0003742479933264661,-0.000216117283920296,-0.0005735023058243316,-0.00038483115297593273,0.00011267096966461582,0.00026392491422074947,-0.0005865876130754083,-0.0007249833922098624,0.00012335132567945785,-0.0012266296733143064,-0.0010399010685338585,-0.00021009689775009996,0.00010399473227211306,0.00017668935542137037,-5.00094671333116e-05,-9.852274340381206e-05,0.00018700167117614099,-0.0001077305915434487,-9.437282122982921e-05,-0.00020112153405256488,-9.438376401995752e-05,8.074869794258291e-05,2.660470001572952e-05,-6.606305552934621e-05,-3.452719086481911e-05,-1.9771231181694445e-05,-1.3436673899802206e-05,-5.051542548521667e-06,-8.974874843647395e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (0,0): np.array([[2.529336343508665e-05,-9.805277452690967e-05,-0.0007558345972562917,-0.0007020280047384657,4.0137753470478364e-05,-0.00022403710786431618,-0.0015354295164832978,-0.002506837651248207,-0.0013216881439631577,0.002457060282022334,0.00588745945373899,0.0019381412203805083,-0.02927046424657357,0.09096676399819445,0.00880494344504046,-0.05581210865520786,0.11607367026017697,0.02255943996309786,-0.044721087565902776,0.04243456786443523,0.011961191843278816,-0.0013163849098388647,0.003862442793573141,0.010396770189600543,0.00045864137966182414,-0.008101531928677944,0.0005205715167804494,0.008672074848544296,0.007396213102380836,-0.006178850066296275,-0.000985905378218059,0.0034345120147729136,-0.004112323178958006,-0.0010475531568725827,0.0008340511307512455,0.0015250505942343787,4.341438041286417e-05,-0.002371243369495627,0.0022173415803588056,0.00023286709563040344,0.00026538824292958316,0.0002042428069232202,-0.0026704032161777944,-0.002187712968378556,0.0020247271426944357,-0.0018178727917649228,-0.005416668061019718,-0.002735302982485152,0.00018012783749620344,6.30103058849182e-05,-0.00087660144215018,-0.00017162790226575046,0.0024006083900570377,0.0023490542130008944,0.00034213468565904193,-0.001334383949126327,-0.0009664956656901881,-0.00045144974433059416,3.9198213031917275e-05,0.0003574700412262531,3.7303378059489434e-05,-0.00017703826258081806,-0.0002597835652305833,0.00012509207509855144,-0.001610197685154813,-0.000988638835835033,-0.0002196058401698261,-0.0001663634621572761,0.00010128791506018173,-0.00015559336101528222,-1.2202166266203524e-05,0.0001242272494579226,-0.0001083036967699264,-0.00012017636290554325,-4.3106139229828896e-05,-7.062682228362188e-06,1.071398983237434e-05,-4.501948451294167e-06,-4.407545953960007e-06,5.446205443044664e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-2.257044702067366e-06,0.0001151420161921633,-0.0008090780612063535,-0.001999212930106974,-0.0016139598092523662,-0.0001705822904063271,-0.00039776552153886224,-0.0025690997276061286,-0.0018513673203916124,0.002524669745922959,0.004446123862322835,0.0038619571375907344,0.0011200230409388946,0.2009798244112898,0.12009056660696314,-0.06591523075200172,0.11150757938092845,-0.019632559597993374,-0.05308748930929494,0.06667714546704856,0.02762922654781794,-0.02748815060741767,-0.003302247452390232,0.0030573223652089784,0.01206290577994334,0.007458237764242532,-0.005111641753994094,-0.0022923895734061265,0.0003780740367922273,0.003960480201509829,-0.00042024272623769485,-0.002767895187577829,-0.004327229995526093,-0.0012968105185836994,0.0031695460662708847,0.000373608873632199,-0.002461850625543569,-0.0013276102951017678,0.00171255069780442,0.002159961453676051,0.00365710142474709,-0.0005873689461193248,-0.0009520082296345757,0.002010565166852565,0.0023532291202084674,-0.0006417632667939853,-0.002019510484584191,-0.0013610211695103559,0.0006210292749572728,4.295802182276945e-05,-0.0019926167315106043,0.000665112189596434,0.0001989973301745165,0.0009133503567552831,-0.001119603418197323,-0.000693849120992773,0.00022953925729580043,-0.0003824334590546006,0.00017480534094419447,-0.00022406853045858958,0.00042472281779130504,-0.00018997322389185015,0.0001583870323654725,-0.00012173671356276115,-0.0013415865425109785,-0.0005504339695835107,-0.00021546180315083413,-0.00022700596023401056,0.00026954797840145585,3.755097688613961e-05,-8.462441596819048e-05,2.62330546175507e-05,-0.0001662726476917455,-6.0617392340354926e-05,-2.10626947619419e-05,-7.14483936836206e-05,-3.1246518004276106e-06,6.9240311705301845e-06,-2.028963988937693e-06,-6.474682372214805e-08,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (0,20): np.array([[6.324530234407994e-06,-0.00029908796665413203,-0.0009147523337651719,-0.0009289156862225729,-0.0005511340736153469,-0.0005173580206829344,-0.0011355827849797507,-0.0014427826221356972,-0.0007793520321375159,0.0002994892407038504,0.001118897939566832,-0.0003270527905054206,-0.015569373664769133,0.11380800624439746,-0.02578624751765939,-0.07391139578804044,0.09859172426659185,-0.004391913941113718,-0.043297317984775985,0.02071450045656484,-0.017295526195878288,-0.006169268895046203,0.0020803576663074074,-0.01327295522640729,-0.01898996162997453,-0.01662407715133038,-0.0058581653785992,0.0013815807992277654,-0.0020853602116409696,-0.004158767994867783,-0.007313711674342457,0.010910917408115969,-0.0007333239834280786,0.002293450142382619,-0.00047890645723025867,-0.0002972077357361705,0.001030651618669639,-0.002085039469751526,0.0009284554978815511,0.00019043889125619962,1.4590315526929685e-05,0.0004958734590406729,-0.002393987384733294,-0.00013666787368648173,-0.0007085964552968629,-0.0012762732436698127,0.0003670066722856837,0.002221985959872562,0.000611726446206136,0.0002265796985320751,0.002305636971869147,-0.0019783644701222,0.0025107583966482573,0.0013649338314759118,-0.003240559959970524,-0.0010285509818873692,0.0012532586180117526,-0.0013060515233392594,-0.00040531314329994843,-1.651807405499907e-05,-0.002631487979931236,0.0006599545639752953,0.0015673008507968288,-0.0007755246482545657,-0.00012966752632119876,0.0003826919453635255,-0.00035200045344636964,-0.0001995510623749056,-0.00012863768769832002,-0.00019528064272776358,-0.0002864936040065261,-2.2214360243674116e-05,8.981675426986204e-06,4.8656774917188844e-05,-2.1030144841517978e-05,-4.210223725331708e-05,-6.002878458138693e-06,-2.110303396283343e-06,-4.138135335973256e-06,-4.4654222189307664e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [1.3234300037200272e-05,-0.0002864004066349885,-0.0010370215011184052,-0.0008358447946868468,-0.00016797947656988356,-0.0002190123899656017,-0.0013648715375293382,-0.0018742977590009307,-0.0001648837437834074,0.002326043013327362,0.0025052135021583497,-0.0003716243771970442,-0.013588941381345606,0.13493715141167215,0.012914736095352262,-0.10603814681386414,0.08025554701591528,-0.018325546100635234,-0.053586175388363126,0.035417251394937836,0.020891404352566305,-0.016339537670719592,0.010613270123741686,-0.015188941201697122,-0.012442929929270702,0.018989439242848072,-0.0039541399567163795,-0.012110463733024584,0.0004107367603153069,-0.004878282648418434,-0.0059671805678939965,0.007132040727294948,-0.003940337042897957,-0.0032574666831854692,0.001252155345862091,-0.0019336814846122163,0.0004656319975328944,0.0006615502055063806,-0.0012522241712549923,0.00044802408734871294,-0.0004265542192889537,-0.0002797179507381244,-0.00014749845617252283,-0.0007370105791230852,-0.0004596218240087199,0.0011665905593216924,0.0007751611799692434,0.0002701100754819818,0.0007733328946643474,0.00019570813255664183,0.0019136247004604365,-0.0030534971430802777,0.0008244212684067824,0.0012942563240053145,-0.0027066875281882754,-0.00181834241585425,0.0006558805594222629,-0.0008404803373823513,-0.003152096560530095,-0.00011459376977132003,-0.002160254826791473,0.00015891996994641742,0.0019094655243280995,-0.001106067798395544,-0.0004707605678512915,0.0005494776217990524,-0.0004372401136180521,-0.00044129367460038585,6.75241272394998e-05,-0.00018532200421071958,-0.0003073484973877909,0.0001728162718409925,-2.8500201701262126e-05,-2.983396152705694e-05,-8.478214957749243e-06,-6.53944790931527e-05,-1.1098310423113185e-05,1.9030904161326727e-06,-4.553136880413066e-06,-1.3989579758134918e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (0,45): np.array([[-2.9175534878556e-06,-0.0003141646539269514,-0.0002728519239003169,7.48904830409785e-05,-0.0008387748193924015,-0.0015485120226616234,-0.00045471174283728236,0.0012594317566526013,0.00020389488780152504,-0.0030842994393804777,-0.0037213891818002392,-0.0029046855537892065,-0.010257138346293429,0.08554757211305415,-0.10194885581789259,-0.03985290214253977,0.04987638424193883,-0.03499640871507626,-0.030360585068940002,-0.0031135616912555016,-0.008343343015954564,-0.008434804039060322,-0.004514944144106279,7.812473653031055e-05,0.006199705424416125,0.0001505171947715386,-0.0048418498919606,0.004344552506807719,0.0010162100767941117,0.005490625309879512,0.005172404433993855,-0.007653254794026463,0.005441939619498074,0.004517456598281184,-0.002257143455890654,-0.0024512321261262447,-0.00010347265207596224,0.00289790371682357,0.0008282793126492705,0.0004651875939345496,-0.0026848202186405593,5.584094138903516e-05,0.00019353995735086374,-0.0021896481683494886,-0.000708078767544627,-0.0011135542241302765,0.0001050222033257375,-0.0016752862160001626,-0.0017025761506732026,0.000435321428698384,-0.00014213535318595256,-0.0003554240227906227,-0.0007880988820249438,0.0009928284096478756,-0.0003866197672752269,-0.0002491649827794256,4.716433596198667e-05,0.0001893530552889642,0.0007676618373725673,-0.0023280866655980173,-0.0010559481696590285,-0.000324043995201995,-0.00184087628426188,-0.004153546921963807,0.0012734641968369792,0.0015703164349656327,-0.0019346320253337721,0.0007135129629441853,-0.0002227197518012539,-0.0005799321781258621,-7.095149980772386e-05,-0.00011022098768499443,-0.00018080327647718712,8.32098033289891e-05,-2.2246068252661854e-05,-7.302414489296496e-05,8.950600231986006e-08,-1.398397363554011e-05,-5.114565196941277e-06,6.217109894498393e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-2.746784526775319e-05,-0.0003459569999076149,1.1522899736078987e-05,-0.0007108344598749709,-0.0019181232807854974,-0.0011255044239262507,0.000508427431871214,0.0005893230377561905,-0.0012850512312121553,-0.0033751372233055085,-0.0032215380418255564,-0.0034573847272145438,-0.012553732347022985,0.1037948700430523,-0.17689317482023215,0.021598099735431444,0.04893992761090989,-0.04458093995822868,-0.03777578707984635,-0.014890220742923697,0.015574131600768586,0.005990858425377699,-0.004942691987103001,-0.011197972714005468,0.00886735520583298,0.01337582053305312,-0.010478071021656838,0.002408030316243794,0.00238340107297432,0.005816009933953295,-0.004372268607576685,-0.0010318706716393015,0.0024636575766129043,0.002323144266208454,0.00032368487475931986,-0.0021635777163248856,0.0026531545597262645,0.0012194528520510996,-0.001595357646947165,0.0002375654457642193,-0.0005791770257251977,-0.001954277002451986,0.0002564657540481548,0.0001346639335209348,-0.0007862378198894868,-0.0009135484016109531,-0.0008293145883337337,-5.690660692612529e-05,-0.00013244188572469388,-2.448893211951775e-05,-0.0008066128914434603,0.00048710038765371953,0.0010140767669210345,0.0019378411277804122,-0.0015998038444652292,-0.001281154740108402,0.0018113736563506824,0.001406241980986767,-0.0008583138754978489,3.275832395967221e-05,0.000830966176238055,-0.004294435399612796,-0.0015281150447391627,-0.0005572642248715816,0.0009342465423869879,-0.00037464361592823016,-0.0009177905935185838,0.0009675235935334754,-0.00040702213377946173,-0.0008274987318343804,1.2634290698372956e-05,2.053528138758142e-05,-0.00022757102864147372,1.4916434384925147e-05,7.285693991451232e-07,-4.81463293820349e-05,-2.182845524670286e-05,-1.170719903616103e-05,-2.772154794115828e-06,2.193385552467453e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (15,-45): np.array([[2.2620720275000054e-06,-0.00023385479399609202,0.0005451161762046634,0.0006999641152119855,-0.0005535447264065861,-0.0017748666155867126,-0.002573692899930569,-0.0005410358502666503,0.0042119973278397435,0.004890995688740817,-0.0006022826573069971,-0.016882878956472586,0.2147267993694999,0.11618723596602712,-0.04792086591040037,0.052629999476616694,0.0982134038561346,-0.09180025040745918,0.03690793709649737,0.0015352798230139012,-0.0027489304156660325,-0.005483604217168287,0.0019657984978766396,0.006109837449512455,-0.011667145067603515,-0.0060476157790559475,-0.003490230710832292,0.017098116570163805,-0.00034745213081247375,-0.006917240989936779,0.005174710668383229,-0.0014620618088906304,0.0003245322631514182,-0.0016300314144626813,0.00023843375429325253,0.005081676933851842,-0.00028502011618108773,0.000130719502185802,0.0017243529227863271,-0.0012087533172214558,-0.0016223786360676296,-0.0016989153549194975,7.346007700497688e-05,-0.002141508634272397,-0.005771376549309611,-0.00409475478802249,-0.001961806669311199,0.0004413302216702765,-0.0011570773563544638,8.289824903457807e-05,0.003382315628419257,0.0006862569441117319,-2.7393739084255538e-05,-0.0015122609321835795,-0.001332105239055387,-0.0008783235845275273,-0.0003460248060408833,-0.0007409454436819002,6.880904562689962e-05,0.0013322816823214641,8.981182884159231e-05,0.0008590810793779525,-0.00041478849715564735,2.8100296268177843e-05,0.002175913971374685,0.0010557304232541903,0.00027863698658914986,-0.0007481190868962605,-0.0008106031294987441,-0.00050846789104711,-0.0005611811635501002,-0.0004914126279349469,-0.0001723182446928682,-9.400628795535389e-05,-4.621229240757362e-05,1.8670083042611517e-05,3.0393791057376805e-05,2.814780893757217e-05,1.1581055592511988e-05,1.1234598943076914e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-5.1815943076405605e-06,-0.00012247678390830158,-0.0005627481414557661,-0.0010834712796111432,-0.0008478152974308568,-0.00022019282692395145,-0.0005427250835685275,-0.0010733333470661116,-0.00019711264116407959,1.846126773499955e-05,-0.0022217141019680465,-0.0022439224550989123,0.005850788837497273,0.08060338291940633,0.054338014768792554,-0.04725471377869175,-0.0067168297975292775,0.00047752488757654867,-0.012402986208564245,-0.0013155112533768873,0.0010326423321092805,-0.0009783452264907377,0.0027935311990542034,-0.005290958667290298,0.0007039442964902268,0.005934611608127124,0.0006446407208389791,0.004989800514093452,0.0026822318208309737,0.0007437179852199038,-0.0007903130597179265,-0.0006696190892409892,-0.002230979322300667,0.0009043405858215371,-0.0012269154628986494,-0.001928590615331703,0.002891451464430356,0.0004945858963567443,-0.0025214790864761224,0.0007202842761769289,0.0016866270987085951,0.00016984304848290533,-0.0009289277925547677,-0.001022800910956911,0.00012980608901869028,0.00042520638454680436,-0.0021377957243532988,-0.002517007347647894,-0.001231181417455633,0.0008684388139866437,-0.0009631797010195096,-0.0018792853060224121,0.0004538484012894021,0.0005708001615573547,-0.0003992792603162557,-0.00030871833854760844,0.00042684303965248837,-9.092049749378841e-05,-0.0003634108099501555,-0.0007534093828171096,-0.0011953082420311765,0.00023787549982386846,-0.0003889501766315323,-0.0008026484907992026,-7.058912797696213e-05,0.00018639579915012054,0.0006003399022708969,0.0004824370284743321,-0.00015401488425032425,-0.0003020433986990321,3.604864119005032e-05,-5.069106712966375e-05,-0.00017454473756138426,-0.00018489804239775373,-0.00013510892169562718,-6.289633447054408e-05,-3.360192570418884e-05,-1.8798860005544482e-05,-1.980336136078198e-06,1.447920710328543e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (15,-20): np.array([[3.6277102323538076e-05,-0.0006446019135750897,-0.0007421681699342556,0.00044670966027186555,0.000490620271752039,-0.0012588033259241193,-0.003614498361652373,-0.0023325991705527246,0.004141661735850925,0.00887859909906054,0.0012427571329220349,-0.04282549414959549,0.15165778422884993,0.026516681777649254,-0.06719177077710758,0.12340712536126819,0.07097421630174149,-0.08868331884800403,-0.009706790229523315,0.004162985810222404,0.025557972551796822,-0.01081717706546322,0.029548854233852895,-0.01350034220727337,-0.000870051886665785,0.0001836083073547525,-0.009545882983188739,0.009668968941072102,-0.010252490737282106,-0.0034658482720368484,-0.000595239189847076,0.0003321803924836527,0.0009157491612973355,-0.0017648696255108448,0.0026524542034819594,-0.0028062015566651007,-0.0011962463627909973,-0.003182616536185999,-0.0006990473467182813,-0.00019036642395739423,-0.0029025158468791194,-0.0029883742353734406,0.00041684793554493956,0.001499605632229672,0.002001029679049675,-0.0026086423129071555,-0.0009344591420354103,-0.0010234197334163299,0.0003351299625256769,-0.0018371269421448475,0.00021074630598361918,0.0007913223835496775,-6.90104928064511e-05,-0.00041001489165621733,-0.00023130372395334687,0.0002594992558306744,-0.0009967820540440686,0.00023980305721691203,-0.0023676906455816116,-0.00032347775371440504,-0.00015820676948598883,-0.001953185588070927,-0.0004623301003520561,-1.8101774293754647e-05,0.00010191084680735929,8.051487285317671e-05,0.0007946493081478433,0.0001869733927722612,-0.0001651188808635944,-0.0003691062544132679,-3.9350323997245355e-05,6.408791386306145e-05,-0.00015715221876317265,-9.024124350697301e-05,7.38942628422685e-05,3.0198387837993384e-05,-2.2695402301414755e-05,-1.5777599088957633e-05,-4.603851467509723e-06,-7.500434508784427e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [7.032387492801015e-06,-0.0002364447195435505,-0.0005755985089359845,-0.0003628790663449231,-0.00016393921770243468,-0.000502411414079721,-0.0008134000458577841,0.00014550889428582592,0.0008398150204162223,-0.0015593704335784939,-0.004456868138875826,-0.00340106785465606,-0.001126200256641328,0.021571736703573652,-0.03966564326534508,-0.06408642209351738,-0.02386466449940354,0.0068043558625951114,-0.004802432281891706,-0.007028449812716388,-0.02090081871593239,0.001358839849497421,-0.006704666451516869,-0.012546204133373885,0.0029604030679060193,-0.0042782195497062395,-0.007305283160293494,0.003925818052635125,0.001083681666192958,-0.003445703335479509,0.0008567915706132584,-0.0008769158228453619,-0.001561263896721861,0.0029077365304657466,-0.0002200026087349691,-0.0018269965902794917,0.00020442767470597195,0.000597921911569982,-0.000555024271926602,-0.001808213830601814,-0.001775052386524012,0.00010512854402970761,-0.002223184213361345,-0.0004611576996743576,0.0008386685357047656,0.00023675383863574784,-0.0015877164896621074,-0.001076066358083711,-0.00022543241856589965,-0.0011332623723989323,-0.0012798107117628125,8.906647099354388e-05,-4.740686726923325e-05,0.0003989306129532929,0.0010719532985584718,0.00010332716266477033,-0.00011151293202165637,-0.00022214853892068133,-0.0002619368981718508,6.095141070946307e-05,-7.596674789987462e-05,-0.0008351057674911039,-0.0006856488127022203,-0.0004972442837564827,-0.0005121551633865749,3.305188992180868e-05,0.00017585437844436282,-0.00021457115255949522,0.00013465924994560767,-1.4185729186229317e-05,-0.0001686218721809703,-5.813936800867104e-05,-8.068116272526178e-05,-7.685888239737274e-05,-9.369521563411817e-06,-5.850408773521294e-06,-4.365830501447415e-06,-4.764370681672235e-06,-5.411447303118407e-06,-9.651973518588445e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (15,0): np.array([[2.553410077806245e-05,-0.0005034574893128626,-0.0005179430052602684,0.0003464488611714148,-0.00017856761362779387,-0.001199771568694334,-0.0012566947817210057,0.0002951344694402508,0.0032883353282770773,0.006409372721334549,0.0024152992180667557,-0.03936033139094756,0.11489115836587942,-0.051556758432042736,-0.06604064986927827,0.20306554671039856,0.02263556849955013,-0.056314975297534764,0.03661910761828571,0.013077927336394315,-0.036305142170046566,-0.030936509561533276,0.013160812379426083,0.008497597864454275,-0.046363759290852735,-0.022093788144612388,-2.4754382381173407e-05,0.003711211537752826,-0.013425287375193594,-0.002939684446698265,0.004467388932858596,-0.013689650540796119,-0.003944341788817486,0.00010373070064589995,0.001855302827806752,0.004622769559937016,-0.0021158006288344864,0.00132546197903016,-0.0018890396608993918,-7.257206385297921e-05,0.00302362876366984,-0.00030622462967740225,-0.0014364809430518967,-9.403684865836476e-06,-5.723905150259035e-05,-0.004502582414415816,-0.0036088349212576427,0.0003090864879359093,0.00040118843847085334,0.0006326766282752028,-0.0008311293575409512,-0.0007381535728053148,-0.0011256730866950415,-0.0010014884741023065,-0.0008730256505732096,0.0014954066248835413,-0.0014363226214776984,-0.0007328529071216803,-0.0003607236347731496,-0.0011721091735089205,0.0017381056142345564,-5.979016711855636e-07,-0.0007232769027749623,-0.0013314269595125488,-0.0006479802336465342,6.106826621042369e-05,-0.00025748924263697146,-0.0004603380097889176,7.408964217883425e-05,0.00015268843909852222,3.229701118252981e-05,9.96041515897631e-05,-5.426853035391675e-05,-0.0001028040385944667,-5.7013866267681236e-05,-3.467258795822851e-05,-1.354700302298125e-05,-2.326707501388108e-06,2.217758438768106e-06,3.2237577955925085e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [7.684621390100677e-06,-0.0002507999121389854,-0.0008442752825206664,-0.0007599712213983739,-0.00024271704266402201,-0.00056560206116038,-0.0012959879545070927,-0.0006558869941164129,7.662307230816973e-05,-0.0012426533479950326,-0.0021144102007524756,-0.00113917426063748,-0.004333597064845726,-0.013993238029983176,-0.002834411920523426,-0.07918031151403904,-0.05969166956119095,0.030788258803048956,0.008887424695536501,-0.027184095288438976,0.004800514584216075,0.01372317192392326,-0.01182836656511832,-0.00037664444937449886,-0.0017964919955801312,-0.009030664623457382,0.0009885374300368046,0.006984069333448872,-0.0041101880440440304,-0.0013226215669061405,0.001192267031094796,0.0014927209830287414,0.0020163367650678304,-0.0015364783519853007,-0.002877924654092424,0.002592796757783185,0.0024039701461878067,8.828642559797214e-05,0.0014351898081810571,-0.0015044539964181667,-0.0005183867669497776,0.0011065115906587861,-0.00017937712597987368,0.0002836289638509684,0.0003656542904034336,-0.0013695408728220514,-0.0014370761896363594,-0.0026840037027068488,-0.0021189737432504036,-0.0018460198873001714,5.694915727118543e-05,0.0012138633266981268,0.0008407235337622482,-0.0007901303145630656,0.001643233147554042,-0.0017085085029816224,-0.0006050720819282125,0.0015822942049113242,0.00017631640798020535,-6.368219381072828e-05,0.00026012710819237495,0.0002346046359355831,0.0003501093901577427,-0.000572089995371944,-0.0009439989289166166,-0.0005830371239391365,-0.000302958623958728,-0.0003606964898243875,-9.377466738153026e-05,-0.00029222516253794145,-8.966808955052902e-05,1.089314774399921e-05,-7.676406915783853e-05,-4.6319023219219646e-05,-1.945567424687932e-05,-2.6998260723589644e-05,-1.382691977544512e-05,-5.533752060399961e-06,-2.3311464546724656e-06,9.13209085555129e-08,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (15,20): np.array([[1.416961999140227e-05,-0.0004472261413535029,-0.0009072569774296478,-0.0007474430503129994,-0.0008861929453263236,-0.0014700729149664667,-0.0015710576513187225,-0.0006668392297486396,0.000564124595305557,0.000996427043754447,-0.0007730493194520923,-0.020754007985328116,0.1541882876219638,-0.08130003317343235,-0.08770400967669109,0.14669757054762458,-0.020203814607307944,-0.08202948771483218,0.036402365929626,-0.023747551466105674,-0.02436261070211362,0.02216237953222904,0.004021299092147334,-0.037686703041139136,-0.011890224690743526,0.014715928623170116,0.0011064774084257563,0.016958096382523952,-0.019926254072805478,0.002710448420143074,0.0067411902926198105,-0.001448121451911942,0.0007460227588435035,-0.0038903506184778157,-0.0002266661007674841,6.120677407209269e-05,2.9039446341092756e-05,0.002957670432164148,0.0025802116726274683,-0.005782328735208048,-0.0005304923509866815,0.0026008407522864828,0.0006619349464766827,0.0010716866641063648,0.000655105500510099,-0.0014096390032959153,0.0015791832858896783,0.0013325463149753717,0.0011765844601280369,-0.002310664864617405,-0.0006388764799506324,0.004575765108622232,0.000192463249786466,-0.005163416745434353,-0.0024681591591553696,-0.000989222598808462,0.002173671999693312,-0.0010870630944125371,-0.002868373955130788,-0.0011815281841010927,-0.00030452317630164127,0.0019455007227103356,0.0025310407331787577,-0.0009280267416632172,-0.0018932280651294973,0.0015997397604439526,-0.0004647638715706049,-0.0007775594903671965,-0.00031737250668872755,-0.0002387675344182148,-0.0003189315896857143,0.00023332698986988821,9.343347334732794e-05,0.00010110769545986496,6.80085444368071e-05,-8.515322507688156e-05,-3.924771826295196e-05,5.993762604417444e-06,-1.3104612563820583e-06,-2.096202680255249e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [8.113147232303792e-06,-0.00024356934588414604,-0.00054860229346167,-0.00046678355420404325,-0.00041313041705257644,-0.0007784733493687667,-0.0009419342035738987,-0.0001664467790680014,0.00026614474605653714,-0.0010725619620609282,-0.002160368171183587,-0.002007962205115029,-0.0031346095941789105,0.003263414000925489,0.06497703752643186,-0.08965056707092225,-0.007643380701476626,0.028984930663216286,0.01606465749470554,-0.013737529426443686,-0.004486675586468528,0.023338976987776955,0.016817244561657906,0.0012371622043645444,-0.0057027086296456385,0.008613631663246746,0.001783511052453791,-0.0036007191052594657,-0.001173559707822439,-0.006010444570402209,-0.0003231500525971794,-0.0005225174199668804,-0.0011306205145678613,0.0016679939480865867,-0.001381892445299193,-0.002668321275488683,-0.0004300077637493254,0.001467533983956281,-0.0024712419271369678,-0.0013632979693507412,-0.0008697379254307906,0.00023888102228730788,0.0008278470804841776,-0.0008339932343754149,-0.0009901635036689766,-0.0016457866922729265,-0.0012055222473244694,0.0019126527498805385,0.0015354695472337087,0.0011464440519548408,-0.0004264357201406923,-0.0025578588284692733,-0.0010518797730822704,0.0018114991510259377,-0.0018160027843792802,-0.0028958264680179267,-0.0011888274372451617,-0.0007963009540496499,0.0007337146684433406,-0.0008966534073857614,-0.0012342181631963488,0.0005333883303594088,1.5532657803655133e-05,-0.000304806810518752,-0.0002799693192365114,-2.4645843467272205e-05,0.000333273177566889,-0.00010698431052775682,-0.0005097290123173391,0.00033024951323367064,-0.00022267559344756488,-0.0002637047625569781,-0.00015225744076442202,-9.552863164260459e-05,-9.01547310272558e-06,1.46955294774823e-05,-2.4553114924001254e-07,-3.1194100333278644e-06,9.423065523568247e-08,-7.264543280043666e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (15,45): np.array([[1.0686530617656265e-05,-0.0002944786617505755,-0.0002151226243355,0.0004642316136326983,-0.00042594010618271597,-0.001568735941550009,-0.0012143685189148273,-0.00023820997004965903,0.000692790909948915,0.002138710818628069,0.0001615977131029107,-0.019796192058555255,0.09008155755526578,-0.05614024286183824,-0.11157247373973739,0.050433440458091595,-0.041795813910555715,-0.04350192106154717,0.002049307636305351,-0.024192195307110188,-0.014600005920678413,-0.008862840231914465,-0.012879450631590735,-0.0048241865740344505,-0.00611430742261657,0.01579736722238207,-0.01337769760685293,-0.009480169990549837,0.005417230462521035,0.011012782865888226,-0.010834483703877231,-0.012398988364169037,0.004362764553432121,-0.0008288744310972785,-9.592995697127346e-05,-0.0011572023673575953,0.0009811298852099124,0.001787389424114048,-6.891069109571679e-05,-0.0014976749360311914,-0.0012386130128906514,-0.0027473646597004817,0.0016412505863705482,-0.0029660136381893737,-0.0016107240145901206,-0.0010417413408075192,0.0006504576192363576,-0.0008925527280421738,-0.003498103443618457,0.0012990653863247702,0.002950478368539307,0.00023502528910544396,-0.0005875334375163421,0.00036615297995654876,-0.0003746892338258905,-0.0008330826662548283,-4.451130156185492e-05,0.0035486274179947538,-0.0009542327612544284,-0.0025481271558297574,0.004106352214479248,-0.0014493942998985785,-0.0036652457036958955,0.0014016883423482913,-0.0003743799999121322,-0.0017580721920222757,0.0005619493891650635,0.0006897801724412992,-0.00021800159450255296,-0.0008882449451818166,-0.00030499301519400755,-1.2315295713705577e-05,-0.00013761550498658212,-0.00011814846094550803,-1.516071633710341e-05,-1.080595487239276e-05,-1.2749019434826848e-05,-6.0970050518763784e-06,5.986969989827953e-08,9.802823503417728e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-2.012275858028864e-06,-0.00012644594249093015,-0.0006186026780197461,-0.0009961436068308708,-0.0007624862203105995,-0.00032357538327468634,-0.00043420778477327565,-0.0010007609133568744,-0.0010013345170132371,-0.00010174910294093626,0.0002591014061668774,-4.8533323363091184e-05,0.0019944879803821705,0.05833117392703486,0.1045442941735981,-0.08030485899776142,0.027638988904002154,0.016042020022906684,0.01447457425273073,0.014855464534623341,-0.0011404148905293197,-0.00184911223179724,0.006285824455592901,0.006398130863896759,0.0072392495015842565,0.0022568592237170357,-0.011884684027077768,-0.008095522696492045,0.007028943066261349,0.002704412508594796,-0.00945192002201662,-0.006314642005728714,-0.0004180142800453837,0.004204102725702033,-0.0044752024616499415,-0.006851225138473071,0.0006145645681850376,-5.100877996640351e-05,-0.0009934197615394712,0.0014081412237382748,-0.0014683429678945996,-0.002176900552030477,0.00208140576632842,-0.0005459825934830018,-0.0027354615914321487,0.00019559757965401973,-0.0002033784217731607,-0.0007894903586859305,0.0005358410116694611,-0.00139614884122458,-0.0011566681142775453,0.0006304074822537826,-6.608664952379582e-05,-0.00011612529821637572,0.0011366257182203847,-0.00023902127028928454,-0.0006204914720964019,-0.00041848470827264756,0.0033690935470902304,0.0012233993756887234,-0.0015380831801459919,-0.0014709725767908786,-0.002337249397798584,-0.0011430426425036197,0.0005813810580644281,0.001170233140009675,-0.00010401403876044311,-0.00042085784865496993,-0.00021611896397817333,-0.0004059812991568927,-0.00014528378880964058,-6.495466238747975e-05,-0.00018002849468363208,-6.673095828785779e-05,-3.558151684280922e-05,-3.1777627306221485e-05,-2.1551293932860857e-05,-2.031238143396734e-05,-7.65189917348762e-06,1.1276367258214251e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (30,-45): np.array([[1.881652642912564e-05,-0.0005158372274789386,-0.0007235885680119686,0.001204309884572255,0.0012210981091164872,-0.0005295786084246522,-0.0010522091688094169,-0.002240050069828651,-0.006351442733141387,-0.005702452374316158,0.01602506319314171,0.18255963436360456,0.034329131962996695,-0.054357070203832326,-0.027348783346624023,0.015307050464918164,-0.048094819224312836,0.03604731871194256,0.00805094244700089,-0.005162602054454557,0.010747033954291853,0.007025498934726474,0.010574544002325454,-0.015600069682889784,-0.002843290781690274,0.004849830574994148,-0.0007410251440135264,-0.012635372268786147,-0.0030811071055957054,0.003888165949727006,-0.0035116699334184517,0.0007245932794606846,-0.005835146502004183,0.0006337526762284253,0.004798131120722723,-0.0020746289625773857,7.284581637516457e-05,0.0023460709986449398,0.003233514986448745,-0.0004415158913565112,0.0007492951519496529,0.002425303398795749,0.0011045264920774007,-0.002448682883031851,-0.005527391242293562,-0.0012737745054912707,0.003049644208948642,0.003925950113683258,0.0015927203931294781,0.0010858834747994289,0.003447654941719133,0.0012246428931529246,-0.0024129498372362154,-0.0013153438562857175,-0.00017511182479316558,-0.0032336267766529697,-0.0027158838394772474,-0.0012641351345115993,-0.0009504106310145531,-0.00043530334277675555,-5.861689441695078e-05,-0.0013889096942039994,3.2149105005640103e-06,0.0008715037859192852,0.0003836972552191594,0.0009472164571963088,-0.0004427802611826374,-0.0009067165951030735,-0.00027701338601926093,-0.000650863482408245,-0.0005793838827600716,-0.0003208446324860281,-0.0001797266205188736,-2.0361383781996566e-05,4.393691576404511e-05,6.0551359599667226e-05,7.343762178655007e-05,4.7615211394168925e-05,1.2599999387056888e-05,1.4846845630622805e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [4.754233419721154e-06,-0.0002625654457156036,-0.0006560557543059413,-0.0005357814725173094,-0.00041853438583563886,-0.0007751145646127839,-0.0010910278463960243,-0.0007333443762485447,-0.0004258382523593291,-0.0007036062012773102,-0.0004889124490434668,0.0002998028686176581,-0.0012228449970032185,-0.010373313413712143,0.046095502590490466,0.03112960929114038,-0.037647314986547054,-0.02589238504043781,0.02419690516855514,0.0026894583547975846,-0.016918653323228252,0.0037342886241276993,0.0025706531260151102,-0.001827823499288602,-0.0016074185072576341,-0.0002441415850634126,-0.0034194532010769054,0.001582471465177766,0.0012733181159437763,-0.0020806974116186535,-0.0018402487597089271,-0.0005594780448152139,-0.001722813449481173,-0.0003666359523107463,-0.0009410335485996554,-0.0016889994405571169,0.0008864923279811909,0.0016114999810681447,0.000959583437317311,-0.0011763085605723369,0.0004576422520514889,0.0030117435205654483,0.00048247017600868406,-0.001556366473802778,-0.0006092743935724752,0.0006425620264650624,0.00028289625810415435,-0.0023270441658062558,-0.004326531724924172,-0.002570353296066246,-0.0003879568486558997,-0.001780745079490323,-0.0021431965212904366,5.9128935442381806e-05,-0.0003170180504958022,-0.0012049250939086358,-0.00040996222397156475,-0.0001575595668060112,-0.0007073386698917641,-0.0011876944947729654,0.00018192208549061323,0.0001772172984368303,0.00022394523588351807,-0.00031893966735345436,-0.0005391401487497482,-0.000145522150249252,0.000619283876926707,0.0007452278725362813,0.00021605623203501358,-9.960203157739445e-05,-7.189207519587212e-05,-5.061289081343619e-05,-0.00012534702965516192,-0.00018840158084592983,-0.00014897634045848055,-6.429569491567601e-05,-2.908165009494845e-05,-1.7379673284926364e-05,-5.744394452517977e-06,-2.698990976622613e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (30,-20): np.array([[2.1414452949383294e-05,-0.0007074010937571471,-0.0007278392991626834,0.001201609453727111,-0.00041277902124888145,-0.004241865633619601,-0.0024902152055782155,0.003589126057957472,0.00482662707332244,0.0027975120422219025,-0.0030303280708350367,0.3091951616192689,0.06816362195589866,-0.046610035787223905,0.1873762487125068,0.05366858917855343,-0.06652030868651726,0.00032369618175316227,0.02572246195965786,0.028629132805407558,0.011026317693236753,0.008459279682976233,-0.0009507315455012546,0.0022404139945702327,-0.0021057489711591468,-0.00014150489158556263,0.003978985701706646,-0.011479711024023927,0.00021110975336036152,-0.0027364724066779806,0.00015491956776272792,0.0011937465098946782,-0.0033556235022370582,0.0023828240908888455,0.0006280001079832106,-0.0020309431974042556,-0.0005255109489866266,-0.0008009957783073089,0.0017876271003514142,-0.004860434650946337,-0.0027152284267538207,0.002944444113454132,0.0001607610649856448,-0.0007308765844208415,-0.0028345152535215203,-0.00013306919434973998,0.0008740051618778174,0.0002716265818848949,-6.65824054857573e-05,-0.0008161565947257335,0.0006981742353918802,0.001252232722270292,0.00014014531678453168,-0.00042166083890236915,0.00015062716690854319,0.0012083739387079506,-0.0007152448534216962,-0.0013456359799076348,-0.0016020009671929942,-0.0010190650672980972,-0.0014430562494530624,-0.0007836430207166411,0.0006901008108057276,-0.0007253462162321478,0.0003086014984673222,0.0004226989072577328,0.0006577447634173271,-0.00011398988961677541,-0.0003765258444142122,-1.632052018088874e-05,6.181026865022546e-05,-0.0001246435223603601,-6.886565138861996e-05,5.690057267248601e-05,4.941705566221525e-05,-1.1055138769912489e-05,-2.1102219239053445e-05,-4.554386727596729e-06,5.262359661118501e-07,-1.2369891986407457e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [3.3512897780623403e-06,-0.00019425128623757368,-0.0004928530417555978,-0.0004476621004155857,-0.0004916249634211176,-0.0005613694280961135,-0.000540473995269079,-0.0005654775815248141,-0.0002753705208474972,3.880014213483771e-05,-0.0010189726173474153,-0.002868570900666405,-0.002161380339379599,0.003862282330163069,0.05856449600553228,0.013329297418071427,-0.01994011268536333,0.008673172762597782,0.012271938157923577,-0.007264013207797816,-0.0012613516716892864,-0.00858005865800233,0.005158265034236498,-0.0018061883412813897,-0.00648473945833391,0.0016532897181092672,0.0027712486856058395,-0.0023024671535614052,0.0016009835903839206,0.001736125357975482,-0.001556751665510255,0.0020563216164468036,0.0005227113070273305,0.0005397434600251714,0.001768417934960678,-0.001307044930262973,-0.0012282456671514892,0.0011515385110189564,-0.0004272848866221997,-0.0005944446510490701,0.0005200931711085897,-0.0024435485846242,-0.0010575669876449476,-0.0009140552478246247,-0.0007279225964819035,0.0008638036976906413,-0.00011670817432526671,-0.0019866908162656355,-0.0015516946539161655,-8.83174714465692e-05,0.0002832371635206491,-0.0006828990518657284,-0.0003743961076366186,-0.00036727428941820555,-4.8134687463057763e-05,0.0006226720147196463,5.167962729899777e-05,-0.0005477388996050942,0.00016412972208072963,0.0006147772742121951,-9.354631420476782e-05,-0.0005647170730150729,-0.0007668855820879368,-0.0008028776572757236,-0.0007054972399119343,-0.0003810726190438352,0.0002226740756287291,1.3844829606485414e-06,-0.00019606857060923413,8.6612262711249e-05,-7.377392820055037e-05,-0.0001141084824101671,-3.3837159908523524e-05,-6.461621043037116e-05,-5.0376078916983876e-05,-1.4568313516729407e-05,-1.2557471813431518e-05,-4.9938855003678995e-06,-3.346331100092808e-06,-1.2045184421351736e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (30,0): np.array([[-6.677589630394268e-06,-0.00031683457554598424,-0.00024029367001537528,-0.00011744876135644576,-0.0012174819070247236,-0.0023894234620299476,-0.0007721617408740873,0.003021722523196951,0.005569164797508668,0.00404023260765129,-0.02069742951675784,0.24338605689833603,-0.014128594383698224,-0.08194796320064204,0.2512954806602271,-0.02595372261348248,-0.06571020316755552,0.062131555987144184,-0.019924983621484104,-0.010756888948781868,-0.0030070342909634252,0.026756734081678012,-0.0038714945947568137,-0.013716625910399974,0.0006226528448479872,0.015492720154506163,0.004281109948660142,0.0032762422996794405,0.010208841504618565,0.006139927245440551,-0.008940970343513776,0.0033127587222325314,0.004713010340446587,0.0011008341056856458,0.0034915344927907484,-0.0010952606209776148,0.001114158331425634,-0.0042572363098953075,0.0006813086068447362,0.002452655074775652,-0.0013731644885571985,-0.0003994992818547518,-0.0024020937466428865,0.00013322801882569547,-0.002189104371047833,-0.005068324756041737,-0.0014429151968668465,-0.0004884191135571147,0.002867250261950898,0.000704312890132525,-0.00206660376921758,-0.0013900928474303931,-0.0003440378845210536,-0.0014481628471835926,-6.581618902313369e-05,0.0015248554228950648,8.710162326863827e-05,-0.0014129626540252934,0.000814633427357636,0.0012125443591929931,-0.0017194798336984,-0.0011933066686732005,-0.0006237521024341487,-0.0007210800310496863,0.00032375050217436663,0.00010440515051528467,-0.00047112797204028047,-0.00010555197836880338,-1.2890296454122935e-05,-8.194697599278737e-05,0.00022798913113585793,-2.2617754418427418e-05,-6.880642952048182e-05,-1.4775760781510721e-05,-4.831679929998037e-05,-2.3325669374452115e-05,-6.088165001732994e-06,3.3023947951158973e-07,-8.854420891421738e-08,-3.832726132227581e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-1.3310869973583525e-06,-0.00019491359886617592,-0.0005990426193307165,-0.0007570080036095065,-0.0006387175996712801,-0.00034007857139774634,-0.0005913037022589762,-0.0009810128662835111,-0.0004629934579135095,6.158418478904076e-06,-0.0010420179426210907,-0.0018394653164932873,-0.0011506262909501998,-0.0023177451189299873,-0.0023477897386515083,0.004013528019349659,-0.04007034843968761,-0.03707599021244689,0.02084757036312867,0.011561022662332153,-0.02247047203266613,0.002545175912796604,0.013791570960009058,-0.008323728096723602,-0.005612274557047548,0.008619709961638616,-0.007424143921329559,-0.0008075794659886664,0.0030661142732063724,-0.004249405994849333,0.001930601490145086,0.0030684199760822686,-0.000627944196388317,-0.00014197095669081185,-0.0010798321331394144,-0.0018956378826534143,0.0015638656916656523,-0.00021117882240680847,0.0015747157735786054,0.0008088748866243774,-0.0025567950435048587,0.00011960857715921012,0.0018349886368003764,-0.00031219651551186485,-0.002036898074388724,0.0006713466546864026,-0.0012242814221361967,-0.0009446664552678605,-0.001157486975069695,-0.0017256429583237807,-0.0007666451003472734,-0.0004306602176938827,0.0005055728274593808,-0.00016907196888520676,-0.00018081091320651808,0.0011385969699977724,-0.0011913630544175629,-0.001384757730808261,0.0012827766638264767,-1.4311150427518282e-05,0.00032478211194725715,-1.9035562718563308e-05,-0.0001710917163686703,-0.0001985855330865742,-0.0005246337742572601,-0.0005953129510783535,-0.00036258411171174147,-0.0004268020912473783,-0.00027041218602903163,-0.00018122978773028075,-0.0002789062308235467,-0.00016055806338186645,-1.7512868679622508e-05,-6.899917315995974e-05,-3.14080258618628e-05,-7.815923886799859e-06,-1.2940334647387333e-05,-2.853603182647578e-06,-1.353960952659532e-06,-6.424093854088253e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (30,20): np.array([[-1.8313219509321845e-06,-0.0003749958401766334,-0.0006586934247865983,-0.0005291729578454508,-0.0012767788738577098,-0.001663670264733548,-0.0006746055168911206,0.0005405276598607155,0.0006779450640671831,9.240602240528446e-05,-0.01353445503821394,0.20875915306933546,-0.05749198066871023,-0.1269836899225076,0.13983024270802488,-0.06707760120516872,-0.09356080743199653,0.02306459054444959,-0.028818154218644088,-0.02232515193113228,0.036463565317156624,-0.0027318875042811725,-0.027922659475627692,0.0046898397570686425,0.04339657588925644,-0.0020528634280438593,-0.003604858365605741,0.022897170885260543,0.0034046745029724756,-0.010462388966137949,0.010992643626142362,0.0027864342030010933,0.0007009663013890914,0.0002466608174858405,-0.003474962643884718,-0.0006348759156042241,0.0006553413037840946,-0.00025146085495167516,-0.00703858188247398,-0.0006980596723302456,-0.0016260761647007085,0.0018693070583767886,0.0006331674781387752,-0.002318404253584172,-0.0002380265531116666,0.0007114296237485534,0.00013730134144323476,0.0009501886956584149,-0.003060280284596898,-0.001141818044376991,0.0011924677443842762,0.00302081724595257,-0.0015031451649597611,-0.005614997667863008,-0.002218307975511883,-0.0011435931915778298,-0.0008530719731390236,-0.0017758101823842022,-0.0012716722767575908,-0.0010177984117909183,-0.0008628302541027979,0.0008709585421252605,-0.00043899412038377994,-0.0002131325362579098,0.0011795604709057459,-0.0017721338433616149,-0.0012064605820023705,1.3891937412092928e-05,-0.00022592525715813008,-0.0003205979371299787,0.0001823674219233587,0.0001866423830843028,0.000127952839686175,0.0001065923343408567,-2.302452227851418e-05,-8.0832946031856e-05,-3.7566142479477136e-05,4.733340084551034e-06,-3.1043534556351486e-06,-2.0343112344377767e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [3.821329700818332e-06,-0.00016836963685537345,-0.0005376965014466708,-0.0005228772866101535,-0.0004655740557420578,-0.0005137328004571542,-0.000708503613763612,-0.0007177728162976699,-0.00026342971269321225,-7.02397709281421e-05,-0.0009829701417006147,-0.0017686821074324578,-0.0013132964684846257,-0.0009528984198489485,0.010637062833505396,0.04539856595690643,-0.06341086193724724,0.0028713533921446957,0.012987275786847833,0.010330274413954923,-0.006227818186719278,0.0007930132030652786,0.008338191026395504,0.0028274026058634906,0.005144356872334371,-0.02110562868895636,-0.012319424403247934,0.008273373426655094,-0.006830859624302086,-0.01170023894998035,-0.0005347489548147924,-0.003236126716045688,-0.0014910636288110405,0.0009885176785369505,0.0001703402798599004,-0.000893034392272288,0.0011500356005200913,0.0007285425985234922,-0.00025590501740131564,-0.0010285947016104439,-2.2810830547748892e-05,-0.0006240282832626123,0.0011522230643942577,6.1647172594499325e-06,-0.00030018980282138275,-0.0008181507541039131,-0.0011942818814016439,0.0011693875437865343,0.0008309263914907593,-6.9132152789723e-05,0.002175665387623971,0.0006765803141227067,-0.0019255097115258484,-0.0019422809884264386,-0.0012125887296880352,-0.001934020459670992,-0.00044257598506040175,0.0006400277062560665,-0.0014577367812764097,-0.0007559862547916652,0.0005226219550900985,-0.00017626052408030552,0.00028737184812950567,0.0002606387383470795,-0.0011116053625102105,-0.0003504659669271902,0.00031235508488348154,0.0002718236326132164,-0.0002230739229437559,-0.00034891744240236525,-1.9332886499508632e-05,-0.00017571756898329677,-0.0001722230029095987,-0.0001706420895380538,-7.047473341013588e-05,-2.41679121177071e-05,-1.2477259154202008e-06,4.665991388929243e-06,2.1411460754269873e-06,-4.794152541466201e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (30,45): np.array([[3.1698923469046736e-05,-0.0004438545066570983,-0.0010841442339467298,0.00017620110110498093,0.0011725875669487142,-0.00045721181521704964,-0.0023618123911311045,-2.745842407766342e-05,0.0033339443205987816,0.004349081104289449,0.010614038756325881,0.11392757155174252,0.2542610490096143,-0.1891065530220225,0.11610705670935868,0.058361431298774774,0.017853687578662496,0.008675901571499717,0.01787954203343994,0.03666100610943527,0.001901123870339576,-0.001220903498737242,-0.003333229095410048,0.002520510395414522,0.005059740518831372,-0.02854622707375273,-0.016487569359836373,-0.007785568332920681,-0.01307141318970296,-0.0044508319337092355,-0.012317474240182286,-0.010900522495905845,-0.0057735306948413015,0.0030823038827890547,-0.0008864909299042207,-0.003931551899347003,0.006075038682705886,-0.0013306067996865277,-0.0007599217929103652,0.0013176753370920636,-0.002450006667971672,0.001341448935669233,-0.0006061933358159648,-0.0007963646341364357,2.5637273199129296e-05,-0.0009224186699157313,0.0003103021224844779,-0.002848070007314261,-0.0007383601795585031,0.0027515619206297005,0.0010419851323488318,3.729198209804913e-05,-0.001972371209919157,0.0021898639360057215,-0.0009836710051458783,-0.00033513039521974273,-0.0016750233317955473,0.0023023408384209064,-0.0003034078091992792,-0.0061305150955825475,0.002756263490193198,-0.0009656742326142509,-0.0030775767814284513,-0.0005767461731925803,-0.0006682920813182557,-0.0015387637235991058,0.0006709508125802404,0.00018542548575026222,-0.0007111810708087835,-0.00014185339596376237,-0.00015121933182966869,-0.00039289535711915075,-7.091771325860113e-05,9.777030158972424e-06,-8.79824290345172e-05,-1.4916103239615434e-05,1.849704821715343e-05,2.294789181856835e-06,-2.028824772178478e-06,8.442601072004844e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [1.6793128320954444e-06,-0.00020639384931899106,-0.0006785622243771901,-0.0007101304157318167,-0.00046101678031940037,-0.000451014314690279,-0.000732365538101266,-0.0007801643559428416,-0.000557694869116462,-0.0006107681513399969,-0.0005495441850044287,0.000118816687440266,-0.0005047370052965767,-0.00691721503612544,0.04909519183692457,0.004659844844082034,-0.03963129385369669,0.0161439051730332,-0.00783009305725003,-0.021977219475428716,0.006500227169504513,0.009097336764954455,-0.00713342793895402,-0.005263591957849154,-0.0008764240282688136,0.0016570618098847708,-0.00962934745230928,-0.008885702605834468,-0.007705806912508062,0.0007177779681674868,0.008073175800281331,-0.007932802299118875,-0.004281446850324675,0.005554468419576388,-0.00040361254437618895,-0.0043739438289519894,0.0030162913891261627,-0.0010063431541836843,0.0004194908047803791,0.0024713138683276686,-0.0005821317167670157,-0.0008459498125156568,0.0005152173083338261,-0.001020945436301324,-0.0019015040997635293,-0.0006971456239741442,0.00019816133926563656,-0.0015983919032551393,0.0003552053639433531,0.00030619156230437796,-0.0009184451078030355,-0.0009683803597144093,0.0004237355317150438,0.0005449290587086838,-0.0003700242169052584,-0.00030959149616317056,-0.0011468485757148584,-0.0006281380936081432,0.0012901260670291736,0.0014920434971522544,0.0007190394607869707,-0.0006678840247056334,-0.0029726936187794654,-0.0019763729639527797,0.0002154579932742204,0.001351391236426837,-0.00042166746389861475,0.00046661297318846827,-0.0003617048794131395,-0.0005606226847119308,-0.00011462640103014477,-0.00010363786016951938,-0.00014348487908139712,-6.339548065930387e-05,-7.783206833922218e-05,-1.9953143206912668e-05,-6.5929385729482455e-06,-2.1459268948195744e-05,-8.959915780058943e-06,-5.643806925169841e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (45,-45): np.array([[6.187845555859513e-06,-0.0003962523183555739,-0.002110465253105987,-0.00375917464792547,-0.002519686045110259,-0.0001219532063265607,0.00015263066473867492,0.0010898631469517686,0.0039470674303168415,-0.003965995751088721,-0.057849788003668606,0.05965294413434108,-0.028686056129901015,-0.19164052525270875,-0.0629340876319302,0.043149277883376545,-0.01848205827056399,-0.06385193747182248,-0.02474811448033738,0.005069991193693149,-0.009591744182187464,-0.004682036027843563,-0.010518938288233684,0.008260201459155195,0.006767528219743106,-0.013869925968874881,-0.013996943199528891,0.001920292519128449,0.003223564294002928,-0.003879998251094644,-0.003241361648053858,-0.0009052036550919676,0.0032049608913989596,0.004175029176357713,-0.0008138164265450244,0.0011719909697587292,0.0025646718095941436,0.0008683532293332878,-0.0015522396863310462,-0.00032441177685750333,0.002291947428163345,0.0003082671442219409,-0.004706749067609219,-0.009906393914605242,-0.006508426884766913,0.00019128817098674735,0.0025767770900737967,-0.0011082435275185912,-0.0006120708627934333,0.00032687939036801165,-0.0014903920918028122,-0.0016417321712310843,-7.141375813963607e-05,-0.0011130237677115114,-0.002543693700888606,-0.0023292070748821714,-0.0014448742651519343,0.0006581008528233253,0.00049900021822902,-0.0004929058732001534,0.0012752934109435965,0.0010259636303662622,0.0017833685512984292,0.002789330059596825,0.002138817714920084,-0.00023522233239124364,-0.0009182794174690086,-0.00019267882051747872,-0.0005278616554718074,-0.0009433731206618299,-0.0007838920158813118,-0.0003549979476614328,-0.00012018081744181403,-0.0001258700712122397,-2.609149684672902e-05,0.0001150823335333255,0.00013059738623885485,5.447197297590316e-05,4.386628244630413e-06,-1.464791822752127e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-2.3765528626446333e-06,-0.0002038056079914642,-0.00042815910063643533,-0.0005538402032236102,-0.0008376277178895069,-0.0007430915392790354,-0.0003480992891922897,-0.0005038788715935406,-0.0008672148466965159,-0.0006526980032801863,-0.0005301897216772978,-0.0010303564630674497,-0.0008368487849818717,0.0020893512734109685,0.03337284711029445,0.03170075803535022,0.0052348104311447695,-0.0005018965633361506,-0.015764536949589958,0.007601207026247169,0.003736412624322001,-0.011602311982497683,0.0022897025397556957,0.00666554650895065,-0.002948797161042297,0.0005730637823064561,-0.003716835751228601,-0.0005136717317813912,0.0016317459766481883,-0.0017571903240326905,-0.0001753566050952271,9.771417353741781e-05,-0.0022460281599012938,-0.0007364519451775466,6.0231016521076126e-05,-0.0016674908298346617,0.0002233841320906821,0.001167321858945415,0.0005226354080185077,-0.00029524318566545387,-0.00025832425301693297,0.0007168411387343419,0.0019022491318741708,-0.00019922145133770286,-0.0014318769177598064,-0.0012110228012588127,-0.0006119329937111973,-0.0008870155377091414,-0.0018722440077975975,-0.0023054126593994194,-0.0012709250291031624,-0.00040808186596852375,0.00014606512404494515,-0.0003813531073740311,-0.0003650742551398079,1.8668396185808577e-06,-0.001150430209786825,-0.000992287754333244,-0.00018658221835611061,0.0002859623245575846,-0.00014295335741711993,-0.00024645176712829623,0.0006305060564856067,-0.0001641969344400877,-0.00039222831757373716,-0.00021624434958866817,4.922836723630612e-06,0.0003703119940482366,0.00037023248160442264,-2.0968283756348337e-05,-0.0002067093682922281,-0.00011901492228965897,-8.728967269108464e-05,-8.329275394500303e-05,-0.00011068632000204728,-7.280218671803108e-05,-1.696124037806748e-05,-2.0004639388870413e-06,-2.4079634425374647e-06,-5.499799291030263e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (45,-20): np.array([[1.4468773902247653e-05,-0.0006989406439911553,-0.0002493167501887214,-0.00010920205049235375,-0.0010095694442222257,-0.00029995506316292847,-0.0012763149278973468,-0.004968698639871248,-0.0019593973681780413,0.0217734247211496,0.2698840896120262,0.05374221566712036,-0.04632714740168934,0.11610773677683371,-0.023489712915516176,-0.02850773716216412,0.00021204079251674001,0.010867251759182954,-0.002975382431754591,0.041656631125795436,0.004745842025768965,0.007196589238307338,0.0018252859021581787,0.0010638358312157975,0.022213085461884257,-0.0024605229637156326,-0.0040236556796914204,0.009053202470137318,-0.004140564561183703,0.0012298922238211732,-0.0023589352156041933,0.0016042785287925514,0.00031044174185378425,-0.003919857395276148,-0.0013931413896256634,0.0036772551574686934,0.0012650695161720792,-0.0013268815677929125,-0.002921644888922438,-0.002492460692076098,0.0006187278595488153,-0.000742718858634072,-0.0023644113054941338,-0.0018230006926430212,-0.0014826024184688342,0.0021104463798715563,-0.0002570664003653873,-8.675375387520686e-05,-0.0003482733621192704,8.205662730001808e-05,-0.0006528335551049781,0.0016766114188923513,-0.0014567289539892747,-0.000379264592206165,0.001152370517328241,0.00039826609633213274,-0.00036906076614462105,-0.0004447023787136805,-0.0016789549193002753,-0.0024069115856678058,-0.0006388929880263739,-6.271838002155718e-05,-2.5275844766956885e-05,-0.0005747501970737304,0.0006375327984180542,0.0009365603796794039,7.091942892731888e-05,-0.0002901791173627151,-3.651700591373834e-05,-0.00019214663144710714,-0.00011364267065473552,1.9684093424763646e-05,2.514359242064318e-05,3.394445381242393e-05,3.332491867879253e-06,-2.1073811701169928e-05,-2.257079264479835e-06,3.7515103012092167e-06,-2.5811486331708447e-06,-1.4513978741977683e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [1.4873811524079139e-06,-0.00017372458168489634,-0.0005017459591319007,-0.0005110373859027708,-0.00044501026386211697,-0.0005615488816375176,-0.0006436504901933605,-0.0005133763199655299,-0.0005568573715623286,-0.0005801350448482103,-0.0002481615577862511,-0.0006009396654956067,-0.00181623108838991,-0.001009093903715734,0.004058020616072468,0.05629396957623571,0.02041818897689093,-0.018061780461566146,0.014844510423274337,0.00989297936939193,-0.012048292079567681,0.01200140263216609,-0.0004499485292959782,-0.00570671599444163,0.007724784051280072,-0.0048769216834902655,-0.0012540426310401257,0.0014548505431983547,-0.005811805031112611,-0.0017338394231108405,0.004396712467937432,-0.00310234970738953,-0.0016837545168679115,0.0013270605505610073,-0.0007362710161859224,-0.0001542058786923473,-8.548661821002691e-05,-0.0021042280889797087,-0.00015662163097500664,2.7016472126654466e-05,-0.0018115832679717187,0.0002364269607718566,-0.00044622754914544553,-0.0023988173820309465,-0.0011529356444069732,0.0006980877983373952,-0.0006568532082080193,7.191895460437549e-05,-0.0003303354825042405,-0.0015002472566449094,-0.0003560336907090186,0.0005225726162766458,-0.00013576649362442007,-0.0005616651334435386,0.00019872418452680546,-0.0006974691306779872,0.00020207127438497206,0.0008707358316230879,-9.780310646155828e-05,0.00012200116069409375,0.00024176648630287642,-0.0001978437540631077,-0.0007204165310643008,-0.000781894023454429,-0.0007729147600061706,-0.0003401450300825567,-0.00019641169494558804,4.470283473736472e-05,3.664151793011119e-06,-0.0002272769788849372,-0.00011056850058545035,-4.1229564918209814e-05,-0.0001414606242529005,-4.449800043879954e-05,-8.897369169430416e-06,-3.121088635650048e-05,-1.0427289434603738e-05,-4.36013470990455e-06,-3.785769369682846e-06,-5.537221284955328e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (45,0): np.array([[-1.5413984130657937e-05,-0.00031713584029736914,-0.00038331662232267286,-0.0011906606271749828,-0.0027141134269798015,-0.0018380468694788618,-0.00016558954804334153,-0.0004612858138357055,0.002684285975884036,0.010537489395376614,0.3002246178954746,0.031403097506872125,-0.12242379127493275,0.2688195358395966,-0.060257718835939895,0.0008070626453405073,0.09001000648535408,-0.02135142490321018,-0.008985795255717313,-0.009441042045305832,0.004115553999767453,-0.018058186810456994,-0.022570508395898723,-0.003015431150955318,0.012372349060850935,0.0013132653782037989,-0.027440525027232557,0.016611341052234167,-0.0006842546554309269,-0.011541313985972275,-8.759278934077247e-05,0.0007088251383755273,0.002763105975029062,-0.002259134980564636,0.002256593177909311,-0.0023957276069620816,-0.0009630777592784075,0.0004157763440826981,0.0013353333074517073,0.0011700004080320526,-0.001627276542030602,-0.0011406472206615466,0.0002746743017798327,-0.0005809487781813086,-0.002517672347646774,-0.0008759377544498063,-0.0027586066033509957,0.001035688394285855,0.004781853332921176,-0.002316388521208401,-0.004321023891531223,5.9237205790202576e-05,-0.0017361015567031429,0.0008521627483674704,-0.00012123776708106107,0.0006069851133909543,0.0004988364261771762,-0.0021283265117135013,0.001699142740790186,-0.0009819133039337664,-0.001198530235166277,-0.001517890173035021,-0.000819642371398326,0.0005419105882420542,8.377545983695519e-06,-0.00039501887470996897,-8.581404954826746e-07,-0.00023292723831868614,-0.00022819399586799597,0.00022533720502096588,3.541358945997087e-05,-5.312195874576289e-05,-2.9007877546650688e-06,-4.236061435531993e-05,-3.981790771257182e-06,-1.900470714779542e-05,-3.0756499184469436e-06,6.4941407496270895e-06,-2.288537967360235e-06,-4.846824583609844e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-2.241115132662221e-06,-0.00018436237845546944,-0.00047156892990670633,-0.0005517829091493286,-0.0006569204678797387,-0.0005994402755943383,-0.00039551123935631377,-0.0005529645570185413,-0.0008439878955361943,-0.0005768931041258851,-0.0002802849854259012,-0.0008717229788312877,-0.0012012695332195222,-0.0005460051482318918,-0.001523227480445838,-0.0036338469429931484,0.011597005262590037,-0.009155171455604088,-0.030887403817303295,0.0158060147568753,0.007419490563459192,-0.0024511953500001194,-0.0015051527894977348,0.016148247969233233,-0.0034083403734217607,-0.0077788907963611134,0.0022727822945858753,-0.0021047363339937886,-0.004643145111410697,-0.001404178559725228,-0.0005971556915487247,-0.00293941414913626,0.0012491503065046154,0.00037222077424176156,0.0009293401304216312,-0.002196557284452588,-0.0005857503113146079,0.0005099987046298433,0.0018912034450320409,0.0014517012379647023,-0.00041241494642132147,-0.0013560623405209867,-0.000752901375196588,0.0004948943938991202,0.00022154029428050025,-0.0009703746298446651,-0.0015806052781902257,-0.0010527918378192037,-0.00032808264715413073,-0.0011906848808392102,-0.0013153445088897777,-0.0014888422238365443,-0.002039878480877185,0.00117341922390449,-4.651915277783726e-05,-0.001172664996008301,0.0011861242603920658,-0.00048368893673337557,6.789706956280311e-05,0.0009053805184651019,-4.987685911853264e-05,0.00012987243918375396,-0.00038362690995323763,-0.00024444867785204313,-0.00018508066039840643,-0.000335448447701906,-0.0003479849326636232,-0.0002850378856046174,-0.0003406753610948983,-0.0003082066869138858,-0.00018541344260336774,-0.0002471345976499965,-0.00013926709085018043,-4.113332531757286e-05,-3.915525111233545e-05,-1.4057584438702865e-05,-4.168479593535476e-06,-6.549635722068711e-06,-3.0913285251799674e-06,-2.196724431795324e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (45,20): np.array([[-1.023378224752779e-05,-0.00041518348477619153,-0.0005459339374695722,-0.0008458755459886319,-0.0014103482614746862,-0.0015874879465877006,-0.0009657994934092676,6.74898171979774e-05,0.0024352037632781394,0.005634688669497172,0.18104799231218552,0.09329560778187376,-0.15264726538509799,0.1193651117567828,-0.009426198924525729,-0.0026681506361886363,0.025029656508589848,-0.036459335631690314,-0.019471474878250117,-0.03335117732552193,-0.027177379828336477,-0.052726127891224966,-0.023161236914609692,-0.012182172673540855,0.0014554408283638306,-0.006540283765106794,-0.005272414205692177,0.008140288853630287,0.024242167519838324,-0.0056524633921648,-0.005415755249156604,0.016557523709091235,0.006124009540846317,-0.0003242342987489528,0.0005942700891678901,0.0038610177245956942,0.003503781266672959,-0.0077073430177682575,-0.001130759887327149,0.0016514503009839843,0.0014509709944271305,-0.002502478348756676,-0.0023662189238388342,-0.0008396986416352241,0.0011407017364811491,-0.0013813036424680882,0.001986374943395471,-0.0003552592437176163,-0.0022080280822821185,-0.0006882758995254407,0.0027820234063860435,0.000528287491306406,-0.002902251333095175,-0.005586739449663451,-0.0016858126769388768,-0.0006930978373927775,-0.0038971617340050916,0.0009164305534775752,0.0014695313989049712,-0.0010093006666282204,-0.0016160199286747656,0.00011016910582021522,-3.989071675852262e-05,0.001227466509198153,-0.0017136560397624232,-0.001268167036706514,2.3131075669224205e-05,-0.00023953845692192756,-0.00027038407525680174,0.00011580894922577604,1.4530517148660207e-05,0.0002072860431214309,0.00015068090939460586,0.00013152959417877413,-8.261387144700522e-05,-4.322805357109507e-05,-4.014259889318541e-05,1.346510584065686e-07,-4.600018701186352e-06,-6.355905128238054e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-1.7479818949656235e-06,-0.0001501904655163806,-0.00044321634615855556,-0.0005556716951452557,-0.0005149717598559872,-0.0004840073618762297,-0.0004825618204340245,-0.0006093625509483434,-0.000681389059886052,-0.0003563186634463151,-0.00016026850424582967,-0.0007533149108121163,-0.0013072793383404225,-0.001157594828677976,-0.0012324303895243553,0.004772396301978874,0.03669259648827304,-0.03637308798707741,-0.0034824739591460233,0.01387598722653318,0.006789650506609741,-0.005636401398008767,0.010149648710217668,0.005902351047737085,-0.010101890125496298,0.006404927725938507,-0.007623983051696263,-0.012094131240080689,0.007211322357444527,-0.0037111874735846167,-0.005223769469078725,0.0004409280489115476,-0.0033140201820562997,-0.0034386867215147536,0.0030727197930770747,-0.0008620957874898972,-0.001758844618392343,0.0011219695898442828,-0.00011300054957039488,-0.001416223098500853,-0.0007327488433145769,0.0007902520488407441,-0.0007090828023278362,-0.0004843035201578551,0.0010228621931525988,0.0004788564234397949,-0.0004846971829432626,-0.00029613692786999694,0.0007963183311137178,-0.001014040064295537,-0.0008434263694707878,0.002179308990101625,0.002566467091089246,-0.0020782730551535335,-0.002976820468677749,-0.0016370808061731484,-3.0374024747974927e-05,0.0015348195688105872,-0.0016608050153785734,-0.0010016233292602893,0.0007489782428471282,0.0011910171065757096,-0.000574420193286077,0.0001135371216808571,-0.00020038988060187496,-0.0006628518748042624,-0.0002782613347546606,2.2239418409197496e-06,0.0001507017918097674,-0.0002941098871532665,-0.0003912286103436015,-9.709322706797964e-05,-0.0001637684854900429,-0.00011832223439612622,-7.812672599005966e-05,-3.1649234096875776e-05,-1.937399360512981e-05,8.385419713611399e-07,3.408693483424595e-06,-1.5594808521972297e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (45,45): np.array([[5.158319217513282e-06,-0.00014699044709776474,-0.0019791200938987317,-0.0032915104526012834,-0.001549035151122692,0.0010362907984424288,-1.4846639078489986e-05,-0.005918105646916246,-0.010596377877205042,-0.010607905189072548,-0.009959106259193778,0.19009108686217205,-0.3097380048955826,-0.0018015416325196877,0.04054400199103912,-0.03556124102637972,-0.0409705453851548,-0.009755539481313481,0.02058984662702344,-0.020543080743997327,0.007317788048981234,0.006862299646939869,0.026586256927840415,-0.007887732262271682,-0.011091043036509315,0.008791293404059666,0.005714805861768277,-0.0006518350597826374,-0.006072511152827208,-0.004811671680824563,0.009493787129853241,-0.014011335386193334,-0.005135687547041724,0.0018711773380214697,0.0004752238579141059,0.004231014002618705,-0.0012343042791633312,0.002818880232877103,0.0007485655848273117,-0.0005129831767184868,-0.0026750067306893143,0.00023064236606524778,-0.0017016473646566718,-0.001213850371503111,-0.0011461675197974612,-0.0022536886045474405,0.000720326037151908,-0.0008152990083134297,-0.0011479086333571076,0.004422130963363335,0.0009453570799143954,-0.0021679143133218515,-0.0007701362041592952,0.0019822241907024333,-0.0004902783236551817,-0.0015907535319074706,-0.0004467167627338636,0.0027014793280261545,-0.002265233932539663,-0.004699383874371255,0.0005046893411988528,-0.0007459729268248882,-0.0023058074761668234,-0.002294592003623267,0.0009533163819353038,-0.0009438622696894888,-0.00047853514182924104,-0.0010405918852143263,0.0005623520327311787,0.00012975651200423927,-0.00025360428148089626,-0.000403297460627295,2.4932953913550375e-05,-8.603416836640922e-05,-8.690053684404662e-05,1.4198230763899926e-06,1.6183160099305674e-05,-1.705257445982218e-06,9.009726272495546e-07,6.987979035010375e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [3.824139724727667e-06,-0.0001712456769619409,-0.0005516611290509549,-0.0005282545414513437,-0.00037436185493606154,-0.00047918271739990433,-0.0006971221229275506,-0.0006802241702099994,-0.00047938584717799933,-0.0003318434372782726,-0.0005615807569753151,-0.0010314545943080297,-0.0010244587831511663,-0.000500106980680876,0.0011396750881700602,0.06096655861974408,-0.03935580463915462,-0.002268742809754292,0.021089568412108118,-0.0043774860068323315,-0.008626250631984552,0.004876360597801015,0.008290697913146464,0.0013014403584220488,0.0010333788269397937,0.0040520354425352175,0.005731452965100071,-0.007361517207091556,-0.003277788743838281,-0.00039006817864999804,-0.003589457125165847,-0.010783112143397132,-0.004288080810952188,-0.006974349552810556,-0.0013275870169305757,-0.004700060530160629,-0.006813864142261342,0.00196166480549026,8.453850326309902e-06,-0.0021925226331910644,-0.0002785874898584334,0.0006842631494892998,-0.00018043931190870227,0.0007636086841989793,-0.0003056535445666174,-0.0010657189798370156,0.00026813169693268345,-0.00024336680562681598,-0.0008931210164059491,0.0007882550584269596,0.0002928041772475644,-0.0002538367150418502,0.000751426948713267,5.358670066312498e-05,-0.001298887281423462,-0.0004945378547163033,0.0001144370551561117,-0.0007252750243875646,0.001188361776973082,-0.0002306697557926874,0.0003054247187159551,0.0031294390417114203,-0.001074291345807126,-0.003730525373162614,-0.00022233432924453146,0.0007845303688250768,0.00039637430803958235,0.0002242284418826145,-9.687729866171099e-05,-0.00045814629305773475,-0.00030397727621961763,-0.00010096410269707222,-5.408559964467657e-05,-9.187210076488852e-05,-5.1211860898584254e-05,-1.535748867624297e-05,-1.3886662013611331e-05,-1.0681661081078485e-05,-6.9197760987257655e-06,-1.4771117783541665e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (60,-45): np.array([[-7.574839540506067e-06,-0.00017918482879302557,0.001638109149358684,0.002962022627924854,0.00041289954367181524,-0.0024893731260631673,-0.0036794054100841825,-0.005809656133321633,-0.004330274642615015,0.021095807858339043,0.22997378654294046,0.06781909652132373,-0.015125032390028631,0.02106355483125525,0.14195281786324138,-0.05387758155263086,0.011261925617236322,0.03869183982771566,-0.0041873132250347484,-0.017743036656425955,-0.010426469472326397,0.01650469333899432,-0.00878946355776239,-0.024521506767240266,-0.00678292705996696,0.00675803723379058,0.005449586953560573,-0.009132364282218005,-0.0014221260987591106,0.008328316256229424,-0.0019462608744526566,-0.0051529979352275225,-0.0017260356768105424,0.004437142610238646,-0.00025429477661213627,0.0001290511642497211,0.0027891407821650045,-0.0018145511153188475,-0.001425647761275373,0.00201327657420243,0.002338865798507799,0.001366447902244771,-0.009748484882887003,-0.011908296460633046,-0.0008158731883046244,0.0070122835835171905,0.004475195509205483,-0.00011532840721754997,0.0017378754388858414,0.0013976102444075206,0.0013704555041518295,-0.0029479298378061853,0.0009591915028908993,6.59907972270991e-05,-0.0010875928541277376,-0.0024085367170188134,-0.0008095808615592241,0.0009467322401510263,0.0007747759581590329,-0.0005287295573422389,-0.00021106386170373774,0.000204928492682076,0.0014932493004364564,4.665475506722913e-05,-0.0006256701500478359,-0.0007240352058693169,-0.00038913786391737786,-0.0006263537485504744,-0.0009296986953475265,-0.0004630090021544136,-0.000182118054310204,-0.00021201225562922604,-0.0002866151495996424,-7.236971876094574e-05,0.00013214899176078478,0.00019695117775393582,0.00011365268285819087,3.4027249384940004e-05,-2.7499252471356755e-07,-2.521557517173882e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [4.6917961025833323e-07,-0.00020596062891476072,-0.0005446076661641194,-0.0006177412044824188,-0.000532448925289717,-0.0005202601375474079,-0.0007038090162793177,-0.000663818828927339,-0.0004424273140988292,-0.000601274272650892,-0.0007012818868508326,-0.00029501610699388277,-0.00030821944769947394,-0.0013730648563395978,-0.003987829191223412,0.03269365921217215,0.020690992471595896,-0.007593108138092742,0.0026949871158265474,0.027834697291903722,-0.008495479973006034,0.0033441878746211687,0.002561989601911337,-0.001486089726323543,0.003758745896602346,0.0018272620905318109,-0.0005936121070920926,-0.0028726932233794186,-0.0014225950392689087,-0.0019695046228290266,-0.001941473152184851,-0.0002907492972239191,-0.0015665898504967737,-0.0008352571456310534,1.0956679607220714e-05,0.0008957481501397248,-0.0003510266512108942,-0.0005313130877579448,0.00039820400777938725,0.0004980317540501355,4.244461913221498e-05,0.00043269020141391886,0.0009380662629483951,0.00044628001520075707,-0.00029362003670635743,-0.002013687291141463,-0.0011026816328559867,-0.0016664864449314257,-0.0008830354899591178,-0.0015618967191886582,-0.0013977740819770784,-0.0006100513427648185,-0.0005068085318306423,-0.001422732130629546,-0.0015324575656614642,-0.0013800721116385994,-0.001881232195742838,-2.12657665816733e-05,-7.354094130129572e-05,-0.0007201326731576685,-0.00028577525550122136,0.0001610340944793407,-0.00015649861406825445,-6.53955618681358e-05,5.051889539685486e-05,-7.365247385347507e-05,-6.043206230291351e-06,0.000322328039370166,-1.6617563685516724e-05,1.9546619910550524e-05,-8.751920438313211e-05,-0.00013116235819619478,-8.028853148224118e-05,-7.000970370480191e-05,-6.83698471585974e-05,-4.60742900472741e-05,-2.2774024633381258e-05,-6.08221684485598e-06,-4.5531508639756915e-07,-1.7903009439183418e-08,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (60,-20): np.array([[1.8870853737859163e-05,-0.00028777525168025714,-0.0011547329007143253,0.00020398008939938995,-0.00018681590446903833,-0.002122596608787153,0.0006797121484683542,0.0026608909360818197,-0.01145359340464715,-0.05158138297290824,0.04508401401558183,-0.18852349918346967,-0.21229171121966128,0.007269140180146941,-0.07397683577644053,-0.07423495547803999,-0.02666481895718518,-0.031147342487776154,-0.023467516183068346,-0.019890378252460365,-0.004466910579776547,-0.014205390388180007,0.017164460559541246,0.00872045956158243,-0.016853530499980993,0.01613445936812972,0.005545744199574162,0.0026670019631654194,0.004904282362233001,-0.001352647408058347,0.0053818047870652945,-0.0018693899442140827,-0.004816824126647058,-0.0031946022032979466,0.0016459618541943425,-0.0017274439107284913,-0.003496159832944211,-0.0012449482282585408,-0.001966168638151348,-0.0023345592086793992,-0.0026984616533717466,-0.0013678111287500584,-0.0005203538092323478,-0.0009644891934148238,-0.0006017647458932918,3.981326910520668e-05,0.00045946068604108204,-0.0003238440778196624,-0.0011819906692367497,0.0009531571054532895,-0.0023830351361726642,-0.0007642820476207555,0.0012273928173767068,-0.0001057522794812669,-0.0010243004271070502,0.0014859906720078577,-0.000598522287860006,-0.001398324815046364,-0.0010635955268903277,-0.00203821713546288,-0.0003757425521393857,-8.972378712161697e-05,-0.0010423162242921896,0.0006379616120072974,0.0007621716508456918,0.00018862475058058187,0.00011299608770268046,-5.468037996267842e-06,-0.00022186113778154638,-1.1459480391274921e-05,-0.0001328169311808513,1.5180005695019483e-06,5.695293098926473e-05,-4.916674587040354e-05,-8.830878289086451e-06,1.609452031780125e-05,-1.3262483065722024e-05,-9.833352772316339e-06,-1.7284156036774364e-06,-9.200819984708492e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [6.027015375412182e-07,-0.00018010357183221491,-0.0004907374436078756,-0.0004946341245981713,-0.0004636003968303232,-0.0004838528929702179,-0.0005896429817184421,-0.0005944499021319288,-0.00044164553223645,-0.0005771963549857955,-0.0007310334478847595,-0.0003253174939430895,-0.00016067736045325312,-0.0009233850504283194,-0.0007464453165505337,0.0023267050564731986,0.04971309365411001,0.02667027058633827,-0.004539248151387634,0.012808023309636674,0.016099625983923702,-0.01105692254728855,0.01320346758654143,-0.0033712561455990268,-0.0002461258837336158,0.007206949289258571,-0.005150353627389617,-0.005191988928475016,0.00382177674449697,-0.0031232068480979173,-0.002567484172773077,0.0022332279432235188,-0.0007027359252847447,-0.0011843499088029861,0.0005291151179133296,-0.0012075898769928603,0.0003064941976512842,0.0006614721536079323,-0.001095720744458653,-0.001613877927806066,-0.0005711386237350152,-0.0023093703186762926,-0.0009709873811707374,-0.00078109089002254,-0.0012492969399996504,-0.0005708559253641246,6.868331251142934e-05,-1.956892894578869e-05,0.0004436078631972046,-3.64761988517329e-05,-0.0016689186791620208,-0.0003266862213931812,6.934683157813762e-05,3.495246129562515e-05,0.0003502474084020785,4.6092062594533904e-05,0.00036411340758438205,0.001110791507869142,0.0002448515623735684,2.4213267358271437e-05,8.851045379468106e-05,-0.0006610364098879851,-0.0006109522432456032,-0.00020943800259410443,-0.0005925017945232749,-0.00044303008965645627,-0.0002318018702244967,-0.00013368241938997944,-0.00021013728800991954,-0.00016002505162957068,-0.0002529852607862396,-0.00013528987278147453,-5.7210932741223644e-05,-7.405814113524293e-05,-1.61318093801844e-05,8.824458087314088e-07,-1.3557508032045825e-05,-5.544164211217338e-06,-1.545029590328593e-06,-5.561543574976602e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (60,0): np.array([[1.9311553834301718e-05,-0.0005094217915446146,-0.0008898127381175114,0.0002762233922917052,-4.417689266134303e-05,-0.003138574200149415,-0.005524922560929552,-0.004658485022866418,4.063890155093502e-05,0.07506238812353598,0.09923498452607991,-0.20679094357849084,0.18867305430857292,-0.050736957725742965,0.057969641428068784,0.07127722727638451,-0.020621320670892937,-0.003122967505966431,0.0007026787264034943,0.006801929916849185,-0.010096828103075337,-0.011133516650619998,0.008611952763089063,-0.007869617230974181,-0.007607702487296079,-5.124652662138314e-05,0.002899123075549236,-0.0004893646730916877,-0.00322283227541995,-0.00523303810922484,0.0025314268874756874,0.008555229464781341,0.0008821831402800662,0.000543847538960918,4.70617961343553e-05,-0.0004056027517242352,0.0016036551021350362,0.0008662444020237869,0.0008485257716273198,0.001720702800563266,-0.004268610934516083,-0.002962274635741569,0.0012150439516403265,-0.0012351913648218167,-0.0034945903761034185,-0.0030162811166378977,-0.0015418364618281976,0.0031636430741494893,9.882323081793234e-05,-0.003596356255141319,-0.0009064724279179082,-0.004237487454692594,0.0012159366505579265,0.003342851058772073,-0.001284946565024195,0.00036912456439699347,-0.0005956198645821655,0.0014576310307935637,-0.001448553518263921,-0.0018101033064637708,0.00013913062810891173,-0.00034637620019974786,0.00027429131848649094,-0.0001350537857196509,-0.0006501455758894455,-0.00032247418929641427,0.0001407858959269947,-0.0002119580126357263,-5.434900002385705e-05,7.959802393450604e-05,5.94540532286722e-05,-1.87258316076098e-05,-3.971312145935303e-05,-9.354768190636054e-06,3.7394344232882624e-06,2.3966930886704518e-06,-9.042501512661705e-06,-2.753733965109237e-06,1.6059608612010966e-06,2.4004985656846153e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [8.475800818696741e-07,-0.00017373021672215414,-0.0005021655572022851,-0.0005324888085414331,-0.0004927570659984284,-0.0005460430584798036,-0.0005624679665124286,-0.0004943085083409076,-0.0005514704708074489,-0.0006676588018574361,-0.0006627554480820401,-0.0006545733405937276,-0.0007721092471852945,-0.0004162037007729815,6.34878900140708e-05,-0.0011513559369431534,-0.005623701515635212,0.026107380305070402,0.010725147068252639,-0.026261672748130807,0.019740575382972417,0.0006956617248351462,-0.007292306231815376,0.0015671786407724143,-0.006805786222213699,-0.002805145729521117,-0.0009256882759661023,-0.00446162582272342,-0.0008268723169944859,0.0014376733255345873,-0.0030963801889512107,-0.003099879737857805,0.0003104811670841092,-0.002313891938930063,-0.0010691358052086683,0.0020654306342487044,-0.002094541544728156,-0.0016783217145980966,0.0020953994488939003,0.0007050360255987321,3.0512761023412377e-05,-0.0009914496420067682,-0.0007875669359808807,-7.234841923423657e-05,0.00028164729554923975,0.00038413831856956874,-0.0009370463470696513,-0.0023508857268739972,-0.00043433353020093797,-0.000753879770017848,-0.00036665660998326207,-0.0009830404000996178,-0.0006987954263976512,-0.0022125919966580992,0.0001365688704434547,0.0003323785118309693,-0.0011014523725822083,0.0002641004620094276,-3.966673014937333e-05,0.00023477156341264094,0.0003768396553687253,0.0003612263724072947,-4.365843636166604e-05,0.00013485643222590384,-0.0004300111804749866,-0.0002608914189957758,5.597485201308502e-05,-0.00041525736427733436,-0.0003811084129754375,-0.00027127226658670174,-0.0001862214814847659,-0.00020810208963466744,-0.00015314463268394985,-8.431220995970693e-05,-5.652115849191563e-05,-3.2460591777200963e-05,-1.5963368288011022e-05,-6.843870385056978e-06,-2.1237675569303894e-06,-2.885391097198308e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (60,20): np.array([[2.0363622147255735e-05,-0.00043246968243771696,-0.0003324242974395181,0.0005910086658914946,-0.00041360786584370877,-0.0010358050259506673,-0.0007302723632757844,-0.0030157633103821404,-0.004113822720107745,0.023777989358742625,0.29222033660147845,-0.2857298110568511,0.13320273558243453,0.041221573538242344,0.02787418625095268,0.02643573062645864,0.021544252267671194,0.04051756846322173,-0.04348521116915488,-0.007170847296805804,0.000962945655115609,0.013263577375742424,-0.019823507845678927,0.0037848290131517167,0.006659583835468413,-8.31290368081859e-05,0.014644928691408583,0.0071766689838696236,0.0024496613416649846,0.002181053874353388,-0.007498499001415308,0.012681175761650109,-0.00023724259111698384,-0.0019378840805407362,5.137862483703978e-05,0.003607329089571309,-0.0012198327721236295,-0.003063514052700034,-0.00010115964212261517,0.0009397677780276676,-0.00032931288707726885,-0.0002080034224446746,-0.003752994232158991,0.00266055341658621,-0.0030390070477464043,-0.0009707731920076866,0.0004916600697025426,0.001123376438717236,0.0007030689006992689,0.0007906426814280785,-0.001436932757990307,8.455079842964888e-05,-0.0011975269506540175,-0.0024934364909675786,-0.0037809934082041062,-0.0018950588000814488,0.0009316192404982853,0.0035490086785455515,-0.001774040303500252,0.0012426459318187538,-0.00252770869194698,-0.0007402858850704151,0.002168776024274184,0.0012540883796861644,-0.0032844583815147535,-0.00021096893437688122,-0.0006621765543317803,1.8859768699509238e-05,-8.844526645216803e-06,-4.5886066758271526e-05,0.00032072669640466524,-1.665144330313133e-05,0.00013335855014022505,1.8770897414615023e-05,-4.8829407604263604e-05,-0.0001036859081100353,-2.5108186339140536e-05,9.6830143342691e-07,4.614916224207898e-06,5.6836028963138945e-08,0.0,0.0,0.0,0.0,0.0,0.0,],
    [3.0044091124949353e-07,-0.00018383640940040186,-0.00043547740434857045,-0.000480000970208091,-0.0005077597517467171,-0.0005606020960633107,-0.0005226163824273851,-0.0004539369670058989,-0.0005277826712722442,-0.0006334796275134524,-0.0004601988823061541,-0.00030762694461098933,-0.0005890894843645842,-0.0007799339781087142,-0.0007148797252381457,-0.0011089321609441135,-0.0002627507349679488,0.035135448690604934,-0.011453931663306426,-0.009175336629753577,0.024748971945811323,0.016929718336439612,-0.00848614539139476,0.0028480648248328665,0.004703157599744909,-0.006149537933212339,0.0008332771419342755,-0.0036984739307554864,-0.0037967196639420438,0.004266073055138432,0.00031785851129496125,-0.0007056017509275706,-0.0007031460386086101,-0.0003577054656131508,-0.0005433800159090786,0.0001860926340805549,-0.0012042021672602247,0.0005320386601781824,0.00173923043956335,-0.0005498362458878378,6.498742621696755e-06,-0.00010564977782877023,0.0011105470822889642,8.616850728862157e-05,0.0006895704069577169,0.0002469018556701981,0.0006977126046070066,-0.0009529993046305107,0.0002980306863029965,0.0005469412174878537,-0.0022643827883174726,-0.002120188146151219,0.0019262475144684326,0.0013338439414365807,-0.0024847819750892313,-0.0022270491967862566,-0.0017055005371229507,-0.001091113180741046,-0.00012817095376995818,0.00029210129989564175,-0.0005266890288750432,-0.0005026491027745263,0.000486570803492779,-7.196400390145311e-05,-0.00018249093324002212,-0.00038929799459098063,-0.0005772009789988093,-0.0005899732729591765,-0.0004439424134655724,-0.00020168317008919316,-0.00022072009229258697,-0.0003326581220097682,-0.00012802064376714271,-5.190379788507398e-05,-2.0774691859951313e-05,-3.330628741605745e-05,-1.5511800709428128e-05,-1.3067514818342713e-05,-3.047358303185449e-06,-2.3130970646147997e-08,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (60,45): np.array([[-4.704604154379919e-05,-0.00010415226732032197,0.0015989243745735324,0.001848674987528514,-0.0012255052290977544,-0.0032312849661147304,0.0008026723822949812,0.007468619380003645,0.01072131960230744,0.0135081192286915,0.12528647573713894,0.2228648046532593,-0.0901274853529357,0.052385120348908895,0.0639497748380682,-0.0007757833368636716,0.01208566131131672,-0.007257687948235296,0.0054808570841180645,0.00920115131135745,-0.020149519148249147,-0.0051219654669315785,-0.004882556577065912,-0.0006963926861637288,-0.019720878280170384,-0.007902989106076203,-0.0024411898077316167,-0.0152631505316343,0.004018589652608702,-0.008873155444789239,0.002117283332850921,0.009447936626247003,-0.005463781977836652,-0.0031641772476885903,0.0026344331414569146,0.0013199665641174944,0.002929250428521558,0.00023433875233364203,-0.0012631872793906365,0.0006770731484893059,0.003878284188839164,-0.003160413542911504,-0.001468247827846094,0.0012636504838226777,-0.0019832402450934037,-0.0014735609097464824,-0.003915047631915785,-7.651389136016141e-05,0.0029942314819276887,-0.00030359457799475283,-0.0006062980522944148,0.00031927295169286715,0.0014176259231976397,-0.0013995845540289381,-0.0005157304659824851,-0.00039555215912300527,5.725152975579787e-05,0.0037270214377118087,-0.0003503461019311225,-0.0012995607143180537,0.00068554947105698,-0.0009463137132630059,-0.0031541246229053717,-0.0009468920228024467,0.0012161583149238738,0.00010344690316584044,-0.0015398075279405667,-3.679538209945934e-05,0.000312753902819688,0.00011078716300969272,-0.0004964810211543914,-2.327965486878565e-05,-7.822535904315639e-05,-9.692439061992263e-05,4.178228059414149e-05,6.15390524362143e-06,-2.0513243516861896e-05,-1.0592547901662743e-05,1.421400351550195e-06,1.475717650921599e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [2.7061540259623163e-06,-0.00020552426716600075,-0.00048689560333370206,-0.00046490811673498766,-0.000510762986398696,-0.0006289351728478506,-0.0006483378811033755,-0.000538797599214849,-0.0004831671439755534,-0.0006301681850430946,-0.0006386418506309228,-0.0003131058776674574,-0.00047424146302680464,-0.0009010624293637577,0.0003374462802449274,0.01868122457436725,0.07195677193933929,-0.037337943602290195,-0.0009228005866489887,0.028602983799400395,0.008765058627726075,0.0022321116797673467,0.0018006640783570713,0.002276578213396988,0.007187902726509847,0.009123849762589335,-0.0012369250982619587,0.006003557989504502,-0.00010008211884323962,-0.003947935788568768,0.004645589959240175,0.0025108643622203694,-0.007817515915199372,0.002204194951814469,0.00016967634130395258,-0.002027422091539363,0.0027515216886573246,-6.194135706883868e-05,-0.0016266801621352097,0.0010617533925453583,0.00011095151981550553,-0.0012002435020378498,0.0007928358569227001,-6.516202188208196e-05,-0.0011446687172630382,0.0007419557391544531,-0.0006891533717814508,-0.0006403336107012405,-0.0002925841753267228,0.00026181597612048933,0.0017561757869390495,0.00046971082087184275,-0.0008787063343704726,-0.0016845344300990674,-0.0002702278749667422,-0.001085747906896999,-0.0005572564020066881,-0.0006151367684275159,-0.0004466832316820941,0.0011317372110626566,-0.001878098967542957,0.0012425592002439571,0.0009458890634351929,-0.001269581870192257,-0.002417306011159176,0.0001498125863893395,-5.866963639601847e-05,0.0001603934320648779,3.036110048917153e-05,-0.0005421446113629541,-9.828637105524766e-05,-0.00016972338132045464,-0.0001694778828092256,6.377667529661629e-06,-3.875413927918756e-05,-4.3732690580832184e-05,-1.8472697298368195e-06,-8.530960960250346e-06,-7.022881578374469e-06,-1.1038573292150877e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (75,-45): np.array([[-5.52063984424152e-05,0.00048251338193943587,0.0011035983064136166,-0.001982600585746987,-0.006139848738495495,-0.0035571523059576113,0.004796577400355053,0.007390707131323924,0.0016501993287729147,-0.0030849559464439535,0.2904537414505483,0.0990892733882652,-0.06642032885485318,0.009843946463630274,0.023881019963443485,-0.019993400965533568,-0.07796827478253766,-0.00851377204694161,-0.016837743916588207,-0.03612397658994752,-0.01743840284465123,-0.006043099105723277,0.006584459084707721,-0.014366616007550308,0.010018133276291481,-0.00011461845187428825,0.0064274276723414216,0.0037550995495262474,-0.006288749303252894,0.0010407022940286666,-0.001634125132493344,-0.0011506520971665624,-0.00015573507974288433,0.00038888073161471204,0.0033519004715346933,0.0019655170214921995,0.0010426018834496951,-0.001685809295448217,-0.0019516403527867457,0.0019166007554111482,0.0017646134348713429,-0.0007562698010997196,-0.011611003465008122,-0.010664034843933752,0.0015600478254212662,0.006436939902382027,0.0025761093562572377,-0.0006056191642873733,0.0005892207095687695,-0.00030235225488528506,5.226884531749991e-05,-0.001568090710263574,-0.00035447765539477823,-0.000512609998082492,-8.732522979651436e-05,-0.00027025394271049547,0.0009271472359456364,0.000585205655103033,0.00026822408179957705,0.0009128437907417028,0.00033924215075451224,0.0014094430469124032,0.0016663004417450315,0.0010317406019115598,-0.0007043217009221397,-0.001050885743486693,-0.00036755003624545934,-0.0006670147886508502,-0.0007744919719783221,-0.0005972831874283401,-0.0002956076141214134,-0.00016964122164049163,-0.00015463117080547774,-8.798838252157213e-05,6.80496612897182e-05,0.00016639141742943837,0.00011076352380981499,2.342945303921745e-05,-7.151146544458549e-06,-2.5226698088354923e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-6.303486158189936e-07,-0.00018729359240827247,-0.0005414136694955796,-0.0006173003665414049,-0.0006113714097772012,-0.0006862727768809987,-0.0007007094842170152,-0.0007044710195924379,-0.0006896450327232081,-0.00058633133632015,-0.0005246708109989232,-0.0008265543195899072,-0.0010552016010809506,-0.0006951063185263902,9.054950913720055e-06,0.0074989674722571965,0.010662624360585784,0.0034845257113299086,0.00941346728688672,0.0013332207183563308,0.005913101800425869,-0.005711494804382172,-0.003663619299784306,0.0005691436348129706,-0.0049939065936827655,0.0004679608553765582,0.003976365646220777,-0.0003339527400489837,-0.002851948285674394,-0.0026333426660571386,-0.0028522191271876554,-0.0001245194352525854,-0.0003108179569962092,-0.0017251494680088424,-0.0013010728851638527,0.000633814635248649,-0.0003625052073362339,-0.00039023607187820453,0.0008700328511305036,-0.0009791428698282094,-0.0003872750609361707,0.00048788945368088575,0.0011301312216232468,0.0006543400969227439,-0.0009048644830935983,-0.0018200574792834375,-0.00180459174777554,-0.0010129745308190409,0.0005138906337204462,-0.0010583808437709637,-0.0010724870830984513,-0.001110874983141634,-0.0006876529460518868,-0.0013029931549780878,-0.0014293630583048745,-0.0010779143051265858,-0.0006553360973471451,-0.0008793330256925057,-0.0007958183039012503,-0.0006880355549366595,-0.00029577259635908286,-0.0003816967517276736,-0.0003278060864179826,0.00012386972220038343,9.857207530067426e-05,3.670730877252328e-05,-8.225590374704795e-05,-7.565616380487205e-05,7.596279404061062e-05,4.448082868645567e-05,-5.955440964676926e-05,-4.138117739305013e-05,-3.3715611000835916e-05,-6.713091745009338e-05,-6.238907132902673e-05,-5.049466637615217e-05,-2.7987363608895476e-05,-4.1203933757795445e-06,2.030196004139071e-06,4.214240294509108e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (75,-20): np.array([[2.745101284247544e-05,-0.0006636705936408722,-0.0017322787594159497,0.0006090235095770806,0.002873386474747775,-0.0028344029735413866,-0.011009785830419094,-0.006671412949986758,0.020235702517484223,0.2291118366417173,-0.03458584289463653,-0.0338474371860828,0.20346667818257294,-0.06524565356661756,0.06529863102851706,0.03309557730505168,0.02390450079502097,-0.008821148714962736,-0.015120742282718832,0.0047035073501731535,-0.009121888201465568,-0.0027424929733832985,-0.012916609568456807,0.01619839729486862,-0.006075077025771097,-0.007921698141968978,0.0015694909451820253,-0.0031354357793911772,0.005717828994659995,7.506890859935864e-05,-0.003713759206916504,0.005367714205754607,-0.00453976410602733,0.0033612056516450443,0.0008457921737246149,-0.002142540480074353,0.0022537716676608245,-0.0008019962007985305,-0.004926583659767986,-0.0023240147331578935,-0.0014805439049804512,-0.0014867637415393454,-0.0016673243297316259,0.0017616700885828083,0.0014202389463576427,-0.0009675344565218477,-0.0017430134423746105,-0.00010973682710867728,-0.00027045354724088686,0.00045677371243046315,-0.0003218005086180714,0.0006069262922659876,-0.002195574982576007,0.0010863825327829453,-0.00026826134849426497,-0.0021524559714630304,-8.927571031104219e-05,-0.0013487337508438933,-0.0022661556297323988,-0.0014733480348902818,-0.00014910212082012498,-0.0007367272109885203,7.447882055734282e-05,-0.0002468833477552301,0.0011416816336881446,0.00030268134085449427,7.274445624234446e-06,3.579626511297594e-05,-0.00022442610102000685,-0.00013036819395387567,1.451542691945908e-06,9.359445461044339e-05,2.235384279441451e-05,-1.0023469442001867e-06,-9.338653301243313e-06,-2.4591242082258804e-06,-1.3121751569286444e-05,-1.2845774015255392e-05,-5.3447058252255845e-06,-9.24051313993386e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [9.156092065720415e-07,-0.00018237117610619375,-0.00048237881430962916,-0.00048486205591058483,-0.0004807378932427852,-0.0005185472441580956,-0.0005275606433561154,-0.000531132148593616,-0.0004966462841369457,-0.0005081362835783015,-0.0005758416460234787,-0.00060041937453287,-0.0003607627683174648,-0.0003248088344990746,-0.0007254926428662576,-0.0011095941583750845,-0.0012033704587424417,0.023795381942406768,0.01740492487408712,0.0036575290976561146,0.0006810931320409069,0.010443715019606503,0.0019252542904055305,0.0029067439751067794,-0.0007588004582855393,-0.0057911093059113735,0.0021069118539587396,-0.0009162424881215711,-0.005052182914693831,-0.00039701866844616123,-0.002312752796664065,-0.002294549486933601,0.00013622772488955547,0.00023539223426905434,-0.001340571229803993,-0.0007441459065896266,0.0006332527539181069,-0.0009849876370035934,-0.0011411203282729566,-0.00044443317822629455,-0.0008319904530176382,-0.0021397695554712624,-0.0016245516333415803,-0.0022118233569119678,0.0005225013341436184,0.0012192098156528699,-0.000567645673682585,-0.001984973256588173,-0.000476781284814625,-0.00021955536524816366,-0.000637709136685668,-0.0012996885858763457,-0.0005673781980058008,-0.0002065768261975024,0.0006023196177494028,0.0003848326822706475,-0.00033679515921011745,0.00024597454195565135,0.0007235365192071495,0.00014934019031780308,0.00030689790348901357,-0.0007057953067076534,-0.0003040614914841862,-0.00041278105369688806,-0.0004487497433762379,-0.0003332476830711632,-0.0003668275966384896,-0.00021337107761820897,-0.000282883842130482,-0.000304813460748555,-0.00018298578800601085,-0.0001419548059067584,-0.00010969141481959448,-5.4103426798786946e-05,-3.1628822896987194e-05,-5.218461892810639e-06,5.383791681415097e-06,-3.783089982910467e-06,-4.295942013047707e-06,-8.362711739488912e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (75,0): np.array([[-2.296457614132619e-05,-0.00010829028722531066,-0.0010713116633574203,-0.002382157450793949,-0.0005468666701084119,0.0011491358994364655,-0.0010737592443489886,0.0011190623619878072,0.012840554753038051,0.33918831437812824,-0.14363655716066662,-0.12146878181166205,0.10870586625828935,-0.12751743090084047,-0.056868342207902454,0.0005417646629075934,-0.05305161323608747,-0.022921537826923848,-0.05658969140932158,-0.026158527948540563,-0.007276005413775097,-0.00814395466614903,-0.0037467232754164344,-0.0012489409215804125,0.0009171703045479812,-0.00992341785512378,0.0071927460493723065,0.009158424339379231,-0.005238231311833354,-0.003596997085913511,0.005039252814497396,0.0011322531413142664,0.0017454515570216424,0.0015856955194616285,0.00023309928758436725,-0.001614881554620025,0.0005280212246340103,-0.0016129786334961652,0.0013840859568302042,-0.0002669540965631913,-0.001135679558646043,-0.000935855065065133,-0.001888461986149172,-0.00415379741264166,0.0009449444335416853,-0.0014880379046610063,-0.0012120782653426412,0.0007037176259471277,-0.00033600529892029,-0.0025738366885796208,-0.0005136746271272064,-0.00031014518374359695,0.0021731611095548485,-0.0010340934004448659,0.00048516081781373307,0.0010154312136075685,-0.002984752201652461,0.001552575067831048,-0.001057698056933173,-0.0012509810387561096,-0.0005056152834035682,0.0002085867309314821,0.001017082221160956,3.2954350910046954e-05,-0.0005006522340963747,0.00010653747116558921,0.0001469172558076274,-0.00017100860855788705,0.00028035917625355607,0.00010389369560259048,2.9362156928559863e-05,-2.5743235296608563e-05,-3.202741648546676e-05,4.287658525116705e-07,-2.219653904411217e-05,3.6094977396336073e-07,4.451008428119032e-06,-2.853667984180318e-06,-1.4488163967848848e-06,3.051512283148645e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [3.8320153299059957e-07,-0.00017636982241605242,-0.000458246394203285,-0.0005248867671239871,-0.0005546997113976814,-0.0005091961006818237,-0.0005021613361476721,-0.0005161679610257882,-0.0005515351166835336,-0.0006014120683594548,-0.0005207410808605487,-0.0004645260921764511,-0.0007001406179653117,-0.0005447581661029399,-4.116655062777047e-05,-0.0001378335480259661,-0.0005627540474986125,-1.7425522630802107e-05,0.039252011183362946,0.03670061149335681,-0.0027171897377064953,0.016041356680937195,0.002409993991430854,-0.009898116429913017,0.002161564567022929,-0.0024337341057868926,-0.0023947508863438488,-0.0013543352977564691,-0.00048466156309720087,0.003149137095374007,0.0004998590946844075,-0.0037488658605510958,-0.0019788248967716046,1.4628572460442058e-05,-0.0010314265926241804,0.000519494642422286,0.001547156643265325,-0.0014497835855739577,-0.0009062247751148595,-0.0004795983253122262,0.0014558847576189935,0.0006284955451666036,-0.00041352069443065324,0.00036663321907445795,5.682026466561277e-05,-0.0005480008320993975,-0.0011251728986534074,-0.0013517602104130947,-0.0008401598273915725,0.0010243514930324208,0.001590542514036337,-0.0004245098511365807,-0.0016704979816220763,-0.0004840217870629958,0.0006132632390657076,-0.00200123690802248,-0.0002188356584147541,2.7302835908676734e-05,-0.0002791445109825941,0.000652849180487117,-0.0002890634488391011,0.00022866963655074026,0.00034529408750028626,1.9063095892556465e-05,5.507512334250079e-05,-0.000493623936597539,-0.00020866122436026988,-0.00029088268324621065,-0.00031271764142339607,-0.00038181259685605095,-0.000209388064511237,-8.55144642866347e-05,-9.420435743550712e-05,-0.00010776511040939178,-8.512690021509454e-05,-5.2609503266013455e-05,-2.2972173802643057e-05,-9.399904074168741e-06,-3.071599254704068e-06,-1.5907688831401653e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (75,20): np.array([[-3.404961528466379e-05,-4.950506741219865e-05,-0.0011609563265040678,-0.0018601108594173278,-0.0016124091981400885,-0.0015289389621756077,-0.0007965755728612456,0.004378581826158521,0.012224437153159315,0.1693477762073975,-0.005364000422784144,-0.05251603165183157,0.045752774359963826,-0.015819379369913015,-0.025867777554192235,0.031946109936289775,-0.04382462867016388,-0.005770757620697992,-0.054007197472859154,-0.052832709875993256,-0.026976112797434244,-0.05061077419769996,-0.0003413040702335508,-0.01763864158390665,-0.011037932588291841,0.010480839063296917,0.003283644346980059,0.007855611893738196,-0.006377083431407402,-0.00462512331029784,0.01389743900434498,0.003686039875302591,0.001131429283346114,-0.0037334872820241074,0.005283195243005105,0.0032654001607421864,-0.008464334376042645,0.0026065333950231594,-0.0002199104038910046,-0.00026715823101082335,-0.002705956724356685,-0.00015901401777513924,0.002482024687892859,-0.00021063696651060562,-0.0036110084594455846,2.5829177923258727e-05,0.00045159857076303004,0.0033644356073991885,0.0033952628520334664,0.0004323047822742512,0.00022767497851728294,-0.001407334791449241,-0.0014125439918026004,-0.0022670425147106135,-0.0030071432000338193,-0.0021151127500631983,0.0023166250288604025,0.0014959253793400127,-0.0017715964038119089,-0.0007749829761532281,-0.0013695400905579535,0.00040781230269897067,0.0022262609857172464,-0.002493938540130814,0.00012207038074388794,-0.0011013008327271359,-0.000607024004092151,-7.426678745287447e-05,0.00017302691667173332,0.00022153114414797743,0.00031989118893538355,0.0001278883734549017,2.5250606561485193e-05,-6.279003571718808e-05,-0.00013406955595192377,-3.816563268359203e-05,-1.5117789553851325e-05,-2.7711949002198236e-06,5.253749979751914e-06,4.894544928127561e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [8.860791175373116e-07,-0.00016383466959326033,-0.00047064978030592834,-0.0005221379893068691,-0.0004591931032225071,-0.000464990026324805,-0.0005223231986169886,-0.0005132425555931379,-0.0004845458805555425,-0.0004962756010868666,-0.0005691146109438458,-0.0005420568109286984,-0.000462954328822501,-0.0004896907664986022,-0.00034157296906840576,-0.00023003419433883937,-0.0007030171264150848,-0.0028718180229376036,0.031662743400477034,0.01741650172539829,-0.002857996335775591,0.008088252118387325,-0.004443283837198987,0.0008208527047942156,0.0018209425415390522,-0.002286446838535639,-0.005616167242327974,0.00013869883371068887,-0.004168942791354056,-0.0019600296086284205,-0.0002552369296271097,-0.001322808749825198,-0.0032805020658138273,-0.0020786076247810126,-0.0002526160024741482,-0.0025410013819828275,-0.0014568040400170486,0.00044242915771481174,-0.000978322207940961,0.0009429307992193649,-0.0002716731067285153,-0.00022853564181391867,-3.865096210625862e-05,0.00030534826966905097,0.0007520159224051835,-0.0002504873645701824,0.0005032075786134601,0.000602333665372899,-0.0009952540545015744,-0.00012465132307506966,-0.00010769461051657462,-0.0020975690275163365,-0.002081888689494726,0.0012645129472509862,0.00024828014709172425,-0.0012670579382109903,-0.0015672331936081745,0.0010679654667370662,-0.001840400950967633,-0.00011770264323277529,0.0018281487036771976,0.00017005133317364414,0.00012310005575323906,0.0008358874758128662,6.55602669981709e-06,-0.0001412269396992167,-0.0005774549060612613,-0.000623581582944007,-0.0005264286230064619,-0.0005130505487637193,-0.00044195366654469534,-6.689095063044725e-05,-0.00013097193911133299,-4.945140148199656e-05,-3.375757512886573e-05,-4.550564002195338e-06,-2.1689817940407088e-05,-1.6387795065339527e-05,-5.471870051814062e-06,-6.738025517519473e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (75,45): np.array([[-2.2512154183700384e-05,0.000640623499962013,-0.0004991322666246831,-0.00282941012322008,-0.0012279270063005664,0.00072096171972839,-0.00032599448984211817,0.001089583527174136,0.003033641927697534,-0.020719279126368347,0.11518127036679329,-0.20299269370669512,-0.04305783397462542,-0.04405363341720702,-0.01994024014442669,-0.0382713235822255,-0.02719481258731841,-0.0002498683370585122,-0.04013770956628394,-0.014666979305042613,-0.0009742727692923574,0.016870030132971402,-0.03840613026312501,-0.013662559416975842,0.022123458010774998,0.006626537538404702,-0.003548026123210423,-0.0026428462688306976,0.0071539359917546785,0.004793967007265205,-0.008912945514755632,0.00418992850666292,0.009893190770284385,-0.005292812418959398,-0.0042704631795632605,0.003426279520615203,0.0017810280214185944,-0.00271604120709589,-0.0023533505021400614,-0.002716317282749735,0.000501111344357171,-0.003007881764668705,-0.002924450499624983,-0.0004259716213716035,0.0015144236926060103,-0.0004698257418717446,-0.0032561280495423473,0.0005937738257870048,0.0016189698430125541,-4.539215291396354e-06,-0.0003871360340827706,-0.0005301118823594405,-0.00019768271316049444,-0.001240848155652857,-0.0014749055992772596,-0.0011928967125865986,0.0005020255598134842,0.0003465117616046326,0.0007257858665603463,0.0012569623870877925,-0.001209010591233257,-0.003551933213038027,-0.0015416033426536996,-0.0001448764476222759,0.0006408113078978093,0.001185961632911512,-0.0005163213354040167,0.00031748268366215803,-5.308981769541111e-05,-0.00015232200569483564,-0.00019673068316350634,-3.292717259585017e-05,-2.166435060732961e-05,-6.00308068713019e-05,3.170195311937664e-05,3.1146942248454107e-06,-1.966823120605338e-05,-6.739179180483768e-06,-1.8839436859931815e-06,-4.4655610751179733e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-1.5325420716512684e-07,-0.00020956795393265096,-0.0004951622778229485,-0.0005562663438429151,-0.0006233333369199046,-0.000627843923832102,-0.0005222764917532646,-0.000502244804193419,-0.0005726310371088174,-0.0006731802518792086,-0.0006156907339498868,-0.0006580385312108774,-0.0005995494299367157,1.9079225535135954e-06,-0.0002059766596109475,-0.0035948165255451285,0.023713642336915697,0.020178292263796322,-0.018836138534409495,0.008047517660518137,0.002060280196089894,-0.009334757067151755,-0.001523009787307397,0.002088731049395158,-0.00021697278101293782,-0.00692324968044362,-0.00013504972300953862,-0.0016790502279344411,0.002676901089526067,-0.0034545461472037537,-0.004970903764737681,-5.31054663465238e-05,0.002246061572812706,-0.0069859154901133706,-0.004037647799291827,-4.45028594283061e-05,-0.002401243660897142,-0.002149463005040082,-0.0007181765910621894,-0.0011499208142205942,-0.0008380153572345895,-0.0012621852644161336,0.0007523391486371554,0.00012000917029298595,-0.0012347069579182687,0.00010809973535792026,2.4736041018837105e-05,-0.0009267359130010587,0.0004349562715798705,0.0005125882262489253,0.0015243876279822553,0.0008142903304581195,-0.0006940325056920226,-0.0013332604993483647,-0.0006896643414221748,-0.0008787581697914075,-0.0013630711784616435,0.00014912640209509593,-0.0001130013958711949,0.00037185979652480476,0.0014547923233886428,-0.00025009227199646977,3.854408551303633e-05,-0.000349461471439253,0.000534094701213531,-0.001412344390947427,-0.0004115749771645688,-0.00010150655658440531,6.295883425733538e-05,-4.407979251290767e-05,-0.0002224419140829584,-0.00013566538789319153,-0.00013238260308172262,-8.265505135619817e-05,-6.456111004735172e-06,-1.6380138739891238e-05,-1.4917823495517318e-05,-6.087311358827607e-06,-5.609849536272843e-06,-1.6442812589614496e-06,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (90,-45): np.array([[-3.9610125069009994e-05,0.00041933318166927276,-0.0007771267149652373,-0.004310667980761271,-0.005180507813963094,0.00016373331791150861,0.007631591307371328,0.008330561162392445,-0.0006847105348671358,-0.025297312884116747,0.2377130835084638,0.06558711518725566,-0.09279252630874671,0.026217281654723753,-0.06296007477191153,0.01564488685823389,-0.06755283379042952,-0.0012256384748428296,0.0048369653568521515,0.006295079543205549,0.00896194591209613,-0.008766749729239326,0.016804056858798034,-0.00540682515594374,0.010123700790632448,-0.0037474243097325717,-0.005117639050998549,0.005227763355667531,-0.008079487874372657,-0.0020249712813281474,-0.003098057199697057,-0.001506516083069322,0.0009988354217991976,-0.003617580570547731,0.0015627791350546394,0.0032096757546351815,0.0011198124967909423,-0.0038146638747545247,-0.002525109210615686,0.0025742838100462485,0.0027183724866429305,-0.002078586040328601,-0.010988842286457313,-0.00858006315882585,0.0028658031291049673,0.004948199045748185,0.0009863572801536821,0.00021932437487001344,0.0010890376331490605,0.0009490451919626974,-0.001627119953450239,-0.0006058469425964206,0.00032045056379981034,-0.0009755030449582702,-0.0009932214794716062,0.0009070679731335205,0.0011000036283432487,0.0005659241582011753,0.0001245196583050377,0.0007508207329970934,-0.00016977205556444985,0.0011782250323130212,0.0016337008163422328,0.0005607121381505282,-0.0011126483850856194,-0.0012330216960655003,-0.00043114899757824436,-0.0005996808892110677,-0.0007120315562965849,-0.0006858706050549859,-0.0001706528052592153,1.32715027347661e-08,-1.1720844889421349e-05,-8.363224233754023e-05,6.039385748041496e-05,0.00016229328786457578,9.150429077479123e-05,4.876774553872896e-06,-1.129790880975998e-05,-2.2417633546964433e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [4.69126433341577e-07,-0.00020005304315338793,-0.0005412968077550985,-0.0006109699749833731,-0.0005981828804430728,-0.0005885998588136265,-0.0006103095876112957,-0.0006293734282443714,-0.0005763896301164025,-0.0006271562251508029,-0.0006968084182345877,-0.0006057817108593427,-0.0005084527232387049,-0.0006167575963244356,-0.0007169624141393877,-0.0006500621319635373,0.012074304900051465,0.020885611757656204,0.013349063204603277,0.003465564558009239,0.0007647659255342949,0.006747615394983551,0.00023193878022946933,-0.005730214280035718,0.0026332448340297466,-0.000246126756495593,0.0015288066319258636,-0.0035211412621690063,0.0037931633280733667,-0.0027029196133276042,-0.002893939495369717,-0.00046937345142434586,-0.00042820657972082526,-0.0013836499984629293,-0.0009581062140220967,-0.0009301609654700237,0.0002942254948362132,0.0008008498158869575,-0.00048554961768580484,0.00029789703764030806,0.0008508416919227393,6.924777731225078e-05,0.0015328687111539733,0.00025723773424841316,-0.00023581515384573073,-0.0009284514060638877,-0.001604844520437356,-0.0014022932691594324,0.0007840018671853003,-0.00029422576289597806,-0.0006225676346243651,-0.0013595704705745958,-0.0017671404598070457,-0.001453205974925689,-0.000982061027332722,-0.0004864557974617994,-0.0002087674489009181,-0.0005946178103675897,-0.0009367605753100441,-0.0011484331417646963,-0.0005689765336120442,-0.0005230869703869125,-0.00044367937175351766,-0.00025875131330427016,0.0004725539497615376,0.00011949637215566242,-0.00016812668695236218,-0.00011261141331354189,-0.0001008634698470083,4.113118764204298e-05,8.082161659858844e-05,2.9026935944354967e-05,-3.768580680066142e-05,-3.354075097460261e-05,-6.376647676899372e-05,-6.0508222495401305e-05,-3.121163033020967e-05,-1.0232507627229764e-05,4.6212692365429603e-07,6.213629652186093e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (90,-20): np.array([[-5.0027448099916324e-05,-0.00023875080739591158,8.945523740008043e-05,-0.000379899033470738,0.0004258978160740542,-0.002061403056033763,-0.008824530505285552,-0.002291170943012074,0.03733226468136775,0.37429385052495323,-0.040289768241262236,0.00199477733689664,0.2281583986084717,-0.12800840755285584,0.019148858247432704,0.003927438972060827,-0.006436485621497708,-0.01549834370702801,-0.030435288521417386,-0.0014864141845936402,-0.010149767064321007,0.010889419125840311,-0.016051666925810847,0.003525935086056891,0.011421285257506077,-0.008928062257646713,-0.0003948764545538689,-0.003948504022125322,0.005294328051581201,0.0010646525778338154,-0.005771945360612053,0.007796644299324621,-0.0033338865301864185,0.00048664134217029926,-0.0011238314339631222,-0.0019145554127367652,0.0011900677116830247,-0.006817546333248579,-0.0056204476603555105,-0.0029689464406761944,3.9916211527771565e-05,0.0009372933956149235,0.0012914395360551542,0.0013559421948648818,0.0014139079282451622,-0.0015178033655522643,-0.0008923827643104445,-0.0012483726444379502,-0.00027139539145727665,0.0003089101698738161,0.001560811842629901,-0.0006773487330988634,-0.0003934701653890028,0.0003771637873182217,-0.0022633691888982936,-0.0010017199819485345,0.0018975876060061082,-0.0012002208521707846,-0.0017846275452096248,-5.385099261895004e-05,9.509120046699636e-05,-0.0006421197715153823,0.00030125931346561383,-0.0003333607394414576,0.0007436130629398996,0.0005077636843079347,-1.1417263001482618e-05,-3.6507913488893274e-05,-0.0001557382398520356,-0.0002000376184313003,6.145382790641986e-05,1.8131759230391555e-05,6.0462382324034e-05,8.469699618390814e-05,-9.288651949197842e-06,-4.2312262938810676e-05,-1.9954401905999264e-05,-1.0559700258251037e-05,-6.313567929927796e-06,-1.4279298948146004e-06,0.0,0.0,0.0,0.0,0.0,0.0,],
    [7.992372260019019e-07,-0.000178335363982483,-0.0004742194676467353,-0.0005270717969937855,-0.0004949431059466477,-0.0005304282996041082,-0.0005318856604266249,-0.00048753266301269395,-0.0004870339605163701,-0.0005529316633694852,-0.0005422893149141889,-0.0004990638094951825,-0.00046932046938613945,-0.0005590281251295413,-0.0006427890268569443,-0.0005349839239642836,-0.0006247814340353475,0.0008075299562177997,0.025918410703580547,0.00950036997482019,0.0016870677448880844,0.0014268647525418226,-0.000408646126091691,0.009558137700145282,0.0019498800336477263,-0.0035613707505356093,0.001552841714870041,0.0008427387653073537,0.0007662225439892134,-0.002810862596200426,-0.002700733246113845,-0.002319005999741268,-0.0005321623385263669,-0.00025693450105861626,-0.00045004339021450964,0.0004042260343984978,-0.000708541295165639,-0.001277679980888464,-0.0011612657768121425,-0.00020551248770636881,-0.0006954515046935978,-0.0005931245064611369,-0.0005289176439851489,-0.0009430487980599948,-0.000642958924962366,0.0004400340235421834,-8.510446004869336e-06,-0.0007418135100812793,-0.0014838252008468943,-8.072709830240218e-05,-0.0005531771783435294,-0.0004274895619457314,-0.0007364511015310375,-3.487995412974066e-05,0.00012804229750899553,0.001150233576635019,0.000681920877573915,4.0114857819617215e-05,1.8426690889233987e-06,0.00038121657169921236,-0.0003637853833929664,-7.838657609484181e-05,-0.00014171718370542476,-0.0006857845562104019,-0.00040655245374456375,-0.00019204712826857088,-0.000490067369636972,-0.0003642894854849834,-0.00033206799359107613,-0.00025495008458386654,-0.0001936290219767853,-0.00016194314764253937,-0.00010810882536110328,-3.863602587924609e-05,-2.565810969863893e-05,-1.2728082341379248e-05,-2.4802492917094055e-06,-4.355313535187634e-06,-3.6283175326271594e-06,-9.758048316588905e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (90,0): np.array([[1.4874623455333892e-05,-0.00016873089352844792,-0.001577452939657328,-0.0025530715685598924,0.00023953652371399396,0.003281187290760555,0.0008629773386712653,-0.0038198157538937567,-0.013949145377224657,0.3875245102562477,-0.1988375876879218,0.015309287652479252,-0.012606776048287329,-0.03824966853051308,-0.03490258125451122,-0.01837160238151081,0.01622805821514921,0.03744303754510311,-0.013504591607154316,-0.026098368082432086,0.011149356199261013,0.016354521354418672,0.005843970394237141,0.003493934703112867,0.009684623679560606,0.011514487915207866,-0.005735661463492332,-0.0016599701721185937,0.0020800671409137964,0.003874727684729237,-0.002983759028409511,-0.0034773845113990278,0.005826886546235449,0.0013582351953907628,-0.0005889494576851632,-0.0005463618690605857,-0.0017043832110196625,-0.0001993298226469787,-0.0003337988698349647,0.0008056460890193877,0.0025459930692866076,-0.0013335160307363246,-0.0031278964387735733,-0.0038834210273771853,-0.00044026749096200975,-0.0001883429433527973,-0.0004547502828371754,0.00038337461800931773,-0.003254265187625309,0.0006895822094997436,-0.0014815920266718922,-0.0006250563596192821,0.0019599388745388177,-0.0014726887031595096,0.00028046187283519737,-0.001489368386985585,0.00043766985281039957,0.0016699425657522617,-0.003227741840907697,-0.0006313261425964655,-0.0008919974900896962,-0.00025046765269896837,0.0006515338480910779,0.000547310808664129,-0.0008985296589908684,5.719374677281384e-05,8.912778441629084e-05,2.3100025528477707e-05,8.733478653243443e-05,0.00010006446483641695,0.0001370668402424641,-6.575400546901069e-07,-5.1553560924588617e-05,-3.3122728373340604e-05,2.601249783031492e-06,2.987466634982047e-06,-1.2033391411037541e-05,-4.014858319580547e-06,5.643038314828173e-07,2.4938107089982024e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-5.618953478078317e-07,-0.0001687087051827834,-0.00045348427343038444,-0.000540766671945948,-0.0005321928292003089,-0.0005107614846857683,-0.0004888625941630826,-0.0005158600452914941,-0.0005327856011657998,-0.0005102921508912808,-0.0004984786345492717,-0.0005346584712583069,-0.0005574889461488778,-0.0004510315535565461,-0.00036250726646443665,-0.00045033478859419937,-0.0008285977396006076,-0.00033846551127071704,0.013194271601386087,0.025336033378737147,0.01842676570646106,0.004954077437582502,-0.010051796148124558,0.006111151782320818,-3.338097844653105e-05,-0.0010416181536580518,-0.0021313111408875905,-0.001418929022965906,0.002948493718359704,0.0002611798968206986,-0.0035572771110046197,-0.0026877169120753355,-0.0018955465653690802,-0.0011863970571856595,-0.00245338412716533,-0.0010334350335207583,-0.00019531171395765487,4.997669507748117e-06,-0.001562088396210706,-0.000985034676521529,-0.0010156297142498887,0.0005681919933244932,0.002250855237401656,0.0017047662266533325,-0.0020500721901359578,-0.0021069208729438773,-0.0008946816900734282,0.00015018009469294001,0.0006711919852910974,-0.00037817091218474993,0.0008067354958123082,0.0016575653044792263,-0.0004180249006870178,-0.0018497048474207071,-0.0008448224232035683,-0.0018323149515474687,-0.0006131523411662795,-0.0007080165003292122,-0.00037772503490075257,2.881401262964801e-05,7.46833479575529e-05,0.0003969858429743164,0.00026067389288402594,0.00040345158489152127,2.6714591026877618e-05,-0.00034398142014216307,-0.0003639759930121734,-0.00047517002157948664,-0.00019902650076161416,-0.00023198055679008218,-0.00018598970623503577,-0.00016914432334172932,-8.876612251376238e-05,-6.714194358862028e-05,-7.81612744205246e-05,-6.465562650655113e-05,-3.4560683769034e-05,-1.0169575086893483e-05,-1.964356252233429e-06,-2.016339392978286e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (90,20): np.array([[8.860510479698824e-06,-9.76763800128745e-05,-0.002049484573699351,-0.0019207190531329932,0.00041738101038939527,-0.00031827956414728356,-0.0022089728023858926,0.002877393134640361,0.005001785061980801,0.2320847777729927,-0.23830551613775752,0.025312333631166896,-0.0881026801790591,-0.034279524528675116,-0.028656649191846913,-0.0061483988080984564,0.04299200012594756,-0.04225112434684756,-0.028056587861862093,0.008325475289053873,0.03815302098432169,-0.019237283210044975,0.0005861006522498335,0.020560858421192894,-0.011968918212792709,0.012431521390894616,-0.00037997377010334305,0.007281883671385935,0.0024078798349978147,-0.009025148817243218,0.00415833402844803,0.0028481050205141916,0.008534971557976306,-0.003568650945473506,-0.0012323954035150556,0.004323059265332786,-0.00574811686058685,0.0034666620822847165,-8.560062372141422e-05,-0.0010434685097163827,0.0002961197515806037,-0.003399062325462805,0.003860114998250029,-0.00043488139575442486,-0.00329751431923757,0.0009756783072531161,-0.000984365215245042,-0.00101823663461818,0.0025873303518364078,0.001492324281570744,-0.0012826050417659336,-0.003753173691810111,-0.0018941398613086967,-0.0008101888606612987,-0.0026240023921437912,-0.00025543370538491314,0.0020715893498553005,0.00038869408351678716,-0.001713782606777865,-0.0014695723960501928,0.0006046687304083896,0.0011277776967527394,0.0012269813644588173,-0.002381469390697054,0.0020895280046455385,-0.001330607331686107,-0.0001907740691049903,0.0001398577155611498,-5.578362664986976e-05,0.0004687789078873365,0.00025391627647883324,0.00011414398363738137,-2.270417799960872e-05,-0.00012469326548299734,-0.00010283154972392927,-2.8654882676315937e-05,6.5263893268467334e-06,-2.700454540137049e-06,7.09632979190256e-06,3.8127428339496426e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [-4.930245143693944e-07,-0.0001670688999923613,-0.00043776484626835015,-0.0005157724360805879,-0.000509936963528584,-0.000487352458717489,-0.0004876643081896105,-0.000502057269865079,-0.0005559034264821582,-0.0005183440055308271,-0.000462637643318017,-0.0005462649050051671,-0.000606945840606258,-0.00046302303143715243,-0.00044329087747540494,-0.0005155558306234845,1.3317551537541053e-05,-8.510773654428666e-05,0.0017485651195964275,0.05382875309925131,-0.002273892658305501,-0.01935895613537127,0.018719250598130337,-0.0044194581963217655,-0.005152419590906376,-0.0009168574826404832,0.004894989689680981,-0.0024592683570929595,-0.0048396506160793265,-0.0011368789697200517,-0.002872695263255205,-0.0034224772573890564,-0.000998884852666933,-0.0034070650712998756,-0.0020973203632775167,-0.0010982998215212261,-0.00267568069394786,-0.00026830090339856897,0.0011518222119323678,-0.0014136004418253353,0.0014639281608404089,0.0007438186475150971,-0.0005144066300986046,-0.00038707706487848355,0.0006996930586638648,0.001247498016433287,-0.000597132887049683,0.0005046132856178509,-0.00012596467421267045,-0.0006000301073226799,-0.0014158304020438466,0.0009844614102797915,0.0005838022566543035,-0.0012026043355877856,-0.00025142503285533526,0.0010161479333124248,0.0010865532944385023,-0.0012275689400360829,0.000862164854407875,0.0004938370076558642,-0.00046116899657589204,0.00018597739136042636,0.0016201790849551879,0.0009125339670184409,-0.0001156762520355459,-0.0005738580364234771,-0.0006038995685665305,-0.0006481561728335016,-0.0005230376240311778,-0.0005984483356974574,-0.0004781887094541109,-0.00016423003775523052,4.757616299361434e-05,-7.228725097440748e-05,2.4653804963584686e-05,-3.446285783665509e-05,-2.593921662882302e-05,-1.430857775344346e-05,-5.750632310099764e-06,-5.044004567996973e-07,0.0,0.0,0.0,0.0,0.0,0.0,]]),
    (90,45): np.array([[2.684099203694466e-05,4.0382547396186424e-05,-0.0018054157258479934,-0.0028627334767266023,-0.00039511961034860833,0.001229481589635352,-0.0018867789035812207,-0.004435066083068927,-0.0055070166311615085,-0.02683995481434705,0.1520410983864356,-0.2768685345023421,-0.03671374019623531,-0.04198243672978389,-0.02803763049774969,-0.025675996473650042,-0.02130791109625555,0.011800508374172734,-0.01720728813514684,-0.012889907866108108,0.0001914862348079571,0.03107400807757766,-0.01825926287026003,-0.015709672467988517,0.017713424510826577,0.005516528167333409,-0.0020477469584332638,-0.0021942108092843295,-0.005510143461402952,0.0046631856030259515,-0.006101978483766135,-0.0056387057284067845,0.0036954390156034187,0.008053624300931024,-0.0020504031818547926,-0.0022746919804578604,0.005215099388426804,0.0005509681254168879,-0.0010874187674291618,-0.0017499782441756213,0.0001553992576823786,-0.0004786142548622188,-0.0025271873291477336,-0.0021461682615556716,0.0015980881368440735,-0.0017975743365232753,-0.001383370689294855,-0.0008713114584381138,-0.000749997255624636,0.0012319035977113037,-0.002041528957207527,0.0004679576816755616,-0.0010562584247624815,-0.0017072114133063116,-0.0016574927568637973,-0.0001873161244037513,0.0007465653531076329,-0.0007582244163631964,0.0016230374102982133,0.0034520166696453713,0.0010922080082061208,-0.004479111473343615,-0.0036441658462524062,0.0012886735992955899,-0.0005148096958107038,-0.0005255213962925439,-0.00011925809127172309,0.00026573618570673415,-0.0006370477090883617,-0.0003998579922084998,-7.46462718468349e-06,1.4735103241812049e-05,-5.057410367105913e-05,-1.1535961230687119e-05,1.1694757888701526e-05,2.2363034724861762e-05,-2.0651369481791926e-05,-1.727953570050735e-05,-3.97428240882867e-06,-6.605118488674351e-07,0.0,0.0,0.0,0.0,0.0,0.0,],
    [9.628877912747798e-07,-0.00017604799454815212,-0.000542874256967256,-0.000560444991947674,-0.0004783489355284033,-0.0004982031994134892,-0.000610336816475366,-0.0006532532270240174,-0.0005404139220450384,-0.0004520471923259537,-0.0005928140567451595,-0.0007176050044116674,-0.0003941088967681847,-0.0003229164737864264,-0.0006939346814514059,-8.073569016500321e-05,0.017846298186304007,0.06720926707860272,-0.01883172790772754,0.001078780159848307,0.020143064279963332,0.003358028222699116,0.004278724186648377,0.0067460205850727465,-0.002603891284662103,0.00520631336485299,0.0070012072883916835,-0.002423441177439854,-0.00258121635786754,0.006565545398591792,-0.003437754135306457,-0.003877157696301251,0.002058631169180518,-0.003771573594303139,-0.004883498998175336,0.0035286938508335107,-0.0031363700969954697,-0.0010318266008669814,0.0005140488361415197,-0.0032721941054900503,0.0005215035393401227,-0.0002844826553588291,-0.0021612388946330827,0.001280413369109034,-0.0006016284066975229,-0.0016317995832629204,0.0008722837635436661,0.0005568481892043203,-0.0017974793706313488,0.0008556452878703682,-0.0003523387169401781,5.812015603015767e-05,-0.0009219290694172029,-0.0016980940518646133,-0.0007220436096348925,-0.00048729377974348194,-0.001623112902150833,-0.00031982531497306557,4.6295457438690174e-05,-0.0006357595962347161,0.0008236862464018564,0.0011622570351072468,-0.0010431685125977573,-0.0007392545931083031,0.00012105656357940809,0.0010300911449489093,-0.0012827820841506943,-0.0007342841438371429,0.00031133317701015985,0.00018352535981277943,-0.00017438757856052641,-6.981006164461088e-05,-5.4306139660780985e-05,-0.00011345137395380956,-2.8344361260996278e-05,-4.242042822540783e-06,-1.8806072668672745e-05,-8.851631856359591e-06,-6.042386873380416e-06,-1.6887835263643188e-06,0.0,0.0,0.0,0.0,0.0,0.0,]])
  }

  return hrtfs[angle]

def convolve_sound(hrtf, sample):
    """
    Convolves the BRIR with the window, and cuts result at the size of the window length.
    window: the window of the sample that should be convolved.

    """
    return torchaudio.functional.convolve(sample.repeat([2,1]), torch.tensor(hrtf, dtype=torch.float))[:,:sample.shape[1]]


### Train model:

In [ ]:
batch_size = 256
writer = SummaryWriter()
total_test_loss = 100000
training_losses = list()
test_losses = list()
test_accuracies = list()

for epoch in range(50):  # maximum amount of iterations
    rnn.train()
    training_loss = 0.0
    for idx in tqdm(range(0, len(train_data), batch_size)):
        batch = train_data[idx:idx+batch_size]
        spatial_audios = torch.zeros(batch_size, 2, n_observations, dtype=torch.float32)
        labels = torch.zeros(batch_size, dtype=torch.long)

        for j, sample in enumerate(batch):
            az, el, filename = sample                  # Split sample into label and filename
            hrtf = torch.tensor(open_hrtf(int(az), int(el)), dtype=torch.float, device=device)         # Open HRTF
            audio, fs = torchaudio.load("/content/samples_500ms/"+filename, format='flac')  # Open audio sample
            audio = audio.to(device)
            spatial_audio = convolve_sound(hrtf[:,::2], audio[:,::2]).to(device)  # Convolve HRTF and audio sample
            spatial_audios[j] = spatial_audio#.flatten()
            labels[j] = az_angles.index(az)*len(el_angles) + el_angles.index(el)

        spatial_audios = spatial_audios.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = rnn(spatial_audios)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=1)
        training_loss += loss.item()

    total_training_loss = training_loss/(len(train_data)//batch_size)
    print("Epoch {}:\n\ttraining loss: {:.3f}".format(epoch, total_training_loss))
    training_losses.append(total_training_loss)
    writer.add_scalar('Loss/train', total_training_loss, epoch)

    rnn.eval()
    correct = 0
    total = 0
    true_labels = torch.zeros(64000)
    predicted_labels = torch.zeros(64000)

    with torch.no_grad():
        test_loss = 0.0
        for i, sample in tqdm(enumerate(val_data)):
            az, el, filename = sample                  # Split sample into label and filename
            hrtf = torch.tensor(open_hrtf(int(az), int(el)), dtype=torch.float, device=device)         # Open HRTF
            audio, fs = torchaudio.load("all_samples_500ms/"+filename, format='flac')  # Open audio sample
            audio = audio.to(device)
            spatial_audio = convolve_sound(hrtf[:,::2], audio[:,::2]).to(device)  # Convolve HRTF and audio sample
            # print(spatial_audio.shape)
            output = rnn(spatial_audio.unsqueeze(0))
            label = az_angles.index(az)*len(el_angles) + el_angles.index(el)
            loss = criterion(output, torch.tensor([label], device=device))
            test_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            # print(predicted.shape)
            total += 1
            correct += int(int(predicted) == label)
            true_labels[i] = label
            predicted_labels[i] = predicted

    test_accuracy = 100 * correct/total
    print("\ttest accuracy: {:,.2f}".format(test_accuracy))
    writer.add_scalar('Accuracy/test', test_accuracy, epoch)
    test_accuracies.append(test_accuracy)

    new_total_test_loss = test_loss/len(val_data)
    print("\ttest loss: {:,.3f}\n".format(new_total_test_loss))
    if new_total_test_loss > total_test_loss:
        test_losses.append(new_total_test_loss)
        break
    else:
        torch.save(rnn.state_dict(), '/content/drive/My Drive/HRTF_weights.pth')
    writer.add_scalar('Loss/test', new_total_test_loss, epoch)
    test_losses.append(new_total_test_loss)
    total_test_loss = new_total_test_loss


## Test once more on the best variables:

In [ ]:
rnn.load_state_dict(torch.load('/content/drive/My Drive/HRTF_weights.pth'))

In [ ]:
rnn.eval()
correct = 0
total = 0
true_labels = torch.zeros(492460)
true_az = torch.zeros(492460)
true_el = torch.zeros(492460)
predicted_labels = torch.zeros(492460)
predicted_az = torch.zeros(492460)
predicted_el = torch.zeros(492460)

with torch.no_grad():
    for i, sample in tqdm(enumerate(test_data)):
        az, el, filename = sample                  # Split sample into label and filename
        hrtf = open_hrtf(int(az), int(el))         # Open HRTF
        audio, fs = torchaudio.load("samples_500ms/"+filename, format='flac')  # Open audio sample
        spatial_audio = convolve_sound(hrtf[:,::2], audio[:,::2]).to(device)  # Convolve HRTF and audio sample
        # print(spatial_audio.shape)
        output = rnn(spatial_audio.unsqueeze(0))
        label = az_angles.index(az)*len(el_angles) + el_angles.index(el)
        az_label = az_angles.index(az)
        el_label = el_angles.index(el)
        _, predicted = torch.max(output.data, 1)
        # print(predicted.shape)
        total += 1
        correct += int(int(predicted) == label)
        true_labels[i] = label
        predicted_labels[i] = predicted
        true_az[i] = az_label
        predicted_az[i] = predicted // len(el_angles)
        true_el[i] = el_label
        predicted_el[i] = predicted % len(el_angles)

test_accuracy = 100* correct/total
print(test_accuracy)

## Statistics:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(true_labels, predicted_labels)
cm = 100 * cm/cm.sum(axis=1)
fig, ax = plt.subplots(figsize=(50,50))
labels = ["(" + az + ",\n" + el + ")" for el in el_angles for az in az_angles]
matrix = ConfusionMatrixDisplay(cm, display_labels=labels)
matrix.plot(ax=ax)
plt.title("Confusion matrix for anechoic data")
plt.show()
plt.savefig("azelhrtf.svg")

In [ ]:
cmaz = confusion_matrix(true_az, predicted_az)
cmaz = 100 * cmaz/cmaz.sum(axis=1)
fig, ax = plt.subplots(figsize=(10,10))
matrix = ConfusionMatrixDisplay(cmaz, display_labels=az_angles)
matrix.plot(ax=ax)
plt.title("Confusion matrix for anechoic data for elevation angles combined")
plt.show()
plt.savefig("azhrtf.svg")

In [ ]:
cmel = confusion_matrix(true_el, predicted_el)
cmel = 100 * cmel/cmel.sum(axis=1)
fig, ax = plt.subplots(figsize=(5,5))
matrix = ConfusionMatrixDisplay(cmel)
matrix.plot(ax=ax)
plt.title("Confusion matrix for anechoic data for\nelevation angles combined")
plt.show()
plt.savefig("elhrtf.svg")